In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
import copy

In [2]:
#Obtain list of top 150 prospects
bigBoard = pd.read_csv("propsectlist.csv")
prospects = bigBoard['NAME'].head(150).tolist()
prospects

['Marvin Harrison Jr.',
 'Caleb Williams',
 'Joe Alt',
 'Malik Nabers',
 'Rome Odunze',
 'JC Latham',
 'Drake Maye',
 'Brock Bowers',
 'Olumuyiwa Fashanu',
 'Dallas Turner',
 'Laiatu Latu',
 'Taliese Fuaga',
 'Kool-Aid McKinstry',
 'Nate Wiggins',
 "Jer'Zhan Newton",
 'Chop Robinson',
 'Ennis Rakestraw Jr.',
 'Keon Coleman',
 'Jared Verse',
 'Byron Murphy II',
 'Brian Thomas Jr.',
 'Troy Fautanu',
 "Ja'Tavion Sanders",
 'Tyler Nubin',
 'Jayden Daniels',
 'Tyler Guyton',
 'Jackson Powers-Johnson',
 'Quinyon Mitchell',
 "T'Vondre Sweat",
 'Kamren Kinchens',
 'Amarius Mims',
 'Terrion Arnold',
 'Graham Barton',
 'Cooper DeJean',
 'Kamari Lassiter',
 'Junior Colson',
 'Kingsley Suamataia',
 'Cedric Gray',
 'Mekhi Wingo',
 'Payton Wilson',
 'Dominick Puni',
 'Chris Braswell',
 'Darius Robinson',
 'Blake Fisher',
 'Patrick Paul',
 'Kalen King',
 'Leonard Taylor III',
 'Zach Frazier',
 'Christian Haynes',
 'Beaux Limmer',
 'Austin Booker',
 'Christian Mahogany',
 'Hunter Nourzad',
 'Adisa Isa

In [3]:
#Obtain the college data of the top QB prospects in the upcoming draft and average their stats
college_passing2023 = pd.read_csv("collegeQBpassing_summary2023.csv")
college_passing2023 = college_passing2023.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
college_passing2022 = pd.read_csv("collegeQBpassing_summary2022.csv")
college_passing2022 = college_passing2022.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
qb_testing_data = college_passing2023.copy()
testing22 = college_passing2022.copy()
qb_testing_data = qb_testing_data[qb_testing_data['player'].isin(prospects)]
testing22 = testing22[testing22['player'].isin(prospects)]
testing22 = testing22[testing22['position'] == 'QB']
qb_testing_data = qb_testing_data[qb_testing_data['position'] == 'QB']
qb_testing_data = qb_testing_data.sort_values(by='player')
testing22 = testing22.sort_values(by='player')
testing22.reset_index(drop=True, inplace=True)
qb_testing_data.reset_index(drop=True, inplace=True)
qb_testing_data['accuracy_percent'] = (qb_testing_data['accuracy_percent'] + testing22['accuracy_percent']) / 2
qb_testing_data['avg_depth_of_target'] = (qb_testing_data['avg_depth_of_target'] + testing22['avg_depth_of_target']) / 2
qb_testing_data['pressure_to_sack_rate'] = (qb_testing_data['pressure_to_sack_rate'] + testing22['pressure_to_sack_rate']) / 2
qb_testing_data['twp_rate'] = (qb_testing_data['twp_rate'] + testing22['twp_rate']) / 2
qb_testing_data['ypa'] = (qb_testing_data['ypa'] + testing22['ypa']) / 2
qb_testing_data['attempts'] = (qb_testing_data['attempts'] + testing22['attempts'])/ 2
qb_testing_data['attempts'] = np.ceil(qb_testing_data['attempts'])
qb_testing_data['attempts'] = qb_testing_data['attempts'].astype(int)
qb_testing_data['dropbacks'] = (qb_testing_data['dropbacks'] + testing22['dropbacks'])/ 2
qb_testing_data['dropbacks'] = np.ceil(qb_testing_data['dropbacks'])
qb_testing_data['dropbacks'] = qb_testing_data['dropbacks'].astype(int)
qb_testing_data['passing_snaps'] = (qb_testing_data['passing_snaps'] + testing22['passing_snaps'])/ 2
qb_testing_data['passing_snaps'] = np.ceil(qb_testing_data['passing_snaps'])
qb_testing_data['passing_snaps'] = qb_testing_data['passing_snaps'].astype(int)
qb_testing_data['scrambles'] = (qb_testing_data['scrambles'] + testing22['scrambles'])/ 2
qb_testing_data['scrambles'] = np.ceil(qb_testing_data['scrambles'])
qb_testing_data['scrambles'] = qb_testing_data['scrambles'].astype(int)
qb_testing_data['yards'] = (qb_testing_data['yards'] + testing22['yards'])/ 2
qb_testing_data['yards'] = np.ceil(qb_testing_data['yards'])
qb_testing_data['yards'] = qb_testing_data['yards'].astype(int)
qb_testing_data

,player,position,team_name,accuracy_percent,attempts,avg_depth_of_target,dropbacks,passing_snaps,pressure_to_sack_rate,scrambles,twp_rate,yards,ypa
0,Bo Nix,QB,OREGON,84.25,388,7.30,417,439,7.60,21,1.40,3618,9.35
1,Caleb Williams,QB,USC,76.80,402,9.45,460,500,19.10,29,2.60,3682,9.20
2,Drake Maye,QB,N CAROLINA,75.90,432,10.65,528,556,19.35,60,1.85,3714,8.60
3,J.J. McCarthy,QB,MICHIGAN,77.35,264,10.20,292,303,13.70,16,2.85,2352,8.95
4,Jayden Daniels,QB,LSU,78.70,340,9.30,432,456,25.65,61,1.10,3189,9.45
5,Joe Milton III,QB,TENNESSEE,70.55,204,14.95,221,233,9.80,8,2.55,1765,10.60
6,Michael Penix Jr.,QB,WASHINGTON,74.55,466,10.65,482,513,5.95,10,1.50,4128,8.90
7,Spencer Rattler,QB,S CAROLINA,78.95,380,7.50,456,478,19.90,36,3.25,2986,7.85


In [4]:
#Obtain 2023 passing data for the NFL's QBs. Include Kyler Murray who missed most of the 2023 season due to injury.
NFL_passing2023 = pd.read_csv("NFLQBpassing_summary2023.csv")
NFL_passing2023 = NFL_passing2023[NFL_passing2023['dropbacks'] >= 356]
NFL_passing2023 = NFL_passing2023.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
qbs = NFL_passing2023['player'].tolist()
qbs.append('Kyler Murray')

In [5]:
#Obtain college passing data for current NFL QBs from 2015-2022. 
college_passing2015 = pd.read_csv("collegeQBpassing_summary2015.csv")
college_passing2015 = college_passing2015[college_passing2015['player'].isin(qbs)]
college_passing2015 = college_passing2015[college_passing2015['dropbacks'] >= 328]
college_passing2015 = college_passing2015.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
college_passing2015 = college_passing2015.sort_values(by='player')
college_passing2015.reset_index(drop=True, inplace=True)
college_passing2016 = pd.read_csv("collegeQBpassing_summary2016.csv")
college_passing2016 = college_passing2016[college_passing2016['player'].isin(qbs)]
college_passing2016 = college_passing2016[college_passing2016['dropbacks'] >= 334]
college_passing2016 = college_passing2016.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
college_passing2016 = college_passing2016.sort_values(by='player')
college_passing2016.reset_index(drop=True, inplace=True)
college_passing2017 = pd.read_csv("collegeQBpassing_summary2017.csv")
college_passing2017 = college_passing2017[college_passing2017['player'].isin(qbs)]
college_passing2017 = college_passing2017[college_passing2017['dropbacks'] >= 300]
college_passing2017 = college_passing2017.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
college_passing2017 = college_passing2017.sort_values(by='player')
college_passing2017.reset_index(drop=True, inplace=True)
college_passing2018 = pd.read_csv("collegeQBpassing_summary2018.csv")
college_passing2018 = college_passing2018[college_passing2018['player'].isin(qbs)]
college_passing2018 = college_passing2018[college_passing2018['dropbacks'] >= 326]
college_passing2018 = college_passing2018.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
college_passing2018 = college_passing2018.sort_values(by='player')
college_passing2018.reset_index(drop=True, inplace=True)
college_passing2019 = pd.read_csv("collegeQBpassing_summary2019.csv")
college_passing2019 = college_passing2019[college_passing2019['player'].isin(qbs)]
college_passing2019 = college_passing2019[college_passing2019['dropbacks'] >= 348]
college_passing2019 = college_passing2019.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
college_passing2019 = college_passing2019.sort_values(by='player')
college_passing2019.reset_index(drop=True, inplace=True)
college_passing2021 = pd.read_csv("collegeQBpassing_summary2021.csv")
college_passing2021 = college_passing2021[college_passing2021['player'].isin(qbs)]
college_passing2021 = college_passing2021[college_passing2021['dropbacks'] >= 328]
college_passing2021 = college_passing2021.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
college_passing2021 = college_passing2021.sort_values(by='player')
college_passing2021.reset_index(drop=True, inplace=True)
college_passing2022 = college_passing2022[college_passing2022['player'].isin(qbs)]
college_passing2022 = college_passing2022[college_passing2022['dropbacks'] >= 304]
college_passing2022 = college_passing2022.sort_values(by='player')
college_passing2022.reset_index(drop=True, inplace=True)

In [6]:
#Average each QB's passing data
tempdf1 = pd.merge(college_passing2016, college_passing2015, on='player', how='outer', suffixes=('_1', '_2'))
tempdf2 = pd.merge(college_passing2017, college_passing2018, on='player', how='outer', suffixes=('_3', '_4'))
tempdf3 = pd.merge(college_passing2021, college_passing2019, on='player', how='outer', suffixes=('_5', '_6'))
tempdf4 = pd.merge(college_passing2022, college_passing2021,  on='player', how='outer', suffixes=('_6', '_7'))
tempdf3['accuracy_percent_5'].fillna(tempdf3['accuracy_percent_6'], inplace=True)
tempdf4['accuracy_percent_7'].fillna(tempdf4['accuracy_percent_6'], inplace=True)
tempdf3['attempts_5'].fillna(tempdf3['attempts_6'], inplace=True)
tempdf4['attempts_7'].fillna(tempdf4['attempts_6'], inplace=True)
tempdf3['avg_depth_of_target_5'].fillna(tempdf3['avg_depth_of_target_6'], inplace=True)
tempdf4['avg_depth_of_target_7'].fillna(tempdf4['avg_depth_of_target_6'], inplace=True)
tempdf3['dropbacks_5'].fillna(tempdf3['dropbacks_6'], inplace=True)
tempdf4['dropbacks_7'].fillna(tempdf4['dropbacks_6'], inplace=True)
tempdf3['passing_snaps_5'].fillna(tempdf3['passing_snaps_6'], inplace=True)
tempdf4['passing_snaps_7'].fillna(tempdf4['passing_snaps_6'], inplace=True)
tempdf3['pressure_to_sack_rate_5'].fillna(tempdf3['pressure_to_sack_rate_6'], inplace=True)
tempdf4['pressure_to_sack_rate_7'].fillna(tempdf4['pressure_to_sack_rate_6'], inplace=True)
tempdf3['scrambles_5'].fillna(tempdf3['scrambles_6'], inplace=True)
tempdf4['scrambles_7'].fillna(tempdf4['scrambles_6'], inplace=True)
tempdf3['twp_rate_5'].fillna(tempdf3['twp_rate_6'], inplace=True)
tempdf4['twp_rate_7'].fillna(tempdf4['twp_rate_6'], inplace=True)
tempdf3['yards_5'].fillna(tempdf3['yards_6'], inplace=True)
tempdf4['yards_7'].fillna(tempdf4['yards_6'], inplace=True)
tempdf3['ypa_5'].fillna(tempdf3['ypa_6'], inplace=True)
tempdf4['ypa_7'].fillna(tempdf4['ypa_6'], inplace=True)
tempdf4['accuracy_percent_6'].fillna(tempdf4['accuracy_percent_7'], inplace=True)
tempdf4['attempts_6'].fillna(tempdf4['attempts_7'], inplace=True)
tempdf4['avg_depth_of_target_6'].fillna(tempdf4['avg_depth_of_target_7'], inplace=True)
tempdf4['dropbacks_6'].fillna(tempdf4['dropbacks_7'], inplace=True)
tempdf4['passing_snaps_6'].fillna(tempdf4['passing_snaps_7'], inplace=True)
tempdf4['pressure_to_sack_rate_6'].fillna(tempdf4['pressure_to_sack_rate_7'], inplace=True)
tempdf4['scrambles_6'].fillna(tempdf4['scrambles_7'], inplace=True)
tempdf4['twp_rate_6'].fillna(tempdf4['twp_rate_7'], inplace=True)
tempdf4['yards_6'].fillna(tempdf4['yards_7'], inplace=True)
tempdf4['ypa_6'].fillna(tempdf4['ypa_7'], inplace=True)
tempdf5 = pd.merge(tempdf4,tempdf3, on='player', how='outer')
tempdf5 = tempdf5.drop(columns=['position_6_y', 'team_name_6_y',
       'accuracy_percent_6_y', 'attempts_6_y', 'avg_depth_of_target_6_y',
       'dropbacks_6_y', 'passing_snaps_6_y', 'pressure_to_sack_rate_6_y',
       'scrambles_6_y', 'twp_rate_6_y', 'yards_6_y', 'ypa_6_y'])
tempdf5['accuracy_percent_6_x'].fillna(tempdf5['accuracy_percent_5'], inplace=True)
tempdf5['accuracy_percent_7'].fillna(tempdf5['accuracy_percent_6_x'], inplace=True)
tempdf5['attempts_6_x'].fillna(tempdf5['attempts_5'], inplace=True)
tempdf5['attempts_7'].fillna(tempdf5['attempts_6_x'], inplace=True)
tempdf5['avg_depth_of_target_6_x'].fillna(tempdf5['avg_depth_of_target_5'], inplace=True)
tempdf5['avg_depth_of_target_7'].fillna(tempdf5['avg_depth_of_target_6_x'], inplace=True)
tempdf5['dropbacks_6_x'].fillna(tempdf5['dropbacks_5'], inplace=True)
tempdf5['dropbacks_7'].fillna(tempdf5['dropbacks_6_x'], inplace=True)
tempdf5['passing_snaps_6_x'].fillna(tempdf5['passing_snaps_5'], inplace=True)
tempdf5['passing_snaps_7'].fillna(tempdf5['passing_snaps_6_x'], inplace=True)
tempdf5['pressure_to_sack_rate_6_x'].fillna(tempdf5['pressure_to_sack_rate_5'], inplace=True)
tempdf5['pressure_to_sack_rate_7'].fillna(tempdf5['pressure_to_sack_rate_6_x'], inplace=True)
tempdf5['scrambles_6_x'].fillna(tempdf5['scrambles_5'], inplace=True)
tempdf5['scrambles_7'].fillna(tempdf5['scrambles_6_x'], inplace=True)
tempdf5['twp_rate_6_x'].fillna(tempdf5['twp_rate_5'], inplace=True)
tempdf5['twp_rate_7'].fillna(tempdf5['twp_rate_6_x'], inplace=True)
tempdf5['yards_6_x'].fillna(tempdf5['yards_5'], inplace=True)
tempdf5['yards_7'].fillna(tempdf5['yards_6_x'], inplace=True)
tempdf5['ypa_6_x'].fillna(tempdf5['ypa_5'], inplace=True)
tempdf5['ypa_7'].fillna(tempdf5['ypa_6_x'], inplace=True)
tempdf5['accuracy_percent_6_x'].fillna(tempdf5['accuracy_percent_7'], inplace=True)
tempdf5['accuracy_percent_7'].fillna(tempdf5['accuracy_percent_5'], inplace=True)
tempdf5['attempts_6_x'].fillna(tempdf5['attempts_7'], inplace=True)
tempdf5['attempts_7'].fillna(tempdf5['attempts_5'], inplace=True)
tempdf5['avg_depth_of_target_6_x'].fillna(tempdf5['avg_depth_of_target_7'], inplace=True)
tempdf5['avg_depth_of_target_7'].fillna(tempdf5['avg_depth_of_target_5'], inplace=True)
tempdf5['dropbacks_6_x'].fillna(tempdf5['dropbacks_7'], inplace=True)
tempdf5['dropbacks_7'].fillna(tempdf5['dropbacks_5'], inplace=True)
tempdf5['passing_snaps_6_x'].fillna(tempdf5['passing_snaps_7'], inplace=True)
tempdf5['passing_snaps_7'].fillna(tempdf5['passing_snaps_5'], inplace=True)
tempdf5['pressure_to_sack_rate_6_x'].fillna(tempdf5['pressure_to_sack_rate_7'], inplace=True)
tempdf5['pressure_to_sack_rate_7'].fillna(tempdf5['pressure_to_sack_rate_5'], inplace=True)
tempdf5['scrambles_6_x'].fillna(tempdf5['scrambles_7'], inplace=True)
tempdf5['scrambles_7'].fillna(tempdf5['scrambles_5'], inplace=True)
tempdf5['twp_rate_6_x'].fillna(tempdf5['twp_rate_7'], inplace=True)
tempdf5['twp_rate_7'].fillna(tempdf5['twp_rate_5'], inplace=True)
tempdf5['yards_6_x'].fillna(tempdf5['yards_7'], inplace=True)
tempdf5['yards_7'].fillna(tempdf5['yards_5'], inplace=True)
tempdf5['ypa_6_x'].fillna(tempdf5['ypa_7'], inplace=True)
tempdf5['ypa_7'].fillna(tempdf5['ypa_5'], inplace=True)
tempdf5['accuracy_percent_5'].fillna(tempdf5['accuracy_percent_7'], inplace=True)
tempdf5['attempts_5'].fillna(tempdf5['attempts_7'], inplace=True)
tempdf5['avg_depth_of_target_5'].fillna(tempdf5['avg_depth_of_target_7'], inplace=True)
tempdf5['dropbacks_5'].fillna(tempdf5['dropbacks_7'], inplace=True)
tempdf5['passing_snaps_5'].fillna(tempdf5['passing_snaps_7'], inplace=True)
tempdf5['pressure_to_sack_rate_5'].fillna(tempdf5['pressure_to_sack_rate_7'], inplace=True)
tempdf5['scrambles_5'].fillna(tempdf5['scrambles_7'], inplace=True)
tempdf5['twp_rate_5'].fillna(tempdf5['twp_rate_7'], inplace=True)
tempdf5['yards_5'].fillna(tempdf5['yards_7'], inplace=True)
tempdf5['ypa_5'].fillna(tempdf5['ypa_7'], inplace=True)
tempdf1['accuracy_percent_2'].fillna(tempdf1['accuracy_percent_1'], inplace=True)
tempdf2['accuracy_percent_3'].fillna(tempdf2['accuracy_percent_4'], inplace=True)
tempdf1['attempts_2'].fillna(tempdf1['attempts_1'], inplace=True)
tempdf2['attempts_3'].fillna(tempdf2['attempts_4'], inplace=True)
tempdf1['avg_depth_of_target_2'].fillna(tempdf1['avg_depth_of_target_1'], inplace=True)
tempdf2['avg_depth_of_target_3'].fillna(tempdf2['avg_depth_of_target_4'], inplace=True)
tempdf1['dropbacks_2'].fillna(tempdf1['dropbacks_1'], inplace=True)
tempdf2['dropbacks_3'].fillna(tempdf2['dropbacks_4'], inplace=True)
tempdf1['passing_snaps_2'].fillna(tempdf1['passing_snaps_1'], inplace=True)
tempdf2['passing_snaps_3'].fillna(tempdf2['passing_snaps_4'], inplace=True)
tempdf1['pressure_to_sack_rate_2'].fillna(tempdf1['pressure_to_sack_rate_1'], inplace=True)
tempdf2['pressure_to_sack_rate_3'].fillna(tempdf2['pressure_to_sack_rate_4'], inplace=True)
tempdf1['scrambles_2'].fillna(tempdf1['scrambles_1'], inplace=True)
tempdf2['scrambles_3'].fillna(tempdf2['scrambles_4'], inplace=True)
tempdf1['twp_rate_2'].fillna(tempdf1['twp_rate_1'], inplace=True)
tempdf2['twp_rate_3'].fillna(tempdf2['twp_rate_4'], inplace=True)
tempdf1['yards_2'].fillna(tempdf1['yards_1'], inplace=True)
tempdf2['yards_3'].fillna(tempdf2['yards_4'], inplace=True)
tempdf1['ypa_2'].fillna(tempdf1['ypa_1'], inplace=True)
tempdf2['ypa_3'].fillna(tempdf2['ypa_4'], inplace=True)
tempdf1['accuracy_percent_1'].fillna(tempdf1['accuracy_percent_2'], inplace=True)
tempdf2['accuracy_percent_4'].fillna(tempdf2['accuracy_percent_3'], inplace=True)
tempdf1['attempts_1'].fillna(tempdf1['attempts_2'], inplace=True)
tempdf2['attempts_4'].fillna(tempdf2['attempts_3'], inplace=True)
tempdf1['avg_depth_of_target_1'].fillna(tempdf1['avg_depth_of_target_2'], inplace=True)
tempdf2['avg_depth_of_target_4'].fillna(tempdf2['avg_depth_of_target_3'], inplace=True)
tempdf1['dropbacks_1'].fillna(tempdf1['dropbacks_2'], inplace=True)
tempdf2['dropbacks_4'].fillna(tempdf2['dropbacks_3'], inplace=True)
tempdf1['passing_snaps_1'].fillna(tempdf1['passing_snaps_2'], inplace=True)
tempdf2['passing_snaps_4'].fillna(tempdf2['passing_snaps_3'], inplace=True)
tempdf1['pressure_to_sack_rate_1'].fillna(tempdf1['pressure_to_sack_rate_2'], inplace=True)
tempdf2['pressure_to_sack_rate_4'].fillna(tempdf2['pressure_to_sack_rate_3'], inplace=True)
tempdf1['scrambles_1'].fillna(tempdf1['scrambles_2'], inplace=True)
tempdf2['scrambles_4'].fillna(tempdf2['scrambles_3'], inplace=True)
tempdf1['twp_rate_1'].fillna(tempdf1['twp_rate_2'], inplace=True)
tempdf2['twp_rate_4'].fillna(tempdf2['twp_rate_3'], inplace=True)
tempdf1['yards_1'].fillna(tempdf1['yards_2'], inplace=True)
tempdf2['yards_4'].fillna(tempdf2['yards_3'], inplace=True)
tempdf1['ypa_1'].fillna(tempdf1['ypa_2'], inplace=True)
tempdf2['ypa_4'].fillna(tempdf2['ypa_3'], inplace=True)
tempdf6 = pd.merge(tempdf1, tempdf2, on='player', how='outer')
tempdf7 = pd.merge(tempdf5, tempdf6, on='player', how='outer')
tempdf6['accuracy_percent_1'].fillna(tempdf6['accuracy_percent_3'], inplace=True)
tempdf6['accuracy_percent_2'].fillna(tempdf6['accuracy_percent_3'], inplace=True)
tempdf6['attempts_1'].fillna(tempdf6['attempts_3'], inplace=True)
tempdf6['attempts_2'].fillna(tempdf6['attempts_3'], inplace=True)
tempdf6['avg_depth_of_target_1'].fillna(tempdf6['avg_depth_of_target_3'], inplace=True)
tempdf6['avg_depth_of_target_2'].fillna(tempdf6['avg_depth_of_target_3'], inplace=True)
tempdf6['dropbacks_1'].fillna(tempdf6['dropbacks_3'], inplace=True)
tempdf6['dropbacks_2'].fillna(tempdf6['dropbacks_3'], inplace=True)
tempdf6['passing_snaps_1'].fillna(tempdf6['passing_snaps_3'], inplace=True)
tempdf6['passing_snaps_2'].fillna(tempdf6['passing_snaps_3'], inplace=True)
tempdf6['pressure_to_sack_rate_1'].fillna(tempdf6['pressure_to_sack_rate_3'], inplace=True)
tempdf6['pressure_to_sack_rate_2'].fillna(tempdf6['pressure_to_sack_rate_3'], inplace=True)
tempdf6['scrambles_1'].fillna(tempdf6['scrambles_3'], inplace=True)
tempdf6['scrambles_2'].fillna(tempdf6['scrambles_3'], inplace=True)
tempdf6['twp_rate_1'].fillna(tempdf6['twp_rate_3'], inplace=True)
tempdf6['twp_rate_2'].fillna(tempdf6['twp_rate_3'], inplace=True)
tempdf6['yards_1'].fillna(tempdf6['yards_3'], inplace=True)
tempdf6['yards_2'].fillna(tempdf6['yards_3'], inplace=True)
tempdf6['ypa_1'].fillna(tempdf6['ypa_3'], inplace=True)
tempdf6['ypa_2'].fillna(tempdf6['ypa_3'], inplace=True)
tempdf6['accuracy_percent_3'].fillna(tempdf6['accuracy_percent_1'], inplace=True)
tempdf6['accuracy_percent_4'].fillna(tempdf6['accuracy_percent_1'], inplace=True)
tempdf6['attempts_3'].fillna(tempdf6['attempts_1'], inplace=True)
tempdf6['attempts_4'].fillna(tempdf6['attempts_1'], inplace=True)
tempdf6['avg_depth_of_target_3'].fillna(tempdf6['avg_depth_of_target_1'], inplace=True)
tempdf6['avg_depth_of_target_4'].fillna(tempdf6['avg_depth_of_target_1'], inplace=True)
tempdf6['dropbacks_3'].fillna(tempdf6['dropbacks_1'], inplace=True)
tempdf6['dropbacks_4'].fillna(tempdf6['dropbacks_1'], inplace=True)
tempdf6['passing_snaps_3'].fillna(tempdf6['passing_snaps_1'], inplace=True)
tempdf6['passing_snaps_4'].fillna(tempdf6['passing_snaps_1'], inplace=True)
tempdf6['pressure_to_sack_rate_3'].fillna(tempdf6['pressure_to_sack_rate_1'], inplace=True)
tempdf6['pressure_to_sack_rate_4'].fillna(tempdf6['pressure_to_sack_rate_1'], inplace=True)
tempdf6['scrambles_3'].fillna(tempdf6['scrambles_1'], inplace=True)
tempdf6['scrambles_4'].fillna(tempdf6['scrambles_1'], inplace=True)
tempdf6['twp_rate_3'].fillna(tempdf6['twp_rate_1'], inplace=True)
tempdf6['twp_rate_4'].fillna(tempdf6['twp_rate_1'], inplace=True)
tempdf6['yards_3'].fillna(tempdf6['yards_1'], inplace=True)
tempdf6['yards_4'].fillna(tempdf6['yards_1'], inplace=True)
tempdf6['ypa_3'].fillna(tempdf6['ypa_1'], inplace=True)
tempdf6['ypa_4'].fillna(tempdf6['ypa_1'], inplace=True)
tempdf7['accuracy_percent_1'].fillna(tempdf7['accuracy_percent_5'], inplace=True)
tempdf7['accuracy_percent_2'].fillna(tempdf7['accuracy_percent_5'], inplace=True)
tempdf7['attempts_1'].fillna(tempdf7['attempts_5'], inplace=True)
tempdf7['attempts_2'].fillna(tempdf7['attempts_5'], inplace=True)
tempdf7['avg_depth_of_target_1'].fillna(tempdf7['avg_depth_of_target_5'], inplace=True)
tempdf7['avg_depth_of_target_2'].fillna(tempdf7['avg_depth_of_target_5'], inplace=True)
tempdf7['dropbacks_1'].fillna(tempdf7['dropbacks_5'], inplace=True)
tempdf7['dropbacks_2'].fillna(tempdf7['dropbacks_5'], inplace=True)
tempdf7['passing_snaps_1'].fillna(tempdf7['passing_snaps_5'], inplace=True)
tempdf7['passing_snaps_2'].fillna(tempdf7['passing_snaps_5'], inplace=True)
tempdf7['pressure_to_sack_rate_1'].fillna(tempdf7['pressure_to_sack_rate_5'], inplace=True)
tempdf7['pressure_to_sack_rate_2'].fillna(tempdf7['pressure_to_sack_rate_5'], inplace=True)
tempdf7['scrambles_1'].fillna(tempdf7['scrambles_5'], inplace=True)
tempdf7['scrambles_2'].fillna(tempdf7['scrambles_5'], inplace=True)
tempdf7['twp_rate_1'].fillna(tempdf7['twp_rate_5'], inplace=True)
tempdf7['twp_rate_2'].fillna(tempdf7['twp_rate_5'], inplace=True)
tempdf7['yards_1'].fillna(tempdf7['yards_5'], inplace=True)
tempdf7['yards_2'].fillna(tempdf7['yards_5'], inplace=True)
tempdf7['ypa_1'].fillna(tempdf7['ypa_5'], inplace=True)
tempdf7['ypa_2'].fillna(tempdf7['ypa_5'], inplace=True)
tempdf7['accuracy_percent_3'].fillna(tempdf7['accuracy_percent_5'], inplace=True)
tempdf7['accuracy_percent_4'].fillna(tempdf7['accuracy_percent_5'], inplace=True)
tempdf7['attempts_3'].fillna(tempdf7['attempts_5'], inplace=True)
tempdf7['attempts_4'].fillna(tempdf7['attempts_5'], inplace=True)
tempdf7['avg_depth_of_target_3'].fillna(tempdf7['avg_depth_of_target_5'], inplace=True)
tempdf7['avg_depth_of_target_4'].fillna(tempdf7['avg_depth_of_target_5'], inplace=True)
tempdf7['dropbacks_3'].fillna(tempdf7['dropbacks_5'], inplace=True)
tempdf7['dropbacks_4'].fillna(tempdf7['dropbacks_5'], inplace=True)
tempdf7['passing_snaps_3'].fillna(tempdf7['passing_snaps_5'], inplace=True)
tempdf7['passing_snaps_4'].fillna(tempdf7['passing_snaps_5'], inplace=True)
tempdf7['pressure_to_sack_rate_3'].fillna(tempdf7['pressure_to_sack_rate_5'], inplace=True)
tempdf7['pressure_to_sack_rate_4'].fillna(tempdf7['pressure_to_sack_rate_5'], inplace=True)
tempdf7['scrambles_3'].fillna(tempdf7['scrambles_5'], inplace=True)
tempdf7['scrambles_4'].fillna(tempdf7['scrambles_5'], inplace=True)
tempdf7['twp_rate_3'].fillna(tempdf7['twp_rate_5'], inplace=True)
tempdf7['twp_rate_4'].fillna(tempdf7['twp_rate_5'], inplace=True)
tempdf7['yards_3'].fillna(tempdf7['yards_5'], inplace=True)
tempdf7['yards_4'].fillna(tempdf7['yards_5'], inplace=True)
tempdf7['ypa_3'].fillna(tempdf7['ypa_5'], inplace=True)
tempdf7['ypa_4'].fillna(tempdf7['ypa_5'], inplace=True)
tempdf7['accuracy_percent_5'].fillna(tempdf7['accuracy_percent_4'], inplace=True)
tempdf7['accuracy_percent_6_x'].fillna(tempdf7['accuracy_percent_4'], inplace=True)
tempdf7['attempts_5'].fillna(tempdf7['attempts_4'], inplace=True)
tempdf7['attempts_6_x'].fillna(tempdf7['attempts_4'], inplace=True)
tempdf7['avg_depth_of_target_5'].fillna(tempdf7['avg_depth_of_target_4'], inplace=True)
tempdf7['avg_depth_of_target_6_x'].fillna(tempdf7['avg_depth_of_target_4'], inplace=True)
tempdf7['dropbacks_5'].fillna(tempdf7['dropbacks_4'], inplace=True)
tempdf7['dropbacks_6_x'].fillna(tempdf7['dropbacks_4'], inplace=True)
tempdf7['passing_snaps_5'].fillna(tempdf7['passing_snaps_4'], inplace=True)
tempdf7['passing_snaps_6_x'].fillna(tempdf7['passing_snaps_4'], inplace=True)
tempdf7['pressure_to_sack_rate_5'].fillna(tempdf7['pressure_to_sack_rate_4'], inplace=True)
tempdf7['pressure_to_sack_rate_6_x'].fillna(tempdf7['pressure_to_sack_rate_4'], inplace=True)
tempdf7['scrambles_5'].fillna(tempdf7['scrambles_4'], inplace=True)
tempdf7['scrambles_6_x'].fillna(tempdf7['scrambles_4'], inplace=True)
tempdf7['twp_rate_5'].fillna(tempdf7['twp_rate_4'], inplace=True)
tempdf7['twp_rate_6_x'].fillna(tempdf7['twp_rate_4'], inplace=True)
tempdf7['yards_5'].fillna(tempdf7['yards_4'], inplace=True)
tempdf7['yards_6_x'].fillna(tempdf7['yards_4'], inplace=True)
tempdf7['ypa_5'].fillna(tempdf7['ypa_4'], inplace=True)
tempdf7['ypa_6_x'].fillna(tempdf7['ypa_4'], inplace=True)
tempdf7['accuracy_percent_7'].fillna(tempdf7['accuracy_percent_2'], inplace=True)
tempdf7['attempts_7'].fillna(tempdf7['attempts_2'], inplace=True)
tempdf7['avg_depth_of_target_7'].fillna(tempdf7['avg_depth_of_target_2'], inplace=True)
tempdf7['dropbacks_7'].fillna(tempdf7['dropbacks_2'], inplace=True)
tempdf7['passing_snaps_7'].fillna(tempdf7['passing_snaps_2'], inplace=True)
tempdf7['pressure_to_sack_rate_7'].fillna(tempdf7['pressure_to_sack_rate_2'], inplace=True)
tempdf7['scrambles_7'].fillna(tempdf7['scrambles_2'], inplace=True)
tempdf7['twp_rate_7'].fillna(tempdf7['twp_rate_2'], inplace=True)
tempdf7['yards_7'].fillna(tempdf7['yards_2'], inplace=True)
tempdf7['ypa_7'].fillna(tempdf7['ypa_2'], inplace=True)
tempdf7['accuracy_percent_1'].fillna(tempdf7['accuracy_percent_6_x'], inplace=True)
tempdf7['accuracy_percent_2'].fillna(tempdf7['accuracy_percent_6_x'], inplace=True)
tempdf7['attempts_1'].fillna(tempdf7['attempts_6_x'], inplace=True)
tempdf7['attempts_2'].fillna(tempdf7['attempts_6_x'], inplace=True)
tempdf7['avg_depth_of_target_1'].fillna(tempdf7['avg_depth_of_target_6_x'], inplace=True)
tempdf7['avg_depth_of_target_2'].fillna(tempdf7['avg_depth_of_target_6_x'], inplace=True)
tempdf7['dropbacks_1'].fillna(tempdf7['dropbacks_6_x'], inplace=True)
tempdf7['dropbacks_2'].fillna(tempdf7['dropbacks_6_x'], inplace=True)
tempdf7['passing_snaps_1'].fillna(tempdf7['passing_snaps_6_x'], inplace=True)
tempdf7['passing_snaps_2'].fillna(tempdf7['passing_snaps_6_x'], inplace=True)
tempdf7['pressure_to_sack_rate_1'].fillna(tempdf7['pressure_to_sack_rate_6_x'], inplace=True)
tempdf7['pressure_to_sack_rate_2'].fillna(tempdf7['pressure_to_sack_rate_6_x'], inplace=True)
tempdf7['scrambles_1'].fillna(tempdf7['scrambles_6_x'], inplace=True)
tempdf7['scrambles_2'].fillna(tempdf7['scrambles_6_x'], inplace=True)
tempdf7['twp_rate_1'].fillna(tempdf7['twp_rate_6_x'], inplace=True)
tempdf7['twp_rate_2'].fillna(tempdf7['twp_rate_6_x'], inplace=True)
tempdf7['yards_1'].fillna(tempdf7['yards_6_x'], inplace=True)
tempdf7['yards_2'].fillna(tempdf7['yards_6_x'], inplace=True)
tempdf7['ypa_1'].fillna(tempdf7['ypa_6_x'], inplace=True)
tempdf7['ypa_2'].fillna(tempdf7['ypa_6_x'], inplace=True)
tempdf7['accuracy_percent_3'].fillna(tempdf7['accuracy_percent_1'], inplace=True)
tempdf7['accuracy_percent_4'].fillna(tempdf7['accuracy_percent_1'], inplace=True)
tempdf7['attempts_3'].fillna(tempdf7['attempts_1'], inplace=True)
tempdf7['attempts_4'].fillna(tempdf7['attempts_1'], inplace=True)
tempdf7['avg_depth_of_target_3'].fillna(tempdf7['avg_depth_of_target_1'], inplace=True)
tempdf7['avg_depth_of_target_4'].fillna(tempdf7['avg_depth_of_target_1'], inplace=True)
tempdf7['dropbacks_3'].fillna(tempdf7['dropbacks_1'], inplace=True)
tempdf7['dropbacks_4'].fillna(tempdf7['dropbacks_1'], inplace=True)
tempdf7['passing_snaps_3'].fillna(tempdf7['passing_snaps_1'], inplace=True)
tempdf7['passing_snaps_4'].fillna(tempdf7['passing_snaps_1'], inplace=True)
tempdf7['pressure_to_sack_rate_3'].fillna(tempdf7['pressure_to_sack_rate_1'], inplace=True)
tempdf7['pressure_to_sack_rate_4'].fillna(tempdf7['pressure_to_sack_rate_1'], inplace=True)
tempdf7['scrambles_3'].fillna(tempdf7['scrambles_1'], inplace=True)
tempdf7['scrambles_4'].fillna(tempdf7['scrambles_1'], inplace=True)
tempdf7['twp_rate_3'].fillna(tempdf7['twp_rate_1'], inplace=True)
tempdf7['twp_rate_4'].fillna(tempdf7['twp_rate_1'], inplace=True)
tempdf7['yards_3'].fillna(tempdf7['yards_1'], inplace=True)
tempdf7['yards_4'].fillna(tempdf7['yards_1'], inplace=True)
tempdf7['ypa_3'].fillna(tempdf7['ypa_1'], inplace=True)
tempdf7['ypa_4'].fillna(tempdf7['ypa_1'], inplace=True)
tempdf7['accuracy_percent_5'].fillna(tempdf7['accuracy_percent_3'], inplace=True)
tempdf7['accuracy_percent_6_x'].fillna(tempdf7['accuracy_percent_3'], inplace=True)
tempdf7['attempts_5'].fillna(tempdf7['attempts_3'], inplace=True)
tempdf7['attempts_6_x'].fillna(tempdf7['attempts_3'], inplace=True)
tempdf7['avg_depth_of_target_5'].fillna(tempdf7['avg_depth_of_target_3'], inplace=True)
tempdf7['avg_depth_of_target_6_x'].fillna(tempdf7['avg_depth_of_target_3'], inplace=True)
tempdf7['dropbacks_5'].fillna(tempdf7['dropbacks_3'], inplace=True)
tempdf7['dropbacks_6_x'].fillna(tempdf7['dropbacks_3'], inplace=True)
tempdf7['passing_snaps_5'].fillna(tempdf7['passing_snaps_3'], inplace=True)
tempdf7['passing_snaps_6_x'].fillna(tempdf7['passing_snaps_3'], inplace=True)
tempdf7['pressure_to_sack_rate_5'].fillna(tempdf7['pressure_to_sack_rate_3'], inplace=True)
tempdf7['pressure_to_sack_rate_6_x'].fillna(tempdf7['pressure_to_sack_rate_3'], inplace=True)
tempdf7['scrambles_5'].fillna(tempdf7['scrambles_3'], inplace=True)
tempdf7['scrambles_6_x'].fillna(tempdf7['scrambles_3'], inplace=True)
tempdf7['twp_rate_5'].fillna(tempdf7['twp_rate_3'], inplace=True)
tempdf7['twp_rate_6_x'].fillna(tempdf7['twp_rate_3'], inplace=True)
tempdf7['yards_5'].fillna(tempdf7['yards_3'], inplace=True)
tempdf7['yards_6_x'].fillna(tempdf7['yards_3'], inplace=True)
tempdf7['ypa_5'].fillna(tempdf7['ypa_3'], inplace=True)
tempdf7['ypa_6_x'].fillna(tempdf7['ypa_3'], inplace=True)
tempdf7['accuracy_percent_7'].fillna(tempdf7['accuracy_percent_1'], inplace=True)
tempdf7['attempts_7'].fillna(tempdf7['attempts_1'], inplace=True)
tempdf7['avg_depth_of_target_7'].fillna(tempdf7['avg_depth_of_target_1'], inplace=True)
tempdf7['dropbacks_7'].fillna(tempdf7['dropbacks_1'], inplace=True)
tempdf7['passing_snaps_7'].fillna(tempdf7['passing_snaps_1'], inplace=True)
tempdf7['pressure_to_sack_rate_7'].fillna(tempdf7['pressure_to_sack_rate_1'], inplace=True)
tempdf7['scrambles_7'].fillna(tempdf7['scrambles_1'], inplace=True)
tempdf7['twp_rate_7'].fillna(tempdf7['twp_rate_1'], inplace=True)
tempdf7['yards_7'].fillna(tempdf7['yards_1'], inplace=True)
tempdf7['ypa_7'].fillna(tempdf7['ypa_1'], inplace=True)
tempdf7['Accuracy (%)'] = (tempdf7['accuracy_percent_1'] + tempdf7['accuracy_percent_2'] + tempdf7['accuracy_percent_3'] + tempdf7['accuracy_percent_4'] + tempdf7['accuracy_percent_5'] + tempdf7['accuracy_percent_6_x'] + tempdf7['accuracy_percent_7']) / 7
tempdf7['Accuracy (%)'] = round(tempdf7['Accuracy (%)'], 2)
tempdf7['Attempts'] = (tempdf7['attempts_1'] + tempdf7['attempts_2'] + tempdf7['attempts_3'] + tempdf7['attempts_4'] + tempdf7['attempts_5'] + tempdf7['attempts_6_x'] + tempdf7['attempts_7']) / 7
tempdf7['Attempts'] = np.ceil(tempdf7['Attempts']).astype(int)
tempdf7['ADOT'] = (tempdf7['avg_depth_of_target_1'] + tempdf7['avg_depth_of_target_2'] + tempdf7['avg_depth_of_target_3'] + tempdf7['avg_depth_of_target_4'] + tempdf7['avg_depth_of_target_5'] + tempdf7['avg_depth_of_target_6_x'] + tempdf7['avg_depth_of_target_7']) / 7
tempdf7['ADOT'] = round(tempdf7['ADOT'], 2)
tempdf7['Dropbacks'] = (tempdf7['dropbacks_1'] + tempdf7['dropbacks_2'] + tempdf7['dropbacks_3'] + tempdf7['dropbacks_4'] + tempdf7['dropbacks_5'] + tempdf7['dropbacks_6_x'] + tempdf7['dropbacks_7']) / 7
tempdf7['Dropbacks'] = np.ceil(tempdf7['Dropbacks']).astype(int)
tempdf7['Passing Snaps'] = (tempdf7['passing_snaps_1'] + tempdf7['passing_snaps_2'] + tempdf7['passing_snaps_3'] + tempdf7['passing_snaps_4'] + tempdf7['passing_snaps_5'] + tempdf7['passing_snaps_6_x'] + tempdf7['passing_snaps_7']) / 7
tempdf7['Passing Snaps'] = np.ceil(tempdf7['Passing Snaps']).astype(int)
tempdf7['P2S%'] = (tempdf7['pressure_to_sack_rate_1'] + tempdf7['pressure_to_sack_rate_2'] + tempdf7['pressure_to_sack_rate_3'] + tempdf7['pressure_to_sack_rate_4'] + tempdf7['pressure_to_sack_rate_5'] + tempdf7['pressure_to_sack_rate_6_x'] + tempdf7['pressure_to_sack_rate_7']) / 7
tempdf7['P2S%'] = round(tempdf7['P2S%'], 2)
tempdf7['Scrambles'] = (tempdf7['scrambles_1'] + tempdf7['scrambles_2'] + tempdf7['scrambles_3'] + tempdf7['scrambles_4'] + tempdf7['scrambles_5'] + tempdf7['scrambles_6_x'] + tempdf7['scrambles_7']) / 7
tempdf7['Scrambles'] = np.ceil(tempdf7['Scrambles']).astype(int)
tempdf7['TWP%'] = (tempdf7['twp_rate_1'] + tempdf7['twp_rate_2'] + tempdf7['twp_rate_3'] + tempdf7['twp_rate_4'] + tempdf7['twp_rate_5'] + tempdf7['twp_rate_6_x'] + tempdf7['twp_rate_7']) / 7
tempdf7['TWP%'] = round(tempdf7['TWP%'], 2)
tempdf7['Yards'] = (tempdf7['yards_1'] + tempdf7['yards_2'] + tempdf7['yards_3'] + tempdf7['yards_4'] + tempdf7['yards_5'] + tempdf7['yards_6_x'] + tempdf7['yards_7']) / 7
tempdf7['Yards'] = np.ceil(tempdf7['Yards']).astype(int)
tempdf7['YPA'] = (tempdf7['ypa_1'] + tempdf7['ypa_2'] + tempdf7['ypa_3'] + tempdf7['ypa_4'] + tempdf7['ypa_5'] + tempdf7['ypa_6_x'] + tempdf7['ypa_7']) / 7
tempdf7['YPA'] = round(tempdf7['YPA'], 2)
tempdf8= tempdf7.copy()
qb_training_data = tempdf8[['player', 'position_1', 'team_name_1', 'Accuracy (%)', 'Attempts', 'ADOT', 'Dropbacks', 'Passing Snaps', 'P2S%', 'Scrambles', 'TWP%', 'Yards', 'YPA']]
qb_training_data = qb_training_data.rename(columns = {'position_1' : 'Pos', 'team_name_1' : 'Team'})
qb_training_data['Pos'].fillna('QB', inplace=True)
qb_training_data.loc[qb_training_data['player'] == 'Kyler Murray', 'Team'] = 'OKLAHOMA'
qb_training_data.loc[qb_training_data['player'] == 'Aidan O\'Connell', 'Team'] = 'PURDUE'
qb_training_data.loc[qb_training_data['player'] == 'Bryce Young', 'Team'] = 'ALABAMA'
qb_training_data.loc[qb_training_data['player'] == 'C.J. Stroud', 'Team'] = 'OHIO STATE'
qb_training_data.loc[qb_training_data['player'] == 'Brock Purdy', 'Team'] = 'IOWA STATE'
qb_training_data.loc[qb_training_data['player'] == 'Desmond Ridder', 'Team'] = 'CINCINNATI'
qb_training_data.loc[qb_training_data['player'] == 'Kenny Pickett', 'Team'] = 'PITTSBURGH'
qb_training_data.loc[qb_training_data['player'] == 'C.J. Stroud', 'Team'] = 'OHIO STATE'
qb_training_data.loc[qb_training_data['player'] == 'Sam Howell', 'Team'] = 'N CAROLINA'
qb_training_data.loc[qb_training_data['player'] == 'Joe Burrow', 'Team'] = 'LSU'
qb_training_data.loc[qb_training_data['player'] == 'Jordan Love', 'Team'] = 'UTAH STATE'
qb_training_data.loc[qb_training_data['player'] == 'Justin Herbert', 'Team'] = 'OREGON'
qb_training_data.loc[qb_training_data['player'] == 'Trevor Lawrence', 'Team'] = 'CLEMSON'
qb_training_data.loc[qb_training_data['player'] == 'Dak Prescott', 'Team'] = 'MISS STATE'
qb_training_data.loc[qb_training_data['player'] == 'Jared Goff', 'Team'] = 'CAL'
qb_training_data.loc[qb_training_data['player'] == 'Gardner Minshew', 'Team'] = 'WASH STATE'
qb_training_data = qb_training_data.sort_values(by='player')
qb_training_data.reset_index(drop=True, inplace=True)
qb_training_data

,player,Pos,Team,Accuracy (%),Attempts,ADOT,Dropbacks,Passing Snaps,P2S%,Scrambles,TWP%,Yards,YPA
0,Aidan O'Connell,QB,PURDUE,78.14,401,8.31,425,444,21.51,7,3.19,3167,7.93
1,Baker Mayfield,QB,OKLAHOMA,80.61,348,10.40,404,420,19.54,31,2.54,3837,11.03
2,Brock Purdy,QB,IOWA STATE,79.30,370,7.60,423,445,16.40,32,2.50,2996,8.10
3,Bryce Young,QB,ALABAMA,79.44,413,9.09,471,491,17.24,29,1.84,3771,9.17
4,C.J. Stroud,QB,OHIO STATE,77.29,391,10.17,413,432,11.39,11,2.50,3787,9.74
5,Dak Prescott,QB,MISS STATE,75.80,437,7.90,483,497,18.30,16,4.40,3419,7.80
6,Desmond Ridder,QB,CINCINNATI,74.41,330,10.54,374,387,23.47,22,2.47,2809,8.46
7,Gardner Minshew,QB,WASH STATE,78.93,571,6.83,605,629,7.11,25,2.53,4144,7.26
8,Jalen Hurts,QB,ALABAMA,77.70,295,10.37,362,377,14.71,48,2.94,3088,10.56
9,Jared Goff,QB,CAL,74.00,494,10.10,542,577,15.50,23,4.20,4259,8.60


In [7]:
#Obtain NFL passing data for current QBs from 2019-2022. 
NFL_passing2022 = pd.read_csv("NFLQBpassing_summary2022.csv")
NFL_passing2022 = NFL_passing2022[NFL_passing2022['player'].isin(qbs)]
NFL_passing2022 = NFL_passing2022[NFL_passing2022['dropbacks'] >= 382]
NFL_passing2022 = NFL_passing2022.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
NFL_passing2022 = NFL_passing2022.sort_values(by='player')
NFL_passing2022.reset_index(drop=True, inplace=True)
NFL_passing2021 = pd.read_csv("NFLQBpassing_summary2021.csv")
NFL_passing2021 = NFL_passing2021[NFL_passing2021['player'].isin(qbs)]
NFL_passing2021 = NFL_passing2021[NFL_passing2021['dropbacks'] >= 375]
NFL_passing2021 = NFL_passing2021.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
NFL_passing2021 = NFL_passing2021.sort_values(by='player')
NFL_passing2021.reset_index(drop=True, inplace=True)
NFL_passing2019 = pd.read_csv("NFLQBpassing_summary2019.csv")
NFL_passing2019 = NFL_passing2019[NFL_passing2019['player'].isin(qbs)]
NFL_passing2019 = NFL_passing2019[NFL_passing2019['dropbacks'] >= 352]
NFL_passing2019 = NFL_passing2019.drop(columns = ['player_id', 'player_game_count', 'aimed_passes', 'avg_time_to_throw', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run', 'hit_as_threw', 'interceptions', 'penalties', 'qb_rating', 'sack_percent', 'sacks', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays'])
NFL_passing2019 = NFL_passing2019.sort_values(by='player')
NFL_passing2019.reset_index(drop=True, inplace=True)
NFL_passing2022

,player,position,team_name,accuracy_percent,attempts,avg_depth_of_target,dropbacks,passing_snaps,pressure_to_sack_rate,scrambles,twp_rate,yards,ypa
0,Baker Mayfield,QB,LA,69.2,335,7.1,389,411,30.8,15,2.9,2163,6.5
1,Dak Prescott,QB,DAL,74.4,394,8.7,430,453,14.8,16,4.0,2860,7.3
2,Derek Carr,QB,LV,70.8,502,9.7,544,575,13.8,14,3.3,3522,7.0
3,Geno Smith,QB,SEA,77.8,572,8.2,652,693,20.7,33,4.2,4282,7.5
4,Jalen Hurts,QB,PHI,78.0,460,8.6,542,573,22.1,44,1.9,3701,8.0
5,Jared Goff,QB,DET,77.8,587,7.6,626,663,12.3,15,3.6,4438,7.6
6,Joe Burrow,QB,CIN,77.5,606,7.5,676,722,22.9,27,2.2,4474,7.4
7,Josh Allen,QB,BUF,74.9,567,9.7,659,696,14.7,57,4.2,4282,7.6
8,Justin Fields,QB,CHI,71.7,318,9.5,444,474,27.1,70,4.4,2242,7.1
9,Justin Herbert,QB,LAC,79.0,699,7.0,764,791,14.0,23,1.6,4739,6.8


In [8]:
#Add Kyler Murray's passing data to the dataframe containing current QB's pasing data
row = NFL_passing2022.iloc[11]
NFL_passing2023 = pd.concat([NFL_passing2023, row.to_frame().transpose()], ignore_index=True)
NFL_passing2023 = NFL_passing2023.sort_values(by='player')
NFL_passing2023.reset_index(drop=True, inplace=True)
NFL_passing2023

,player,position,team_name,accuracy_percent,attempts,avg_depth_of_target,dropbacks,passing_snaps,pressure_to_sack_rate,scrambles,twp_rate,yards,ypa
0,Aidan O'Connell,QB,LV,74.0,343,8.0,370,380,19.7,0,2.6,2218,6.5
1,Baker Mayfield,QB,TB,73.7,566,9.0,639,684,18.3,29,3.5,4045,7.1
2,Brock Purdy,QB,SF,78.3,444,8.6,498,525,14.6,17,3.4,4280,9.6
3,Bryce Young,QB,CAR,71.9,527,7.8,622,663,24.5,31,3.3,2877,5.5
4,C.J. Stroud,QB,HST,73.9,499,9.4,562,593,19.6,23,2.9,4108,8.2
5,Dak Prescott,QB,DAL,77.6,591,8.4,661,710,17.9,28,2.0,4519,7.6
6,Derek Carr,QB,NO,76.8,548,8.2,588,609,17.1,9,2.3,3878,7.1
7,Desmond Ridder,QB,ATL,72.7,388,8.5,439,464,22.3,19,5.6,2836,7.3
8,Gardner Minshew,QB,IND,74.2,490,7.8,546,580,17.9,21,3.8,3305,6.7
9,Geno Smith,QB,SEA,77.4,499,8.2,553,591,13.9,20,3.6,3624,7.3


In [9]:
#Average current NFL QB's NFL passing data
temp1 = NFL_passing2023.copy()
temp2 = NFL_passing2022.copy()
temp3 = NFL_passing2021.copy()
temp4 = NFL_passing2019.copy()
temp1 = temp1.sort_values(by='player')
temp2 = temp2.sort_values(by='player')
temp3 = temp3.sort_values(by='player')
temp4 = temp4.sort_values(by='player')
temp1.reset_index(drop=True, inplace=True)
temp2.reset_index(drop=True, inplace=True)
temp3.reset_index(drop=True, inplace=True)
temp4.reset_index(drop=True, inplace=True)
temp5 = pd.merge(temp1, temp2, on='player', how='outer', suffixes=('_1', '_2'))
temp6 = pd.merge(temp3, temp4, on='player', how='outer', suffixes=('_3', '_4'))
temp5['accuracy_percent_2'].fillna(temp5['accuracy_percent_1'], inplace=True)
temp5['accuracy_percent_1'].fillna(temp5['accuracy_percent_2'], inplace=True)
temp6['accuracy_percent_4'].fillna(temp6['accuracy_percent_3'], inplace=True)
temp6['accuracy_percent_3'].fillna(temp6['accuracy_percent_4'], inplace=True)
temp5['attempts_2'].fillna(temp5['attempts_1'], inplace=True)
temp5['attempts_1'].fillna(temp5['attempts_2'], inplace=True)
temp6['attempts_4'].fillna(temp6['attempts_3'], inplace=True)
temp6['attempts_3'].fillna(temp6['attempts_4'], inplace=True)
temp5['avg_depth_of_target_2'].fillna(temp5['avg_depth_of_target_1'], inplace=True)
temp5['avg_depth_of_target_1'].fillna(temp5['avg_depth_of_target_2'], inplace=True)
temp6['avg_depth_of_target_4'].fillna(temp6['avg_depth_of_target_3'], inplace=True)
temp6['avg_depth_of_target_3'].fillna(temp6['avg_depth_of_target_4'], inplace=True)
temp5['dropbacks_2'].fillna(temp5['dropbacks_1'], inplace=True)
temp5['dropbacks_1'].fillna(temp5['dropbacks_2'], inplace=True)
temp6['dropbacks_4'].fillna(temp6['dropbacks_3'], inplace=True)
temp6['dropbacks_3'].fillna(temp6['dropbacks_4'], inplace=True)
temp5['passing_snaps_2'].fillna(temp5['passing_snaps_1'], inplace=True)
temp5['passing_snaps_1'].fillna(temp5['passing_snaps_2'], inplace=True)
temp6['passing_snaps_4'].fillna(temp6['passing_snaps_3'], inplace=True)
temp6['passing_snaps_3'].fillna(temp6['passing_snaps_4'], inplace=True)
temp5['pressure_to_sack_rate_2'].fillna(temp5['pressure_to_sack_rate_1'], inplace=True)
temp5['pressure_to_sack_rate_1'].fillna(temp5['pressure_to_sack_rate_2'], inplace=True)
temp6['pressure_to_sack_rate_4'].fillna(temp6['pressure_to_sack_rate_3'], inplace=True)
temp6['pressure_to_sack_rate_3'].fillna(temp6['pressure_to_sack_rate_4'], inplace=True)
temp5['scrambles_2'].fillna(temp5['scrambles_1'], inplace=True)
temp5['scrambles_1'].fillna(temp5['scrambles_2'], inplace=True)
temp6['scrambles_4'].fillna(temp6['scrambles_3'], inplace=True)
temp6['scrambles_3'].fillna(temp6['scrambles_4'], inplace=True)
temp5['twp_rate_2'].fillna(temp5['twp_rate_1'], inplace=True)
temp5['twp_rate_1'].fillna(temp5['twp_rate_2'], inplace=True)
temp6['twp_rate_4'].fillna(temp6['twp_rate_3'], inplace=True)
temp6['twp_rate_3'].fillna(temp6['twp_rate_4'], inplace=True)
temp5['yards_2'].fillna(temp5['yards_1'], inplace=True)
temp5['yards_1'].fillna(temp5['yards_2'], inplace=True)
temp6['yards_4'].fillna(temp6['yards_3'], inplace=True)
temp6['yards_3'].fillna(temp6['yards_4'], inplace=True)
temp5['ypa_2'].fillna(temp5['ypa_1'], inplace=True)
temp5['ypa_1'].fillna(temp5['ypa_2'], inplace=True)
temp6['ypa_4'].fillna(temp6['ypa_3'], inplace=True)
temp6['ypa_3'].fillna(temp6['ypa_4'], inplace=True)
temp7 = pd.merge(temp5, temp6, on='player', how='outer')
temp7['accuracy_percent_2'].fillna(temp7['accuracy_percent_3'], inplace=True)
temp7['accuracy_percent_1'].fillna(temp7['accuracy_percent_4'], inplace=True)
temp7['accuracy_percent_4'].fillna(temp7['accuracy_percent_1'], inplace=True)
temp7['accuracy_percent_3'].fillna(temp7['accuracy_percent_2'], inplace=True)
temp7['attempts_2'].fillna(temp7['attempts_3'], inplace=True)
temp7['attempts_1'].fillna(temp7['attempts_4'], inplace=True)
temp7['attempts_4'].fillna(temp7['attempts_1'], inplace=True)
temp7['attempts_3'].fillna(temp7['attempts_2'], inplace=True)
temp7['avg_depth_of_target_2'].fillna(temp7['avg_depth_of_target_3'], inplace=True)
temp7['avg_depth_of_target_1'].fillna(temp7['avg_depth_of_target_4'], inplace=True)
temp7['avg_depth_of_target_4'].fillna(temp7['avg_depth_of_target_1'], inplace=True)
temp7['avg_depth_of_target_3'].fillna(temp7['avg_depth_of_target_2'], inplace=True)
temp7['dropbacks_2'].fillna(temp7['dropbacks_3'], inplace=True)
temp7['dropbacks_1'].fillna(temp7['dropbacks_4'], inplace=True)
temp7['dropbacks_4'].fillna(temp7['dropbacks_1'], inplace=True)
temp7['dropbacks_3'].fillna(temp7['dropbacks_2'], inplace=True)
temp7['passing_snaps_2'].fillna(temp7['passing_snaps_3'], inplace=True)
temp7['passing_snaps_1'].fillna(temp7['passing_snaps_4'], inplace=True)
temp7['passing_snaps_4'].fillna(temp7['passing_snaps_1'], inplace=True)
temp7['passing_snaps_3'].fillna(temp7['passing_snaps_2'], inplace=True)
temp7['pressure_to_sack_rate_2'].fillna(temp7['pressure_to_sack_rate_3'], inplace=True)
temp7['pressure_to_sack_rate_1'].fillna(temp7['pressure_to_sack_rate_4'], inplace=True)
temp7['pressure_to_sack_rate_4'].fillna(temp7['pressure_to_sack_rate_1'], inplace=True)
temp7['pressure_to_sack_rate_3'].fillna(temp7['pressure_to_sack_rate_2'], inplace=True)
temp7['scrambles_2'].fillna(temp7['scrambles_3'], inplace=True)
temp7['scrambles_1'].fillna(temp7['scrambles_4'], inplace=True)
temp7['scrambles_4'].fillna(temp7['scrambles_1'], inplace=True)
temp7['scrambles_3'].fillna(temp7['scrambles_2'], inplace=True)
temp7['twp_rate_2'].fillna(temp7['twp_rate_3'], inplace=True)
temp7['twp_rate_1'].fillna(temp7['twp_rate_4'], inplace=True)
temp7['twp_rate_4'].fillna(temp7['twp_rate_1'], inplace=True)
temp7['twp_rate_3'].fillna(temp7['twp_rate_2'], inplace=True)
temp7['yards_2'].fillna(temp7['yards_3'], inplace=True)
temp7['yards_1'].fillna(temp7['yards_4'], inplace=True)
temp7['yards_4'].fillna(temp7['yards_1'], inplace=True)
temp7['yards_3'].fillna(temp7['yards_2'], inplace=True)
temp7['ypa_2'].fillna(temp7['ypa_3'], inplace=True)
temp7['ypa_1'].fillna(temp7['ypa_4'], inplace=True)
temp7['ypa_4'].fillna(temp7['ypa_1'], inplace=True)
temp7['ypa_3'].fillna(temp7['ypa_2'], inplace=True)
temp7['Accuracy (%)'] = (temp7['accuracy_percent_1'] + temp7['accuracy_percent_2'] + temp7['accuracy_percent_3'] + temp7['accuracy_percent_4']) / 4
temp7['Accuracy (%)'] = temp7['Accuracy (%)'].astype('float')
temp7['Accuracy (%)'] = round(temp7['Accuracy (%)'], 2)
temp7['Attempts'] = (temp7['attempts_1'] + temp7['attempts_2'] + temp7['attempts_3'] + temp7['attempts_4']) / 4
temp7['Attempts'] = np.ceil(temp7['Attempts']).astype(int)
temp7['ADOT'] = (temp7['avg_depth_of_target_1'] + temp7['avg_depth_of_target_2'] + temp7['avg_depth_of_target_3'] + temp7['avg_depth_of_target_4']) / 4
temp7['ADOT'] = temp7['ADOT'].astype('float')
temp7['ADOT'] = round(temp7['ADOT'], 2)
temp7['Dropbacks'] = (temp7['dropbacks_1'] + temp7['dropbacks_2'] + temp7['dropbacks_3'] + temp7['dropbacks_4']) / 4
temp7['Dropbacks'] = np.ceil(temp7['Dropbacks']).astype(int)
temp7['Passing Snaps'] = (temp7['passing_snaps_1'] + temp7['passing_snaps_2'] + temp7['passing_snaps_3'] + temp7['passing_snaps_4']) / 4
temp7['Passing Snaps'] = np.ceil(temp7['Passing Snaps']).astype(int)
temp7['P2S%'] = (temp7['pressure_to_sack_rate_1'] + temp7['pressure_to_sack_rate_2'] + temp7['pressure_to_sack_rate_3'] + temp7['pressure_to_sack_rate_4']) / 4
temp7['P2S%'] = temp7['P2S%'].astype('float')
temp7['P2S%'] = round(temp7['P2S%'], 2)
temp7['Scrambles'] = (temp7['scrambles_1'] + temp7['scrambles_2'] + temp7['scrambles_3'] + temp7['scrambles_4']) / 4
temp7['Scrambles'] = np.ceil(temp7['Scrambles']).astype(int)
temp7['TWP%'] = (temp7['twp_rate_1'] + temp7['twp_rate_2'] + temp7['twp_rate_3'] + temp7['twp_rate_4']) / 4
temp7['TWP%'] = temp7['TWP%'].astype('float')
temp7['TWP%'] = round(temp7['TWP%'], 2)
temp7['Yards'] = (temp7['yards_1'] + temp7['yards_2'] + temp7['yards_3'] + temp7['yards_4']) / 4
temp7['Yards'] = np.ceil(temp7['Yards']).astype(int)
temp7['YPA'] = (temp7['ypa_1'] + temp7['ypa_2'] + temp7['ypa_3'] + temp7['ypa_4']) / 4
temp7['YPA'] = temp7['YPA'].astype('float')
temp7['YPA'] = round(temp7['YPA'], 2)
temp8 = temp7.copy()
qb_valid_data = temp8[['player', 'position_1', 'team_name_1', 'Accuracy (%)', 'Attempts', 'ADOT', 'Dropbacks', 'Passing Snaps', 'P2S%', 'Scrambles', 'TWP%', 'Yards', 'YPA']]
qb_valid_data = qb_valid_data.rename(columns = {'position_1' : 'Pos', 'team_name_1' : 'Team'})
qb_valid_data['Pos'].fillna('QB', inplace=True)
qbs = qb_training_data['player'].tolist()
qb_valid_data = qb_valid_data[qb_valid_data['player'].isin(qbs)]
qb_valid_data = qb_valid_data.sort_values(by='player')
qb_valid_data.reset_index(drop=True, inplace=True)
qb_valid_data

,player,Pos,Team,Accuracy (%),Attempts,ADOT,Dropbacks,Passing Snaps,P2S%,Scrambles,TWP%,Yards,YPA
0,Aidan O'Connell,QB,LV,74.00,343,8.00,370,380,19.70,0,2.60,2218,6.50
1,Baker Mayfield,QB,TB,71.15,464,8.62,527,565,24.60,22,3.50,3262,7.00
2,Brock Purdy,QB,SF,78.30,444,8.60,498,525,14.60,17,3.40,4280,9.60
3,Bryce Young,QB,CAR,71.90,527,7.80,622,663,24.50,31,3.30,2877,5.50
4,C.J. Stroud,QB,HST,73.90,499,9.40,562,593,19.60,23,2.90,4108,8.20
5,Dak Prescott,QB,DAL,76.35,545,8.78,596,637,14.80,22,3.05,4183,7.65
6,Desmond Ridder,QB,ATL,72.70,388,8.50,439,464,22.30,19,5.60,2836,7.30
7,Gardner Minshew,QB,IND,72.70,480,7.85,551,589,17.40,36,3.50,3288,6.85
8,Jalen Hurts,QB,PHI,75.22,466,9.02,546,576,15.80,48,2.70,3462,7.45
9,Jared Goff,QB,DET,76.58,578,7.42,618,655,13.62,12,3.25,4224,7.30


In [10]:
#Create a  polynomial regression model with L2 regularization , cross-validation, hyperparameter tuning, and early stopping to predict accuracy
X = qb_training_data[['Accuracy (%)']]
y = qb_valid_data[['Accuracy (%)']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 5
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
acc_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
acc_model.fit(X_train, y_train)
y_test_pred = acc_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.20748e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.51674e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.86749e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.2866e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill

Validation RMSE: 2.7354250394343493
Validation R2 Score -1.1525184358378069
    Accuracy (%)
0          78.14
17         71.93
15         77.66
1          80.61
8          77.70
    Accuracy (%)
0          74.00
17         76.40
15         72.55
1          71.15
8          75.22
[[75.50047288]
 [74.40230957]
 [75.41559069]
 [75.93726248]
 [75.42266421]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.48475e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.85125e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.09829e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.5525e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill

In [11]:
#Create a  polynomial regression model with L2 regularization , cross-validation, hyperparameter tuning, and early stopping to predict ADOT
X = qb_training_data[['ADOT']]
y = qb_valid_data[['ADOT']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 16
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
adot_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
adot_model.fit(X_train, y_train)
y_test_pred = adot_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 16 folds for each of 24 candidates, totalling 384 fits
Validation RMSE: 0.638848953281469
Validation R2 Score -0.09881963768854019
     ADOT
0    8.31
17  11.56
15   9.34
1   10.40
8   10.37
    ADOT
0   8.00
17  9.38
15  7.75
1   8.62
8   9.02
[[8.14577472]
 [8.32680795]
 [8.20314833]
 [8.26219302]
 [8.26052194]]


In [12]:
#Create a  polynomial regression model with L2 regularization , cross-validation, hyperparameter tuning, and early stopping to predict P2S%

X = qb_training_data[['P2S%']]
y = qb_valid_data[['P2S%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 5
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
p2s_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
p2s_model.fit(X_train, y_train)
y_test_pred = p2s_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.48055e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.44388e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.4758e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.32753e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill

Validation RMSE: 2.689780362804631
Validation R2 Score 0.14863280770423493
     P2S%
0   21.51
17  19.26
15  19.77
1   19.54
8   14.71
     P2S%
0   19.70
17  18.55
15  18.10
1   24.60
8   15.80
[[19.49908331]
 [18.59922512]
 [18.80319297]
 [18.71120747]
 [16.77951189]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.45518e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.5029e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.52122e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.33096e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [13]:
#Create a  polynomial regression model with L2 regularization , cross-validation, hyperparameter tuning, and early stopping to predict TWP%
X = qb_training_data[['TWP%']]
y = qb_valid_data[['TWP%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 5
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
twp_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
twp_model.fit(X_train, y_train)
y_test_pred = twp_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Validation RMSE: 0.7542232061417531
Validation R2 Score -1.9566145773531458
    TWP%
0   3.19
17  3.19
15  2.94
1   2.54
8   2.94
    TWP%
0   2.60
17  2.60
15  2.15
1   3.50
8   2.70
[[3.31737223]
 [3.31737223]
 [3.33114996]
 [3.35319433]
 [3.33114996]]


In [16]:
#Create a  polynomial regression model with L2 regularization , cross-validation, hyperparameter tuning, and early stopping to predict passing yards

X = qb_training_data[['Yards']]
y = qb_valid_data[['Yards']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 16
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
yards_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
yards_model.fit(X_train, y_train)
y_test_pred = yards_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 16 folds for each of 28 candidates, totalling 448 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.01661e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.0704e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.05978e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=5.07889e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill

Validation RMSE: 1018.3392383161633
Validation R2 Score -2.352006482740819
    Yards
0    3167
17   3435
15   3418
1    3837
8    3088
    Yards
0    2218
17   3342
15   2237
1    3262
8    3462
[[3701.88588513]
 [3776.34828586]
 [3771.62492462]
 [3888.04188696]
 [3679.93614761]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


In [14]:
#Create a  polynomial regression model with L2 regularization , cross-validation, hyperparameter tuning, and early stopping to predict YPA

X = qb_training_data[['YPA']]
y = qb_valid_data[['YPA']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 5
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
ypa_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
ypa_model.fit(X_train, y_train)
y_test_pred = ypa_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Validation RMSE: 0.6059196820397118
Validation R2 Score -0.13741282431317892
      YPA
0    7.93
17   8.74
15   8.19
1   11.03
8   10.56
     YPA
0   6.50
17  7.82
15  6.30
1   7.00
8   7.45
[[7.21183739]
 [7.1891922 ]
 [7.20456856]
 [7.12517063]
 [7.13831043]]


In [17]:
#Predict the NFL stats for prospect QBs and create a newdataframe containing all the data
newcols = {'accuracy_percent' : 'Accuracy (%)', 'attempts' : 'Attempts', 'avg_depth_of_target' : 'ADOT', 'pressure_to_sack_rate' : 'P2S%', 'yards' : 'Yards', 'ypa' : 'YPA', 'twp_rate' : 'TWP%', 'scrambles' : 'Scrambles', 'dropbacks' : 'Dropbacks'}
qb_predict_data = qb_testing_data.copy()
qb_predict_data = qb_predict_data.rename(columns=newcols)
prospectQBs = qb_predict_data['player'].tolist()
teams = qb_predict_data['team_name'].tolist()
X_acc = qb_predict_data[['Accuracy (%)']]
acc_predict = acc_model.predict(X_acc)
X_adot = qb_predict_data[['ADOT']]
adot_predict = adot_model.predict(X_adot)
X_p2s = qb_predict_data[['P2S%']]
p2s_predict = p2s_model.predict(X_p2s)
X_twp = qb_predict_data[['TWP%']]
twp_predict = twp_model.predict(X_twp)
X_yards = qb_predict_data[['Yards']]
yards_predict = yards_model.predict(X_yards)
X_ypa = qb_predict_data[['YPA']]
ypa_predict = ypa_model.predict(X_ypa)
pos = ['QB'] * 8
acc_predict = acc_predict.tolist()
adot_predict = adot_predict.tolist()
p2s_predict = p2s_predict.tolist()
twp_predict = twp_predict.tolist()
yards_predict = yards_predict.tolist()
ypa_predict = ypa_predict.tolist()
acc = []
adot = []
p2s = []
twp = []
yards = []
ypa = []
for l in acc_predict:
    l[0] = round(l[0], 2)
    acc.extend(l)
for l in adot_predict:
    l[0] = round(l[0], 2)
    adot.extend(l)
for l in p2s_predict:
    l[0] = round(l[0], 2)
    p2s.extend(l)
for l in twp_predict:
    l[0] = round(l[0], 2)
    twp.extend(l)
for l in yards_predict:
    l[0] = np.ceil(l[0]).astype(int)
    yards.extend(l)
for l in ypa_predict:
    l[0] = round(l[0], 2)
    ypa.extend(l)
qb_data = pd.DataFrame({'Player' : prospectQBs, 'Position' : pos, 'Team' : teams, 'Yards' : yards, 'YPA' : ypa, 'Accuracy (%)' : acc, 'ADOT' : adot, 'P2S%' : p2s, 'TWP%' : twp})
qb_data

,Player,Position,Team,Yards,YPA,Accuracy (%),ADOT,P2S%,TWP%
0,Bo Nix,QB,OREGON,3828,7.17,76.58,8.09,13.94,3.42
1,Caleb Williams,QB,USC,3845,7.18,75.26,8.21,18.54,3.35
2,Drake Maye,QB,N CAROLINA,3854,7.19,75.10,8.28,18.64,3.39
3,J.J. McCarthy,QB,MICHIGAN,3476,7.18,75.36,8.25,16.38,3.34
4,Jayden Daniels,QB,LSU,3708,7.17,75.60,8.20,21.15,3.43
5,Joe Milton III,QB,TENNESSEE,3313,7.14,74.16,8.52,14.82,3.35
6,Michael Penix Jr.,QB,WASHINGTON,3969,7.18,74.87,8.28,13.28,3.41
7,Spencer Rattler,QB,S CAROLINA,3652,7.21,75.64,8.10,18.86,3.31


In [18]:
#Get the college data for prospect WRs
college_receiving2023 = pd.read_csv("collegeWRreceiving_summary2023.csv")
college_receiving2023['Receiving Snaps'] = college_receiving2023['slot_snaps'] + college_receiving2023['wide_snaps']
college_receiving2023 = college_receiving2023.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wrs_predict_data = college_receiving2023.copy()
wrs_predict_data = wrs_predict_data[wrs_predict_data['player'].isin(prospects)]
wrs_predict_data = wrs_predict_data[wrs_predict_data['targets'] >= 70]
wrs_predict_data = wrs_predict_data[wrs_predict_data['position'] == 'WR']
wrs_predict_data = wrs_predict_data.sort_values(by='player')
wrs_predict_data.reset_index(drop=True, inplace=True)
wrs_predict_data['ATR'] = wrs_predict_data['avoided_tackles'] / wrs_predict_data['receptions']
wrs_predict_data['Target%'] = wrs_predict_data['targets'] / wrs_predict_data['Receiving Snaps']
wrs_predict_data = wrs_predict_data.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
newcol = {'player' : 'Player', 'contested_catch_rate' : 'CTC%', 'drop_rate' : 'Drop%', 'yards' : "Rec Yards", 'yards_after_catch_per_reception' : 'YAC/Rec', 'yards_per_reception' :'Yards/Rec', 'position' : 'Position', 'team_name' : 'Team'}
wrs_predict_data = wrs_predict_data.rename(columns=newcol)
wrs_predict_data

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,Adonai Mitchell,WR,TEXAS,33.3,2.2,704,3.0,15.6,0.111111,0.171149
1,Brenden Rice,WR,USC,25.0,4.3,791,5.2,17.6,0.200000,0.216049
2,Brian Thomas Jr.,WR,LSU,58.3,7.7,1079,6.1,18.0,0.183333,0.175355
3,Ja'Lynn Polk,WR,WASHINGTON,55.0,6.8,943,4.4,17.1,0.181818,0.209135
4,Javon Baker,WR,UCF,46.2,12.2,966,7.4,22.5,0.139535,0.209440
5,Keon Coleman,WR,FLORIDA ST,33.3,0.0,639,6.2,13.9,0.217391,0.216216
6,Malachi Corley,WR,W KENTUCKY,23.5,7.4,959,8.6,12.8,0.173333,0.302198
7,Malik Nabers,WR,LSU,47.6,5.5,1545,6.7,18.0,0.348837,0.294537
8,Marvin Harrison Jr.,WR,OHIO STATE,43.3,8.2,1211,6.4,18.1,0.074627,0.314050
9,Ricky Pearsall,WR,FLORIDA,50.0,3.0,963,5.8,14.8,0.138462,0.191630


In [19]:
NFL_receiving2023 = pd.read_csv("NFLreceiving_summary2023.csv")
NFL_receiving2023['Receiving Snaps'] = NFL_receiving2023['slot_snaps'] + NFL_receiving2023['wide_snaps']
NFL_receiving2023 = NFL_receiving2023.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wr_NFL_data2023 = NFL_receiving2023.copy()
wr_NFL_data2023 = wr_NFL_data2023[wr_NFL_data2023['targets'] >= 80]
wr_NFL_data2023 = wr_NFL_data2023[wr_NFL_data2023['position'] == 'WR']
wr_NFL_data2023 = wr_NFL_data2023.sort_values(by='player')
wr_NFL_data2023.reset_index(drop=True, inplace=True)
wr_NFL_data2023['ATR'] = wr_NFL_data2023['avoided_tackles'] / wr_NFL_data2023['receptions']
wr_NFL_data2023['Target%'] = wr_NFL_data2023['targets'] / wr_NFL_data2023['Receiving Snaps']
wr_NFL_data2023 = wr_NFL_data2023.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
wr_NFL_data2023 = wr_NFL_data2023.rename(columns=newcol)
wrs = wr_NFL_data2023['Player'].tolist()
wr_NFL_data2023

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,A.J. Brown,WR,PHI,45.2,3.6,1456,4.6,13.7,0.179245,0.251240
1,Adam Thielen,WR,CAR,50.0,3.7,1014,3.4,9.8,0.029126,0.200886
2,Amari Cooper,WR,CLV,51.7,6.5,1250,3.8,17.4,0.069444,0.212174
3,Amon-Ra St. Brown,WR,DET,50.0,3.3,1515,5.7,12.7,0.142857,0.263333
4,Brandin Cooks,WR,DAL,52.9,1.8,660,2.5,12.0,0.072727,0.144876
5,Brandon Aiyuk,WR,SF,61.5,2.6,1342,4.9,17.9,0.160000,0.213531
6,Calvin Ridley,WR,JAX,36.4,8.4,1016,2.7,13.4,0.052632,0.191860
7,CeeDee Lamb,WR,DAL,42.3,3.6,1749,5.0,13.0,0.148148,0.268366
8,Chris Godwin,WR,TB,59.3,4.6,1024,4.6,12.3,0.060241,0.204659
9,Chris Olave,WR,NO,40.0,6.5,1123,4.1,12.9,0.045977,0.247756


In [20]:
NFL_receiving2022 = pd.read_csv("NFLreceiving_summary2022.csv")
NFL_receiving2022['Receiving Snaps'] = NFL_receiving2022['slot_snaps'] + NFL_receiving2022['wide_snaps']
NFL_receiving2022 = NFL_receiving2022.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wr_NFL_data2022 = NFL_receiving2022.copy()
wr_NFL_data2022 = wr_NFL_data2022[wr_NFL_data2022['targets'] >= 80]
wr_NFL_data2022 = wr_NFL_data2022[wr_NFL_data2022['player'].isin(wrs)]
wr_NFL_data2022 = wr_NFL_data2022.sort_values(by='player')
wr_NFL_data2022.reset_index(drop=True, inplace=True)
wr_NFL_data2022['ATR'] = wr_NFL_data2022['avoided_tackles'] / wr_NFL_data2022['receptions']
wr_NFL_data2022['Target%'] = wr_NFL_data2022['targets'] / wr_NFL_data2022['Receiving Snaps']
wr_NFL_data2022 = wr_NFL_data2022.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
wr_NFL_data2022 = wr_NFL_data2022.rename(columns=newcol)
wr_NFL_data2022

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,A.J. Brown,WR,PHI,50.0,6.4,1496,6.4,17.0,0.193182,0.224590
1,Adam Thielen,WR,MIN,21.1,1.4,716,2.3,10.2,0.014286,0.143654
2,Amari Cooper,WR,CLV,55.6,7.1,1160,3.8,14.9,0.089744,0.209302
3,Amon-Ra St. Brown,WR,DET,38.5,3.6,1161,5.1,11.0,0.084906,0.275794
4,Brandin Cooks,WR,HST,55.6,9.7,699,3.5,12.5,0.017857,0.196429
5,Brandon Aiyuk,WR,SF,40.9,6.0,1015,4.7,13.0,0.205128,0.197880
6,CeeDee Lamb,WR,DAL,46.2,4.5,1359,4.6,12.7,0.130841,0.249158
7,Chris Godwin,WR,TB,57.7,4.6,1023,5.0,9.8,0.105769,0.221325
8,Chris Olave,WR,NO,33.3,4.0,1044,3.0,14.5,0.027778,0.253898
9,Christian Kirk,WR,JAX,27.8,8.7,1108,4.6,13.2,0.083333,0.190402


In [21]:
NFL_receiving2021 = pd.read_csv("NFLreceiving_summary2021.csv")
NFL_receiving2021['Receiving Snaps'] = NFL_receiving2021['slot_snaps'] + NFL_receiving2021['wide_snaps']
NFL_receiving2021 = NFL_receiving2021.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wr_NFL_data2021 = NFL_receiving2021.copy()
wr_NFL_data2021 = wr_NFL_data2021[wr_NFL_data2021['targets'] >= 85]
wr_NFL_data2021 = wr_NFL_data2021[wr_NFL_data2021['player'].isin(wrs)]
wr_NFL_data2021 = wr_NFL_data2021.sort_values(by='player')
wr_NFL_data2021.reset_index(drop=True, inplace=True)
wr_NFL_data2021['ATR'] = wr_NFL_data2021['avoided_tackles'] / wr_NFL_data2021['receptions']
wr_NFL_data2021['Target%'] = wr_NFL_data2021['targets'] / wr_NFL_data2021['Receiving Snaps']
wr_NFL_data2021 = wr_NFL_data2021.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
wr_NFL_data2021 = wr_NFL_data2021.rename(columns=newcol)
wr_NFL_data2021

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,A.J. Brown,WR,TEN,50.0,11.3,869,3.6,13.8,0.111111,0.276712
1,Adam Thielen,WR,MIN,66.7,2.9,726,3.8,10.8,0.104478,0.182203
2,Amari Cooper,WR,DAL,60.9,4.2,865,3.3,12.7,0.161765,0.174912
3,Amon-Ra St. Brown,WR,DET,50.0,2.2,912,4.8,10.1,0.088889,0.212963
4,Brandin Cooks,WR,HST,57.1,4.3,1037,3.9,11.5,0.066667,0.230906
5,CeeDee Lamb,WR,DAL,70.0,8.1,1102,5.5,13.9,0.240506,0.199313
6,Chris Godwin,WR,TB,63.6,4.9,1103,6.0,11.3,0.153061,0.214162
7,Christian Kirk,WR,ARZ,70.6,7.2,984,3.1,12.8,0.038961,0.174439
8,Cooper Kupp,WR,LA,52.2,4.6,1947,5.9,13.4,0.165517,0.291217
9,Courtland Sutton,WR,DEN,42.9,1.7,776,2.5,13.4,0.051724,0.165794


In [22]:
NFL_receiving2019 = pd.read_csv("NFLreceiving_summary2019.csv")
NFL_receiving2019['Receiving Snaps'] = NFL_receiving2019['slot_snaps'] + NFL_receiving2019['wide_snaps']
NFL_receiving2019 = NFL_receiving2019.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wr_NFL_data2019 = NFL_receiving2019.copy()
wr_NFL_data2019 = wr_NFL_data2019[wr_NFL_data2019['targets'] >= 80]
wr_NFL_data2019 = wr_NFL_data2019[wr_NFL_data2019['player'].isin(wrs)]
wr_NFL_data2019 = wr_NFL_data2019.sort_values(by='player')
wr_NFL_data2019.reset_index(drop=True, inplace=True)
wr_NFL_data2019['ATR'] = wr_NFL_data2019['avoided_tackles'] / wr_NFL_data2019['receptions']
wr_NFL_data2019['Target%'] = wr_NFL_data2019['targets'] / wr_NFL_data2019['Receiving Snaps']
wr_NFL_data2019 = wr_NFL_data2019.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
wr_NFL_data2019 = wr_NFL_data2019.rename(columns=newcol)
wr_NFL_data2019

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,A.J. Brown,WR,TEN,31.8,7.1,1051,8.9,20.2,0.307692,0.201923
1,Amari Cooper,WR,DAL,38.5,8.1,1189,2.9,15.1,0.075949,0.211293
2,Calvin Ridley,WR,ATL,50.0,4.5,866,2.2,13.7,0.063492,0.164557
3,Chris Godwin,WR,TB,66.7,1.1,1333,6.7,15.5,0.197674,0.182986
4,Christian Kirk,WR,ARZ,31.3,4.2,709,4.4,10.4,0.058824,0.184701
5,Cooper Kupp,WR,LA,33.3,3.1,1161,5.8,12.4,0.085106,0.213592
6,Courtland Sutton,WR,DEN,56.5,4.0,1112,5.0,15.4,0.222222,0.200000
7,Curtis Samuel,WR,CAR,30.0,11.5,627,2.9,11.6,0.055556,0.154192
8,D.K. Metcalf,WR,SEA,53.1,12.1,900,4.8,15.5,0.155172,0.167530
9,DJ Moore,WR,CAR,51.7,5.4,1175,4.5,13.5,0.149425,0.214521


In [23]:
#Average the wide reciever NFL recieving data 
wrs2019 = wr_NFL_data2019['Player'].tolist()
wrs2021 = wr_NFL_data2021['Player'].tolist()
wrs2022 = wr_NFL_data2022['Player'].tolist()
wr_valid_data = wr_NFL_data2023.copy()
for w in wrs:
    if w in wrs2022:
        if w in wrs2021:
            if w in wrs2019:
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'CTC%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'CTC%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'CTC%'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'CTC%'].iloc[0] + wr_NFL_data2019.loc[wr_NFL_data2019['Player'] == w, 'CTC%'].iloc[0]) / 4
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'Drop%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Drop%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Drop%'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'Drop%'].iloc[0] + wr_NFL_data2019.loc[wr_NFL_data2019['Player'] == w, 'Drop%'].iloc[0]) / 4
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'Rec Yards'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Rec Yards'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Rec Yards'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'Rec Yards'].iloc[0] + wr_NFL_data2019.loc[wr_NFL_data2019['Player'] == w, 'Rec Yards'].iloc[0]) / 4
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'YAC/Rec'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'YAC/Rec'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'YAC/Rec'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'YAC/Rec'].iloc[0] + wr_NFL_data2019.loc[wr_NFL_data2019['Player'] == w, 'YAC/Rec'].iloc[0]) / 4
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'ATR'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'ATR'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'ATR'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'ATR'].iloc[0] + wr_NFL_data2019.loc[wr_NFL_data2019['Player'] == w, 'ATR'].iloc[0]) / 4
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'Target%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Target%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Target%'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'Target%'].iloc[0] + wr_NFL_data2019.loc[wr_NFL_data2019['Player'] == w, 'Target%'].iloc[0]) / 4
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'Yards/Rec'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Yards/Rec'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Yards/Rec'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'Yards/Rec'].iloc[0] + wr_NFL_data2019.loc[wr_NFL_data2019['Player'] == w, 'Yards/Rec'].iloc[0]) / 4
            else:
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'CTC%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'CTC%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'CTC%'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'CTC%'].iloc[0]) / 3
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'Drop%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Drop%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Drop%'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'Drop%'].iloc[0]) / 3
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'Rec Yards'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Rec Yards'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Rec Yards'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'Rec Yards'].iloc[0]) / 3
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'YAC/Rec'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'YAC/Rec'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'YAC/Rec'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'YAC/Rec'].iloc[0]) / 3
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'ATR'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'ATR'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'ATR'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'ATR'].iloc[0]) / 3
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'Target%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Target%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Target%'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'Target%'].iloc[0]) / 3
                wr_valid_data.loc[wr_valid_data['Player'] == w, 'Yards/Rec'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Yards/Rec'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Yards/Rec'].iloc[0] + wr_NFL_data2021.loc[wr_NFL_data2021['Player'] == w, 'Yards/Rec'].iloc[0]) / 3
        else:
            wr_valid_data.loc[wr_valid_data['Player'] == w, 'CTC%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'CTC%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'CTC%'].iloc[0]) / 2
            wr_valid_data.loc[wr_valid_data['Player'] == w, 'Drop%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Drop%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Drop%'].iloc[0]) / 2
            wr_valid_data.loc[wr_valid_data['Player'] == w, 'Rec Yards'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Rec Yards'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Rec Yards'].iloc[0]) / 2
            wr_valid_data.loc[wr_valid_data['Player'] == w, 'YAC/Rec'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'YAC/Rec'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'YAC/Rec'].iloc[0]) / 2
            wr_valid_data.loc[wr_valid_data['Player'] == w, 'ATR'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'ATR'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'ATR'].iloc[0]) / 2
            wr_valid_data.loc[wr_valid_data['Player'] == w, 'Target%'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Target%'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Target%'].iloc[0]) / 2
            wr_valid_data.loc[wr_valid_data['Player'] == w, 'Yards/Rec'] = (wr_valid_data.loc[wr_valid_data['Player'] == w, 'Yards/Rec'].iloc[0] + wr_NFL_data2022.loc[wr_NFL_data2022['Player'] == w, 'Yards/Rec'].iloc[0]) / 2
wr_valid_data

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,A.J. Brown,WR,PHI,44.250000,7.100000,1218.000000,5.875000,16.175000,0.197808,0.238616
1,Adam Thielen,WR,CAR,45.933333,2.666667,818.666667,3.166667,10.266667,0.049297,0.175581
2,Amari Cooper,WR,CLV,51.675000,6.475000,1116.000000,3.450000,15.025000,0.099226,0.201920
3,Amon-Ra St. Brown,WR,DET,46.166667,3.033333,1196.000000,5.200000,11.266667,0.105551,0.250697
4,Brandin Cooks,WR,DAL,55.200000,5.266667,798.666667,3.300000,12.000000,0.052417,0.190737
5,Brandon Aiyuk,WR,SF,51.200000,4.300000,1178.500000,4.800000,15.450000,0.182564,0.205705
6,Calvin Ridley,WR,JAX,36.400000,8.400000,1016.000000,2.700000,13.400000,0.052632,0.191860
7,CeeDee Lamb,WR,DAL,52.833333,5.400000,1403.333333,5.033333,13.200000,0.173165,0.238946
8,Chris Godwin,WR,TB,61.825000,3.800000,1120.750000,5.575000,12.225000,0.129186,0.205783
9,Chris Olave,WR,NO,36.650000,5.250000,1083.500000,3.550000,13.700000,0.036877,0.250827


In [24]:
college_receiving2022 = pd.read_csv("collegeWRreceiving_summary2022.csv")
college_receiving2022['Receiving Snaps'] = college_receiving2022['slot_snaps'] + college_receiving2022['wide_snaps']
college_receiving2022 = college_receiving2022.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wr_college_data2022 = college_receiving2022.copy()
wr_college_data2022 = wr_college_data2022[wr_college_data2022['targets'] >= 50]
wr_college_data2022 = wr_college_data2022[wr_college_data2022['player'].isin(wrs)]
wr_college_data2022 = wr_college_data2022.sort_values(by='player')
wr_college_data2022.reset_index(drop=True, inplace=True)
wr_college_data2022['ATR'] = wr_college_data2022['avoided_tackles'] / wr_college_data2022['receptions']
wr_college_data2022['Target%'] = wr_college_data2022['targets'] / wr_college_data2022['Receiving Snaps']
wr_college_data2022 = wr_college_data2022.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
wr_college_data2022 = wr_college_data2022.rename(columns=newcol)
wr_college_data2022

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,Jayden Reed,WR,MICH STATE,64.7,8.3,636,4.0,11.6,0.054545,0.224000
1,Jonathan Mingo,WR,OLE MISS,46.7,5.9,808,7.3,16.8,0.166667,0.224432
2,Jordan Addison,WR,USC,28.6,3.6,810,7.3,15.0,0.111111,0.244755
3,Josh Downs,WR,N CAROLINA,75.0,3.5,929,4.0,11.2,0.072289,0.241860
4,Puka Nacua,WR,BYU,50.0,5.9,625,7.4,13.0,0.125000,0.370166
5,Rashee Rice,WR,SMU,48.5,8.6,1344,6.2,14.0,0.197917,0.333333
6,Zay Flowers,WR,BOSTON COL,58.3,9.3,1077,6.4,13.8,0.192308,0.246521


In [25]:
college_receiving2021 = pd.read_csv("college_receiving_summary2021.csv")
college_receiving2021['Receiving Snaps'] = college_receiving2021['slot_snaps'] + college_receiving2021['wide_snaps']
college_receiving2021 = college_receiving2021.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wr_college_data2021 = college_receiving2021.copy()
wr_college_data2021 = wr_college_data2021[wr_college_data2021['targets'] >= 50]
wr_college_data2021 = wr_college_data2021[wr_college_data2021['player'].isin(wrs)]
wr_college_data2021 = wr_college_data2021.sort_values(by='player')
wr_college_data2021.reset_index(drop=True, inplace=True)
wr_college_data2021['ATR'] = wr_college_data2021['avoided_tackles'] / wr_college_data2021['receptions']
wr_college_data2021['Target%'] = wr_college_data2021['targets'] / wr_college_data2021['Receiving Snaps']
wr_college_data2021 = wr_college_data2021.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
wr_college_data2021 = wr_college_data2021.rename(columns=newcol)
wr_college_data2021

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,Chris Olave,WR,OHIO STATE,62.5,7.1,936,4.2,14.4,0.015385,0.237089
1,Drake London,WR,USC,65.4,8.3,1084,5.2,12.3,0.250000,0.369565
2,Garrett Wilson,WR,OHIO STATE,61.5,7.9,1058,6.0,15.1,0.271429,0.276423
3,Jaxon Smith-Njigba,WR,OHIO STATE,100.0,7.0,1259,7.6,15.7,0.200000,0.260163
4,Jayden Reed,WR,MICH STATE,48.0,10.2,946,6.5,17.8,0.264151,0.255618
5,Jordan Addison,WR,PITTSBURGH,58.3,8.6,1353,6.8,15.9,0.200000,0.242424
6,Josh Downs,WR,N CAROLINA,30.8,7.5,1273,7.5,13.0,0.142857,0.298283
7,Puka Nacua,WR,BYU,64.7,4.4,796,6.0,18.5,0.186047,0.273973
8,Rashee Rice,WR,SMU,42.9,1.5,682,5.9,10.7,0.250000,0.233728
9,Romeo Doubs,WR,NEVADA,60.0,4.8,1114,4.0,13.9,0.125000,0.216110


In [26]:
college_receiving2019 = pd.read_csv("college_receiving_summary2019.csv")
college_receiving2019['Receiving Snaps'] = college_receiving2019['slot_snaps'] + college_receiving2019['wide_snaps']
college_receiving2019 = college_receiving2019.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wr_college_data2019 = college_receiving2019.copy()
wr_college_data2019 = wr_college_data2019[wr_college_data2019['targets'] >= 40]
wr_college_data2019 = wr_college_data2019[wr_college_data2019['player'].isin(wrs)]
wr_college_data2019 = wr_college_data2019.sort_values(by='player')
wr_college_data2019.reset_index(drop=True, inplace=True)
wr_college_data2019['ATR'] = wr_college_data2019['avoided_tackles'] / wr_college_data2019['receptions']
wr_college_data2019['Target%'] = wr_college_data2019['targets'] / wr_college_data2019['Receiving Snaps']
wr_college_data2019 = wr_college_data2019.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
wr_college_data2019 = wr_college_data2019.rename(columns=newcol)
wr_college_data2019

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,Amon-Ra St. Brown,WR,USC,41.7,6.8,879,5.1,12.9,0.250000,0.185629
1,Brandon Aiyuk,WR,ARIZONA ST,25.0,8.5,1192,10.9,18.3,0.215385,0.242054
2,CeeDee Lamb,WR,OKLAHOMA,30.0,3.8,1033,11.1,20.7,0.360000,0.248227
3,Chris Olave,WR,OHIO STATE,50.0,6.8,705,3.0,17.2,0.048780,0.286385
4,DeVonta Smith,WR,ALABAMA,54.5,3.0,1203,11.5,18.8,0.234375,0.240688
5,Drake London,WR,USC,40.0,5.4,533,6.6,15.2,0.342857,0.153374
6,Elijah Moore,WR,OLE MISS,50.0,8.2,850,5.0,12.7,0.194030,0.316804
7,George Pickens,WR,GEORGIA,33.3,0.0,498,3.2,15.1,0.030303,0.219409
8,Ja'Marr Chase,WR,LSU,54.2,5.4,1457,8.0,20.8,0.300000,0.257895
9,Jerry Jeudy,WR,ALABAMA,50.0,9.0,959,6.9,13.5,0.197183,0.293605


In [27]:
college_receiving2018 = pd.read_csv("college_receiving_summary2018.csv")
college_receiving2018['Receiving Snaps'] = college_receiving2018['slot_snaps'] + college_receiving2018['wide_snaps']
college_receiving2018 = college_receiving2018.drop(columns = ['player_id', 'player_game_count', 'avg_depth_of_target', 'caught_percent', 'contested_receptions', 'contested_targets', 'fumbles', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'grades_hands_fumble', 'grades_offense', 'grades_pass_block', 'grades_hands_drop', 'grades_pass_route', 'interceptions', 'penalties', 'inline_rate', 'inline_snaps', 'longest', 'pass_block_rate', 'pass_blocks', 'touchdowns', 'route_rate', 'routes', 'slot_rate', 'slot_snaps', 'targeted_qb_rating',  'wide_rate', 'pass_plays', 'wide_snaps', 'yards_after_catch', 'yprr'])
wr_college_data2018 = college_receiving2018.copy()
wr_college_data2018 = wr_college_data2018[wr_college_data2018['targets'] >= 30]
wr_college_data2018 = wr_college_data2018[wr_college_data2018['player'].isin(wrs)]
wr_college_data2018 = wr_college_data2018.sort_values(by='player')
wr_college_data2018.reset_index(drop=True, inplace=True)
wr_college_data2018['ATR'] = wr_college_data2018['avoided_tackles'] / wr_college_data2018['receptions']
wr_college_data2018['Target%'] = wr_college_data2018['targets'] / wr_college_data2018['Receiving Snaps']
wr_college_data2018 = wr_college_data2018.drop(columns = ['avoided_tackles', 'receptions', 'targets', 'Receiving Snaps'])
wr_college_data2018 = wr_college_data2018.rename(columns=newcol)
wr_college_data2018

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,A.J. Brown,WR,OLE MISS,40.0,5.6,1307,7.1,15.6,0.202381,0.254989
1,Amon-Ra St. Brown,WR,USC,33.3,6.3,750,3.8,12.5,0.133333,0.288591
2,Brandon Aiyuk,WR,ARIZONA ST,16.7,7.7,413,8.8,17.2,0.333333,0.184615
3,CeeDee Lamb,WR,OKLAHOMA,50.0,8.9,882,7.2,17.3,0.156863,0.221875
4,D.K. Metcalf,WR,OLE MISS,45.5,10.3,569,9.7,21.9,0.115385,0.196262
5,DeVonta Smith,WR,ALABAMA,28.6,0.0,495,9.0,18.3,0.185185,0.158333
6,Deebo Samuel,WR,S CAROLINA,47.6,6.1,882,9.5,14.2,0.338710,0.254545
7,Diontae Johnson,WR,TOLEDO,0.0,14.0,663,9.0,15.4,0.232558,0.250784
8,Elijah Moore,WR,OLE MISS,33.3,5.1,407,5.5,11.0,0.027027,0.201613
9,Jakobi Meyers,WR,NC STATE,75.0,3.3,1029,4.1,11.6,0.134831,0.317935


In [28]:
#Average out the college data for current WR prospects
wrList = wr_college_data2018['Player'].tolist()
wr_train_data = wr_college_data2018.copy()
temp1 = wr_college_data2019.copy()
temp2 = wr_college_data2021.copy()
temp3 = wr_college_data2022.copy()
temp1 = temp1[~temp1['Player'].isin(wrList)]

#Add unique rows to the 2018 college dataframe
wr_train_data = pd.concat([wr_train_data, temp1]).drop_duplicates(keep=False)
tempWR = temp1['Player'].tolist()
collegeWR2019 = wr_college_data2019.copy()
collegeWR2019 = collegeWR2019[~collegeWR2019['Player'].isin(tempWR)]
wrList = wr_train_data['Player'].tolist()
temp2 = temp2[temp2['Player'].isin(wrList)]
tempWR = temp2['Player'].tolist()
collegeWR2021 = wr_college_data2021.copy()
collegeWR2021 = collegeWR2021[~collegeWR2021['Player'].isin(tempWR)]
wr_train_data = pd.concat([wr_train_data, temp2]).drop_duplicates(keep=False)
wrList = wr_train_data['Player'].tolist()
temp3 = temp3[~temp3['Player'].isin(wrList)]
wr_train_data = pd.concat([wr_train_data, temp3]).drop_duplicates(keep=False)
tempWR = temp3['Player'].tolist()
collegeWR2022 = wr_college_data2022.copy()
collegeWR2022 = collegeWR2022[~collegeWR2022['Player'].isin(tempWR)]
wr_train_data.reset_index(drop=True, inplace=True)
wrs = wr_train_data['Player'].tolist()
wr22 = collegeWR2022['Player'].tolist()
wr21 = collegeWR2021['Player'].tolist()
wr19 = collegeWR2019['Player'].tolist()
for w in wrs:
    if w in wr22:
        if w in wr21:
            if w in wr19:
                wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'CTC%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'CTC%'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'CTC%'].iloc[0]) / 4
                wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Drop%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Drop%'].iloc[0]  + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Drop%'].iloc[0]) / 4
                wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Rec Yards'].iloc[0]) / 4
                wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'YAC/Rec'].iloc[0]) / 4
                wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'ATR'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'ATR'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'ATR'].iloc[0]) / 4
                wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Target%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Target%'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Target%'].iloc[0]) / 4
                wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Yards/Rec'].iloc[0]) / 4
            else:
                wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'CTC%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'CTC%'].iloc[0]) / 3
                wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Drop%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Drop%'].iloc[0]) / 3
                wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Rec Yards'].iloc[0]) / 3
                wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'YAC/Rec'].iloc[0]) / 3
                wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'ATR'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'ATR'].iloc[0]) / 3
                wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Target%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Target%'].iloc[0]) / 3
                wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Yards/Rec'].iloc[0]) / 3
        else:
            wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'CTC%'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Drop%'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Rec Yards'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'YAC/Rec'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'ATR'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Target%'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2022.loc[collegeWR2022['Player'] == w, 'Yards/Rec'].iloc[0]) / 2
    elif w in wr21:
        if w in wr19:
            wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'CTC%'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'CTC%'].iloc[0]) / 3
            wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Drop%'].iloc[0]  + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Drop%'].iloc[0]) / 3
            wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Rec Yards'].iloc[0]) / 3
            wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'YAC/Rec'].iloc[0]) / 3
            wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'ATR'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'ATR'].iloc[0]) / 3
            wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Target%'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Target%'].iloc[0]) / 3
            wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Yards/Rec'].iloc[0]) / 3
        else:
            wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'CTC%'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Drop%'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Rec Yards'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'YAC/Rec'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'ATR'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Target%'].iloc[0]) / 2
            wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2021.loc[collegeWR2021['Player'] == w, 'Yards/Rec'].iloc[0]) / 2
    elif w in wr19:
        wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'CTC%'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'CTC%'].iloc[0]) / 2
        wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Drop%'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Drop%'].iloc[0]) / 2
        wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Rec Yards'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Rec Yards'].iloc[0]) / 2
        wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'YAC/Rec'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'YAC/Rec'].iloc[0]) / 2
        wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'ATR'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'ATR'].iloc[0]) / 2
        wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Target%'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Target%'].iloc[0]) / 2
        wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'] = (wr_train_data.loc[wr_train_data['Player'] == w, 'Yards/Rec'].iloc[0] + collegeWR2019.loc[collegeWR2019['Player'] == w, 'Yards/Rec'].iloc[0]) / 2

#Average duplicates with differing stat values        
wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'CTC%']  = (wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'CTC%'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'CTC%'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Drop%']  = (wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Drop%'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Drop%'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Rec Yards']  = (wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Rec Yards'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Rec Yards'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'YAC/Rec']  = (wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'YAC/Rec'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'YAC/Rec'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'ATR']  = (wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'ATR'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'ATR'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Target%']  = (wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Target%'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Target%'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Yards/Rec']  = (wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Yards/Rec'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Chris Olave', 'Yards/Rec'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'CTC%']  = (wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'CTC%'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'CTC%'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Drop%']  = (wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Drop%'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Drop%'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Rec Yards']  = (wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Rec Yards'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Rec Yards'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'YAC/Rec']  = (wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'YAC/Rec'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'YAC/Rec'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'ATR']  = (wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'ATR'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'ATR'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Target%']  = (wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Target%'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Target%'].iloc[1]) / 2
wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Yards/Rec']  = (wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Yards/Rec'].iloc[0] + wr_train_data.loc[wr_train_data['Player'] == 'Drake London', 'Yards/Rec'].iloc[1]) / 2
wr_train_data = wr_train_data.drop_duplicates(subset=['Player'])
wr_train_data.reset_index(drop=True, inplace=True)
wr_train_data = wr_train_data.sort_values(by='Player')
wr_train_data

,Player,Position,Team,CTC%,Drop%,Rec Yards,YAC/Rec,Yards/Rec,ATR,Target%
0,A.J. Brown,WR,OLE MISS,40.000,5.600,1307.00,7.100,15.600,0.202381,0.254989
1,Amon-Ra St. Brown,WR,USC,37.500,6.550,814.50,4.450,12.700,0.191667,0.237110
2,Brandon Aiyuk,WR,ARIZONA ST,20.850,8.100,802.50,9.850,17.750,0.274359,0.213335
3,CeeDee Lamb,WR,OKLAHOMA,40.000,6.350,957.50,9.150,19.000,0.258431,0.235051
19,Chris Olave,WR,OHIO STATE,56.250,6.950,820.50,3.600,15.800,0.032083,0.261737
4,D.K. Metcalf,WR,OLE MISS,45.500,10.300,569.00,9.700,21.900,0.115385,0.196262
5,DeVonta Smith,WR,ALABAMA,41.550,1.500,849.00,10.250,18.550,0.209780,0.199511
6,Deebo Samuel,WR,S CAROLINA,47.600,6.100,882.00,9.500,14.200,0.338710,0.254545
7,Diontae Johnson,WR,TOLEDO,0.000,14.000,663.00,9.000,15.400,0.232558,0.250784
20,Drake London,WR,USC,52.700,6.850,808.50,5.900,13.750,0.296429,0.261470


In [29]:
NFL_rushing2023 = pd.read_csv("NFLrushing_summary2023.csv")
NFL_rushing2023 = NFL_rushing2023.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
NFL_rushing2023['ATR'] = NFL_rushing2023['avoided_tackles'] / NFL_rushing2023['attempts']
NFL_rushing2023['YCO'] = NFL_rushing2023['yards_after_contact'] / NFL_rushing2023['attempts']
NFLRB_data2023 = NFL_rushing2023.copy()
NFLRB_data2023 = NFLRB_data2023[NFLRB_data2023['position'] == 'HB']
NFLRB_data2023 = NFLRB_data2023[NFLRB_data2023['attempts'] > 140]
NFLRB_data2023 = NFLRB_data2023.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
NFLRB_data2023 = NFLRB_data2023.sort_values(by='player')
NFLRB_data2023.reset_index(drop=True, inplace=True)
rbs = NFLRB_data2023['player'].tolist()
NFLRB_data2023

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,AJ Dillon,HB,GB,11.7,43.1,8,3.4,0.117978,2.691011
1,Aaron Jones,HB,GB,18.6,58.5,15,4.5,0.181818,3.160839
2,Alexander Mattison,HB,MIN,16.9,52.5,17,3.9,0.205556,2.755556
3,Alvin Kamara,HB,NO,13.7,34.5,15,3.9,0.122222,2.511111
4,Austin Ekeler,HB,LAC,20.4,49.4,7,3.5,0.173184,2.642458
5,Bijan Robinson,HB,ATL,24.5,78.6,30,4.5,0.242991,3.098131
6,Breece Hall,HB,NYJ,40.5,88.6,20,4.5,0.207207,3.427928
7,Brian Robinson,HB,WAS,23.4,64.3,19,4.1,0.207865,2.926966
8,Christian McCaffrey,HB,SF,36.4,65.5,44,5.4,0.220588,3.415441
9,Chuba Hubbard,HB,CAR,18.8,50.7,21,3.8,0.172269,2.865546


In [30]:
NFL_rushing2022 = pd.read_csv("NFLrushing_summary2022.csv")
NFL_rushing2022 = NFL_rushing2022.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
NFL_rushing2022['ATR'] = NFL_rushing2022['avoided_tackles'] / NFL_rushing2022['attempts']
NFL_rushing2022['YCO'] = NFL_rushing2022['yards_after_contact'] / NFL_rushing2022['attempts']
NFLRB_data2022 = NFL_rushing2022.copy()
NFLRB_data2022 = NFLRB_data2022[NFLRB_data2022['player'].isin(rbs)]
NFLRB_data2022 = NFLRB_data2022[NFLRB_data2022['attempts'] > 140]
NFLRB_data2022 = NFLRB_data2022.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
NFLRB_data2022 = NFLRB_data2022.sort_values(by='player')
NFLRB_data2022.reset_index(drop=True, inplace=True)
NFLRB_data2022

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,AJ Dillon,HB,GB,15.6,36.7,15,4.2,0.124324,2.897297
1,Aaron Jones,HB,GB,24.9,71.4,31,5.3,0.247664,3.196262
2,Alvin Kamara,HB,NO,13.4,47.6,20,4.0,0.152466,2.717489
3,Austin Ekeler,HB,LAC,25.5,67.3,22,4.5,0.210784,2.990196
4,Brian Robinson,HB,WAS,13.9,44.0,18,3.9,0.156098,2.692683
5,Christian McCaffrey,HB,SF,29.9,47.6,28,4.7,0.147541,2.897541
6,Dameon Pierce,HB,HST,29.6,98.5,22,4.3,0.281818,3.281818
7,David Montgomery,HB,CHI,8.9,69.0,17,4.0,0.228856,2.895522
8,Derrick Henry,HB,TEN,27.6,70.7,37,4.4,0.197708,3.601719
9,Devin Singletary,HB,BUF,32.7,53.0,21,4.6,0.186441,2.847458


In [31]:
NFL_rushing2021 = pd.read_csv("NFLrushing_summary2021.csv")
NFL_rushing2021 = NFL_rushing2021.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
NFL_rushing2021['ATR'] = NFL_rushing2021['avoided_tackles'] / NFL_rushing2021['attempts']
NFL_rushing2021['YCO'] = NFL_rushing2021['yards_after_contact'] / NFL_rushing2021['attempts']
NFLRB_data2021 = NFL_rushing2021.copy()
NFLRB_data2021 = NFLRB_data2021[NFLRB_data2021['player'].isin(rbs)]
NFLRB_data2021 = NFLRB_data2021[NFLRB_data2021['attempts'] > 140]
NFLRB_data2021 = NFLRB_data2021.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
NFLRB_data2021 = NFLRB_data2021.sort_values(by='player')
NFLRB_data2021.reset_index(drop=True, inplace=True)
NFLRB_data2021

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,AJ Dillon,HB,GB,11.6,57.6,9,4.3,0.165775,3.181818
1,Aaron Jones,HB,GB,26.7,68.4,16,4.7,0.228070,3.175439
2,Alvin Kamara,HB,NO,14.5,50.7,21,3.7,0.187500,2.795833
3,Austin Ekeler,HB,LAC,18.1,57.9,22,4.4,0.179612,3.014563
4,Chuba Hubbard,HB,CAR,22.1,37.4,13,3.6,0.127907,2.627907
5,D'Andre Swift,HB,DET,30.8,41.8,21,4.1,0.112583,2.403974
6,David Montgomery,HB,CHI,19.1,44.5,21,3.8,0.168889,2.528889
7,Derrick Henry,HB,TEN,28.2,49.1,20,4.3,0.146119,3.324201
8,Devin Singletary,HB,BUF,33.4,75.3,20,4.6,0.239362,3.122340
9,Ezekiel Elliott,HB,DAL,14.7,25.0,24,4.2,0.080169,2.725738


In [32]:
NFL_rushing2019 = pd.read_csv("NFLrushing_summary2019.csv")
NFL_rushing2019 = NFL_rushing2019.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
NFL_rushing2019['ATR'] = NFL_rushing2019['avoided_tackles'] / NFL_rushing2019['attempts']
NFL_rushing2019['YCO'] = NFL_rushing2019['yards_after_contact'] / NFL_rushing2019['attempts']
NFLRB_data2019 = NFL_rushing2019.copy()
NFLRB_data2019 = NFLRB_data2019[NFLRB_data2019['player'].isin(rbs)]
NFLRB_data2019 = NFLRB_data2019[NFLRB_data2019['attempts'] > 140]
NFLRB_data2019 = NFLRB_data2019.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
NFLRB_data2019 = NFLRB_data2019.sort_values(by='player')
NFLRB_data2019.reset_index(drop=True, inplace=True)
NFLRB_data2019

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,Aaron Jones,HB,GB,27.4,58.2,25,4.6,0.187234,3.191489
1,Alvin Kamara,HB,NO,28.5,70.6,19,4.7,0.245614,3.175439
2,Christian McCaffrey,HB,CAR,31.6,49.4,31,4.8,0.170732,2.620209
3,David Montgomery,HB,CHI,15.0,48.0,16,3.7,0.194215,2.330579
4,Derrick Henry,HB,TEN,32.8,82.1,42,5.1,0.191419,4.184818
5,Devin Singletary,HB,BUF,36.4,69.6,25,5.1,0.236842,3.000000
6,Ezekiel Elliott,HB,DAL,18.5,52.8,38,4.5,0.159468,3.229236
7,Joe Mixon,HB,CIN,26.9,61.5,29,4.1,0.187050,3.154676
8,Josh Jacobs,HB,OAK,32.3,103.6,25,4.8,0.285124,3.479339
9,Saquon Barkley,HB,NYG,33.0,63.6,27,4.6,0.193548,3.225806


In [33]:
rbs2019 = NFLRB_data2019['player'].tolist()
rbs2021 = NFLRB_data2021['player'].tolist()
rbs2022 = NFLRB_data2022['player'].tolist()
rb_valid_data = NFLRB_data2023.copy()
for r in rbs:
    if w in rbs2022:
        if w in rbs2021:
            if w in rbs2019:
                rb_valid_data.loc[rb_valid_data['player'] == r, 'breakaway_percent'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'breakaway_percent'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'breakaway_percent'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'breakaway_percent'].iloc[0] + NFLRB_data2019.loc[NFLRB_data2019['player'] == r, 'breakaway_percent'].iloc[0]) / 4
                rb_valid_data.loc[rb_valid_data['player'] == r, 'elusive_rating'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'elusive_rating'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'elusive_rating'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'elusive_rating'].iloc[0] + NFLRB_data2019.loc[NFLRB_data2019['player'] == r, 'elusive_rating'].iloc[0]) / 4
                rb_valid_data.loc[rb_valid_data['player'] == r, 'explosive'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'explosive'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'explosive'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'explosive'].iloc[0] + NFLRB_data2019.loc[NFLRB_data2019['player'] == r, 'explosive'].iloc[0]) / 4
                rb_valid_data.loc[rb_valid_data['player'] == r, 'ypa'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'ypa'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'ypa'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'ypa'].iloc[0] + NFLRB_data2019.loc[NFLRB_data2019['player'] == r, 'ypa'].iloc[0]) / 4
                rb_valid_data.loc[rb_valid_data['player'] == r, 'ATR'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'ATR'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'ATR'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'ATR'].iloc[0] + NFLRB_data2019.loc[NFLRB_data2019['player'] == r, 'ATR'].iloc[0]) / 4
                rb_valid_data.loc[rb_valid_data['player'] == r, 'YCO'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'YCO'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'YCO'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'YCO'].iloc[0] + NFLRB_data2019.loc[NFLRB_data2019['player'] == r, 'YCO'].iloc[0]) / 4
            else:
                rb_valid_data.loc[rb_valid_data['player'] == r, 'breakaway_percent'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'breakaway_percent'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'breakaway_percent'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'breakaway_percent'].iloc[0]) / 3
                rb_valid_data.loc[rb_valid_data['player'] == r, 'elusive_rating'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'elusive_rating'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'elusive_rating'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'elusive_rating'].iloc[0]) / 3
                rb_valid_data.loc[rb_valid_data['player'] == r, 'explosive'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'explosive'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'explosive'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'explosive'].iloc[0]) / 3
                rb_valid_data.loc[rb_valid_data['player'] == r, 'ypa'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'ypa'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'ypa'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'ypa'].iloc[0]) / 3
                rb_valid_data.loc[rb_valid_data['player'] == r, 'ATR'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'ATR'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'ATR'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'ATR'].iloc[0]) / 3
                rb_valid_data.loc[rb_valid_data['player'] == r, 'YCO'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'YCO'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'YCO'].iloc[0] + NFLRB_data2021.loc[NFLRB_data2021['player'] == r, 'YCO'].iloc[0]) / 3
        else:
            rb_valid_data.loc[rb_valid_data['player'] == r, 'breakaway_percent'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'breakaway_percent'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'breakaway_percent'].iloc[0]) / 2
            rb_valid_data.loc[rb_valid_data['player'] == r, 'elusive_rating'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'elusive_rating'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'elusive_rating'].iloc[0]) / 2
            rb_valid_data.loc[rb_valid_data['player'] == r, 'explosive'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'explosive'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'explosive'].iloc[0]) / 2
            rb_valid_data.loc[rb_valid_data['player'] == r, 'ypa'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'ypa'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'ypa'].iloc[0]) / 2
            rb_valid_data.loc[rb_valid_data['player'] == r, 'ATR'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'ATR'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'ATR'].iloc[0]) / 2
            rb_valid_data.loc[rb_valid_data['player'] == r, 'YCO'] = (rb_valid_data.loc[rb_valid_data['player'] == r, 'YCO'].iloc[0] + NFLRB_data2022.loc[NFLRB_data2022['player'] == r, 'YCO'].iloc[0]) / 2
rb_valid_data

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,AJ Dillon,HB,GB,11.7,43.1,8,3.4,0.117978,2.691011
1,Aaron Jones,HB,GB,18.6,58.5,15,4.5,0.181818,3.160839
2,Alexander Mattison,HB,MIN,16.9,52.5,17,3.9,0.205556,2.755556
3,Alvin Kamara,HB,NO,13.7,34.5,15,3.9,0.122222,2.511111
4,Austin Ekeler,HB,LAC,20.4,49.4,7,3.5,0.173184,2.642458
5,Bijan Robinson,HB,ATL,24.5,78.6,30,4.5,0.242991,3.098131
6,Breece Hall,HB,NYJ,40.5,88.6,20,4.5,0.207207,3.427928
7,Brian Robinson,HB,WAS,23.4,64.3,19,4.1,0.207865,2.926966
8,Christian McCaffrey,HB,SF,36.4,65.5,44,5.4,0.220588,3.415441
9,Chuba Hubbard,HB,CAR,18.8,50.7,21,3.8,0.172269,2.865546


In [34]:
college_rushing2023 = pd.read_csv("college_rushing_summary2023.csv")
college_rushing2023 = college_rushing2023.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
college_rushing2023['ATR'] = college_rushing2023['avoided_tackles'] / college_rushing2023['attempts']
college_rushing2023['YCO'] = college_rushing2023['yards_after_contact'] / college_rushing2023['attempts']
rb_predict_data = college_rushing2023.copy()
rb_predict_data = rb_predict_data[rb_predict_data['position'] == 'HB']
rb_predict_data = rb_predict_data[rb_predict_data['player'].isin(prospects)]
rb_predict_data = rb_predict_data[rb_predict_data['attempts'] > 140]
rb_predict_data = rb_predict_data.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
rb_predict_data = rb_predict_data.sort_values(by='player')
rb_predict_data.reset_index(drop=True, inplace=True)
rb_predict_data

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,Audric Estime,HB,NOTRE DAME,44.1,126.5,38,6.4,0.306220,4.267943
1,Blake Corum,HB,MICHIGAN,30.6,25.3,20,4.8,0.099010,2.564356
2,Blake Watson,HB,MEMPHIS,44.0,102.9,30,5.9,0.254237,3.485876
3,Braelon Allen,HB,WISCONSIN,40.4,99.2,24,5.4,0.270718,3.767956
4,Bucky Irving,HB,OREGON,38.8,152.1,31,6.5,0.374233,3.993865
5,Carson Steele,HB,UCLA,22.4,100.4,26,5.1,0.239521,3.850299
6,Cody Schrader,HB,MISSOURI,39.0,57.9,36,6.0,0.181452,3.177419
7,Daijun Edwards,HB,GEORGIA,39.0,69.9,23,5.3,0.224490,2.863946
8,Dillon Johnson,HB,WASHINGTON,38.5,65.3,22,5.5,0.213873,3.040462
9,Emani Bailey,HB,TCU,43.9,117.1,38,5.4,0.313901,3.677130


In [35]:
college_rushing2022 = pd.read_csv("college_rushing_summary2022.csv")
college_rushing2022 = college_rushing2022.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
college_rushing2022['ATR'] = college_rushing2022['avoided_tackles'] / college_rushing2022['attempts']
college_rushing2022['YCO'] = college_rushing2022['yards_after_contact'] / college_rushing2022['attempts']
collegeRB_data2022 = college_rushing2022.copy()
collegeRB_data2022 = collegeRB_data2022[collegeRB_data2022['player'].isin(rbs)]
collegeRB_data2022 = collegeRB_data2022[collegeRB_data2022['attempts'] > 120]
collegeRB_data2022 = collegeRB_data2022.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
collegeRB_data2022 = collegeRB_data2022.sort_values(by='player')
collegeRB_data2022.reset_index(drop=True, inplace=True)
collegeRB_data2022

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,Bijan Robinson,HB,TEXAS,38.6,169.1,41,6.1,0.404669,4.167315
1,Jahmyr Gibbs,HB,ALABAMA,54.8,98.7,23,6.3,0.257353,3.316176


In [36]:
college_rushing2021 = pd.read_csv("collegeRushing_summary2021.csv")
college_rushing2021 = college_rushing2021.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
college_rushing2021['ATR'] = college_rushing2021['avoided_tackles'] / college_rushing2021['attempts']
college_rushing2021['YCO'] = college_rushing2021['yards_after_contact'] / college_rushing2021['attempts']
collegeRB_data2021 = college_rushing2021.copy()
collegeRB_data2021 = collegeRB_data2021[collegeRB_data2021['player'].isin(rbs)]
collegeRB_data2021 = collegeRB_data2021[collegeRB_data2021['attempts'] > 120]
collegeRB_data2021 = collegeRB_data2021.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
collegeRB_data2021 = collegeRB_data2021.sort_values(by='player')
collegeRB_data2021.reset_index(drop=True, inplace=True)
collegeRB_data2021

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,Bijan Robinson,HB,TEXAS,36.7,155.2,27,5.8,0.405128,3.943590
1,Breece Hall,HB,IOWA STATE,53.3,88.8,36,5.8,0.302789,2.840637
2,Brian Robinson,HB,ALABAMA,33.4,93.0,30,4.9,0.294686,3.275362
3,Isiah Pacheco,HB,RUTGERS,24.6,24.0,15,3.9,0.072289,2.867470
4,Jahmyr Gibbs,HB,GA TECH,44.1,80.8,19,5.2,0.237762,3.286713
5,Jaylen Warren,HB,OKLA STATE,33.6,103.8,30,4.8,0.324895,3.139241
6,Jerome Ford,HB,CINCINNATI,38.4,79.9,27,5.8,0.265193,3.044199
7,Kenneth Walker III,HB,MICH STATE,53.9,145.9,46,6.2,0.335878,4.458015
8,Kyren Williams,HB,NOTRE DAME,33.0,108.6,25,4.9,0.284314,3.710784
9,Rachaad White,HB,ARIZONA ST,31.2,89.8,24,5.5,0.240437,3.382514


In [37]:
college_rushing2019 = pd.read_csv("collegeRushing_summary2019.csv")
college_rushing2019 = college_rushing2019.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
college_rushing2019['ATR'] = college_rushing2019['avoided_tackles'] / college_rushing2019['attempts']
college_rushing2019['YCO'] = college_rushing2019['yards_after_contact'] / college_rushing2019['attempts']
collegeRB_data2019 = college_rushing2019.copy()
collegeRB_data2019 = collegeRB_data2019[collegeRB_data2019['player'].isin(rbs)]
collegeRB_data2019 = collegeRB_data2019[collegeRB_data2019['attempts'] > 120]
collegeRB_data2019 = collegeRB_data2019.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
collegeRB_data2019 = collegeRB_data2019.sort_values(by='player')
collegeRB_data2019.reset_index(drop=True, inplace=True)
collegeRB_data2019

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,AJ Dillon,HB,BOSTON COL,31.7,88.6,41,5.4,0.255521,3.482650
1,Breece Hall,HB,IOWA STATE,34.4,107.0,27,5.0,0.309524,3.654762
2,Chuba Hubbard,HB,OKLA STATE,48.3,89.9,44,6.3,0.226537,3.854369
3,D'Andre Swift,HB,GEORGIA,38.0,72.3,38,6.2,0.206186,3.530928
4,Isiah Pacheco,HB,RUTGERS,29.4,47.3,17,4.3,0.142012,3.307692
5,Javonte Williams,HB,N CAROLINA,34.5,140.0,33,5.6,0.344371,4.198675
6,Jonathan Taylor,HB,WISCONSIN,38.6,102.3,55,6.3,0.272401,3.849462
7,Najee Harris,HB,ALABAMA,26.4,104.3,41,5.9,0.275676,3.567568
8,Travis Etienne Jr.,HB,CLEMSON,53.9,229.3,40,8.3,0.452381,5.351190
9,Zack Moss,HB,UTAH,42.4,194.3,33,6.2,0.402010,4.668342


In [38]:
college_rushing2018 = pd.read_csv("collegeRushing_summary2018.csv")
college_rushing2018 = college_rushing2018.drop(columns = ['player_id', 'player_game_count', 'breakaway_attempts', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_hands_fumble', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'total_touches', 'touchdowns', 'yards', 'yco_attempt', 'yprr', 'zone_attempts', 'run_plays', 'targets'])
college_rushing2018['ATR'] = college_rushing2018['avoided_tackles'] / college_rushing2018['attempts']
college_rushing2018['YCO'] = college_rushing2018['yards_after_contact'] / college_rushing2018['attempts']
collegeRB_data2018 = college_rushing2018.copy()
collegeRB_data2018 = collegeRB_data2018[collegeRB_data2018['player'].isin(rbs)]
collegeRB_data2018 = collegeRB_data2018[collegeRB_data2018['attempts'] > 120]
collegeRB_data2018 = collegeRB_data2018.drop(columns = ['scrambles', 'scramble_yards', 'avoided_tackles', 'yards_after_contact', 'attempts'])
collegeRB_data2018 = collegeRB_data2018.sort_values(by='player')
collegeRB_data2018.reset_index(drop=True, inplace=True)
collegeRB_data2018

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,AJ Dillon,HB,BOSTON COL,40.1,71.9,24,4.9,0.234513,3.172566
1,Alexander Mattison,HB,BOISE ST,17.9,59.7,30,4.6,0.213740,2.736641
2,D'Andre Swift,HB,GEORGIA,48.6,95.8,29,6.9,0.223022,4.143885
3,David Montgomery,HB,IOWA STATE,33.1,135.7,36,4.7,0.408696,3.504348
4,Devin Singletary,HB,FAU,47.2,138.3,42,5.1,0.363636,3.890152
5,Jonathan Taylor,HB,WISCONSIN,42.9,89.7,52,7.1,0.211470,4.222222
6,Travis Etienne Jr.,HB,CLEMSON,57.0,109.6,37,8.0,0.256098,4.237805
7,Zack Moss,HB,UTAH,39.2,113.6,27,6.1,0.296089,3.541899


In [39]:
rbList = collegeRB_data2018['player'].tolist()
rb_train_data = collegeRB_data2018.copy()
temp1 = collegeRB_data2019.copy()
temp2 = collegeRB_data2021.copy()
temp3 = collegeRB_data2022.copy()
temp1 = temp1[~temp1['player'].isin(rbList)]
rb_train_data = pd.concat([rb_train_data, temp1]).drop_duplicates(keep=False)
tempRB = temp1['player'].tolist()
collegeRB2019 = collegeRB_data2019.copy()
collegeRB2019 = collegeRB2019[~collegeRB2019['player'].isin(tempRB)]
rbList = rb_train_data['player'].tolist()
temp2 = temp2[temp2['player'].isin(rbList)]
tempRB = temp2['player'].tolist()
collegeRB2021 = collegeRB_data2021.copy()
collegeRB2021 = collegeRB2021[~collegeRB2021['player'].isin(tempRB)]
rb_train_data = pd.concat([rb_train_data, temp2]).drop_duplicates(keep=False)
rbList = rb_train_data['player'].tolist()
temp3 = temp3[~temp3['player'].isin(rbList)]
rb_train_data = pd.concat([rb_train_data, temp3]).drop_duplicates(keep=False)
tempRB = temp3['player'].tolist()
collegeRB2022 = collegeRB_data2022.copy()
collegeRB2022 = collegeRB2022[~collegeRB2022['player'].isin(tempRB)]
rb_train_data.reset_index(drop=True, inplace=True)
rbs = rb_train_data['player'].tolist()
rb22 = collegeRB2022['player'].tolist()
rb21 = collegeRB2021['player'].tolist()
rb19 = collegeRB2019['player'].tolist()
for r in rbs:
    if r in rb22:
        if r in rb21:
            if r in rb19:
                rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'] = (rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'breakaway_percent'].iloc[0]) / 4
                rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'] = (rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'elusive_rating'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'elusive_rating'].iloc[0]  + collegeRB2019.loc[collegeRB2019['player'] == r, 'elusive_rating'].iloc[0]) / 4
                rb_train_data.loc[rb_train_data['player'] == r, 'explosive'] = (rb_train_data.loc[rb_train_data['player'] == r, 'explosive'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'explosive'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'explosive'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'explosive'].iloc[0]) / 4
                rb_train_data.loc[rb_train_data['player'] == r, 'ypa'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ypa'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'ypa'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'ypa'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'ypa'].iloc[0]) / 4
                rb_train_data.loc[rb_train_data['player'] == r, 'ATR'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ATR'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'ATR'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'ATR'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'ATR'].iloc[0]) / 4
                rb_train_data.loc[rb_train_data['player'] == r, 'YCO'] = (rb_train_data.loc[rb_train_data['player'] == r, 'YCO'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'YCO'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'YCO'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'YCO'].iloc[0]) / 4
            else:
                rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'] = (rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'breakaway_percent'].iloc[0]) / 3
                rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'] = (rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'elusive_rating'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'elusive_rating'].iloc[0]) / 3
                rb_train_data.loc[rb_train_data['player'] == r, 'explosive'] = (rb_train_data.loc[rb_train_data['player'] == r, 'explosive'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'explosive'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'explosive'].iloc[0]) / 3
                rb_train_data.loc[rb_train_data['player'] == r, 'ypa'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ypa'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'ypa'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'ypa'].iloc[0]) / 3
                rb_train_data.loc[rb_train_data['player'] == r, 'ATR'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ATR'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'ATR'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'ATR'].iloc[0]) / 3
                rb_train_data.loc[rb_train_data['player'] == r, 'YCO'] = (rb_train_data.loc[rb_train_data['player'] == r, 'YCO'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'YCO'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'YCO'].iloc[0]) / 3
        else:
            rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'] = (rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'breakaway_percent'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'] = (rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'elusive_rating'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'explosive'] = (rb_train_data.loc[rb_train_data['player'] == r, 'explosive'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'explosive'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'ypa'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ypa'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'ypa'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'ATR'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ATR'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'ATR'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'YCO'] = (rb_train_data.loc[rb_train_data['player'] == r, 'YCO'].iloc[0] + collegeRB2022.loc[collegeRB2022['player'] == r, 'YCO'].iloc[0]) / 2
    elif r in rb21:
        if r in rb19:
            rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'] = (rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'breakaway_percent'].iloc[0]) / 3
            rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'] = (rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'elusive_rating'].iloc[0]  + collegeRB2019.loc[collegeRB2019['player'] == r, 'elusive_rating'].iloc[0]) / 3
            rb_train_data.loc[rb_train_data['player'] == r, 'explosive'] = (rb_train_data.loc[rb_train_data['player'] == r, 'explosive'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'explosive'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'explosive'].iloc[0]) / 3
            rb_train_data.loc[rb_train_data['player'] == r, 'ypa'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ypa'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'ypa'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'ypa'].iloc[0]) / 3
            rb_train_data.loc[rb_train_data['player'] == r, 'ATR'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ATR'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'ATR'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'ATR'].iloc[0]) / 3
            rb_train_data.loc[rb_train_data['player'] == r, 'YCO'] = (rb_train_data.loc[rb_train_data['player'] == r, 'YCO'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'YCO'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'YCO'].iloc[0]) / 3
        else:
            rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'] = (rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'breakaway_percent'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'] = (rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'elusive_rating'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'explosive'] = (rb_train_data.loc[rb_train_data['player'] == r, 'explosive'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'explosive'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'ypa'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ypa'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'ypa'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'ATR'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ATR'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'ATR'].iloc[0]) / 2
            rb_train_data.loc[rb_train_data['player'] == r, 'YCO'] = (rb_train_data.loc[rb_train_data['player'] == r, 'YCO'].iloc[0] + collegeRB2021.loc[collegeRB2021['player'] == r, 'YCO'].iloc[0]) / 2
    elif r in rb19:
        rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'] = (rb_train_data.loc[rb_train_data['player'] == r, 'breakaway_percent'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'breakaway_percent'].iloc[0]) / 2
        rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'] = (rb_train_data.loc[rb_train_data['player'] == r, 'elusive_rating'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'elusive_rating'].iloc[0]) / 2
        rb_train_data.loc[rb_train_data['player'] == r, 'explosive'] = (rb_train_data.loc[rb_train_data['player'] == r, 'explosive'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'explosive'].iloc[0]) / 2
        rb_train_data.loc[rb_train_data['player'] == r, 'ypa'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ypa'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'ypa'].iloc[0]) / 2
        rb_train_data.loc[rb_train_data['player'] == r, 'ATR'] = (rb_train_data.loc[rb_train_data['player'] == r, 'ATR'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'ATR'].iloc[0]) / 2
        rb_train_data.loc[rb_train_data['player'] == r, 'YCO'] = (rb_train_data.loc[rb_train_data['player'] == r, 'YCO'].iloc[0] + collegeRB2019.loc[collegeRB2019['player'] == r, 'YCO'].iloc[0]) / 2

rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'breakaway_percent'] = (rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'breakaway_percent'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'breakaway_percent'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'elusive_rating'] = (rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'elusive_rating'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'elusive_rating'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'explosive'] = (rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'explosive'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'explosive'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'ypa'] = (rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'ypa'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'ypa'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'ATR'] = (rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'ATR'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'ATR'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'YCO'] = (rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'YCO'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Breece Hall', 'YCO'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'breakaway_percent'] = (rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'breakaway_percent'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'breakaway_percent'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'elusive_rating'] = (rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'elusive_rating'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'elusive_rating'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'explosive'] = (rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'explosive'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'explosive'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'ypa'] = (rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'ypa'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'ypa'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'ATR'] = (rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'ATR'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'ATR'].iloc[1]) / 2        
rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'YCO'] = (rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'YCO'].iloc[0] + rb_train_data.loc[rb_train_data['player'] == 'Isiah Pacheco', 'YCO'].iloc[1]) / 2   
rb_train_data = rb_train_data.drop_duplicates(subset=['player'])
rb_train_data = rb_train_data.sort_values(by='player')
rb_train_data.reset_index(drop=True, inplace=True)
rb_train_data

,player,position,team_name,breakaway_percent,elusive_rating,explosive,ypa,ATR,YCO
0,AJ Dillon,HB,BOSTON COL,35.90,80.25,32.5,5.15,0.245017,3.327608
1,Alexander Mattison,HB,BOISE ST,17.90,59.70,30.0,4.60,0.213740,2.736641
2,Bijan Robinson,HB,TEXAS,37.65,162.15,34.0,5.95,0.404899,4.055452
3,Breece Hall,HB,IOWA STATE,43.85,97.90,31.5,5.40,0.306156,3.247700
4,Chuba Hubbard,HB,OKLA STATE,48.30,89.90,44.0,6.30,0.226537,3.854369
5,D'Andre Swift,HB,GEORGIA,43.30,84.05,33.5,6.55,0.214604,3.837406
6,David Montgomery,HB,IOWA STATE,33.10,135.70,36.0,4.70,0.408696,3.504348
7,Devin Singletary,HB,FAU,47.20,138.30,42.0,5.10,0.363636,3.890152
8,Isiah Pacheco,HB,RUTGERS,27.00,35.65,16.0,4.10,0.107150,3.087581
9,Jahmyr Gibbs,HB,ALABAMA,49.45,89.75,21.0,5.75,0.247558,3.301445


In [40]:
college_coverage2023 = pd.read_csv("college_coverage_summary2023.csv")
college_coverage2023 = college_coverage2023.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
cb_predict_data = college_coverage2023.copy()
cb_predict_data = cb_predict_data[cb_predict_data['player'].isin(prospects)]
cb_predict_data = cb_predict_data[cb_predict_data['snap_counts_pass_play'] >= 259]
cb_predict_data = cb_predict_data[cb_predict_data['position'] == 'CB']
cb_predict_data = cb_predict_data.sort_values(by='player')
cb_predict_data.reset_index(drop=True, inplace=True)
cb_predict_data['YAC%'] = cb_predict_data['yards_after_catch'] / cb_predict_data['receptions']
cb_predict_data['Target Rate'] = cb_predict_data['targets'] / cb_predict_data['snap_counts_pass_play']
cols = {'player' : 'Player', 'position' : 'Position', 'missed_tackle_rate' : 'MIS%', 'yards_per_reception' : 'Yards/Rec', 'team_name' : 'Team', 'forced_incompletion_rate' : 'FI%'}
cb_predict_data = cb_predict_data.rename(columns = cols)
cb_predict_data = cb_predict_data.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
cb_predict_data

,Player,Position,Team,FI%,MIS%,Yards/Rec,YAC%,Target Rate
0,Andru Phillips,CB,KENTUCKY,7.4,19.0,11.8,4.189189,0.139535
1,Caelen Carson,CB,WAKE,10.9,18.2,12.1,4.055556,0.180282
2,Cooper DeJean,CB,IOWA,13.0,12.5,9.7,2.200000,0.118557
3,D.J. James,CB,AUBURN,28.8,7.9,12.0,3.406250,0.142857
4,Ennis Rakestraw Jr.,CB,MISSOURI,10.7,8.3,10.9,5.555556,0.098592
5,Jarvis Brownlee Jr.,CB,LOUISVILLE,12.9,6.7,14.9,2.000000,0.118321
6,Josh Newton,CB,TCU,19.2,12.5,12.1,4.800000,0.123223
7,Kalen King,CB,PENN STATE,7.3,14.8,11.6,4.280000,0.150183
8,Kamari Lassiter,CB,GEORGIA,27.8,4.0,8.5,2.500000,0.097297
9,Khyree Jackson,CB,OREGON,25.0,5.6,10.4,5.388889,0.119205


In [41]:
NFL_coverage2023 = pd.read_csv("defense_coverage_summary_2023.csv")
NFL_coverage2023 = NFL_coverage2023.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
j = NFL_coverage2023[(NFL_coverage2023['player'] == 'Brian Branch')].index[0]
NFL_coverage2023.at[j, 'position'] = 'S'
cb_NFL_data2023 = NFL_coverage2023.copy()
cb_NFL_data2023 = cb_NFL_data2023[cb_NFL_data2023['snap_counts_pass_play'] >= 400]
cb_NFL_data2023 = cb_NFL_data2023[cb_NFL_data2023['position'] == 'CB']
cb_NFL_data2023 = cb_NFL_data2023.sort_values(by='player')
cb_NFL_data2023.reset_index(drop=True, inplace=True)
cb_NFL_data2023['Target Rate'] = cb_NFL_data2023['targets'] / cb_NFL_data2023['snap_counts_pass_play']
cb_NFL_data2023['YAC%'] = cb_NFL_data2023['yards_after_catch'] / cb_NFL_data2023['receptions']
cb_NFL_data2023 = cb_NFL_data2023.rename(columns = cols)
cb_NFL_data2023 = cb_NFL_data2023.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
cbs = cb_NFL_data2023['Player'].tolist()
cb_NFL_data2023

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,A.J. Terrell,CB,ATL,18.4,17.4,11.5,0.124386,3.750000
1,Adoree' Jackson,CB,NYG,9.8,16.4,13.9,0.179825,7.074074
2,Ahkello Witherspoon,CB,LA,12.5,19.1,15.1,0.150072,3.865385
3,Akayleb Evans,CB,MIN,8.7,14.3,12.4,0.130435,5.020833
4,Alontae Taylor,CB,NO,11.5,13.6,11.1,0.188648,5.493333
...,...,...,...,...,...,...,...,...
70,Taron Johnson,CB,BUF,7.4,7.9,8.4,0.117733,4.774194
71,Trent McDuffie,CB,KC,8.9,9.6,11.2,0.114213,4.684211
72,Tyrique Stevenson,CB,CHI,12.7,9.4,11.6,0.198915,4.763889
73,Xavien Howard,CB,MIA,15.5,11.4,11.4,0.148847,4.431818


In [42]:
NFL_coverage2022 = pd.read_csv("defense_coverage_summary_2022.csv")
NFL_coverage2022 = NFL_coverage2022.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
cb_NFL_data2022 = NFL_coverage2022.copy()
cb_NFL_data2022 = cb_NFL_data2022[cb_NFL_data2022['snap_counts_pass_play'] >= 400]
cb_NFL_data2022 = cb_NFL_data2022[cb_NFL_data2022['player'].isin(cbs)]
cb_NFL_data2022 = cb_NFL_data2022.sort_values(by='player')
cb_NFL_data2022.reset_index(drop=True, inplace=True)
cb_NFL_data2022['Target Rate'] = cb_NFL_data2022['targets'] / cb_NFL_data2022['snap_counts_pass_play']
cb_NFL_data2022['YAC%'] = cb_NFL_data2022['yards_after_catch'] / cb_NFL_data2022['receptions']
cb_NFL_data2022 = cb_NFL_data2022.rename(columns = cols)
cb_NFL_data2022 = cb_NFL_data2022.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
cb_NFL_data2022

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,A.J. Terrell,CB,ATL,20.6,20.0,11.0,0.152809,3.641026
1,Adoree' Jackson,CB,NYG,14.1,8.5,10.5,0.154217,3.324324
2,Amik Robertson,CB,LV,10.7,21.1,15.5,0.122807,5.969697
3,Asante Samuel Jr.,CB,LAC,13.0,7.8,12.8,0.156495,3.910714
4,Benjamin St-Juste,CB,WAS,13.8,9.4,15.9,0.154394,5.142857
5,Cam Taylor-Britt,CB,CIN,12.5,7.4,13.8,0.129032,4.714286
6,Cameron Sutton,CB,PIT,18.3,14.7,11.4,0.123909,3.666667
7,Carlton Davis III,CB,TB,15.8,6.3,12.5,0.173993,5.115385
8,Charvarius Ward,CB,SF,16.0,1.4,12.0,0.128240,2.581818
9,D.J. Reed,CB,NYJ,13.3,8.8,9.9,0.125378,2.553191


In [43]:
NFL_coverage2021 = pd.read_csv("defense_coverage_summary_2021.csv")
NFL_coverage2021 = NFL_coverage2021.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
cb_NFL_data2021 = NFL_coverage2021.copy()
cb_NFL_data2021 = cb_NFL_data2021[cb_NFL_data2021['snap_counts_pass_play'] >= 400]
cb_NFL_data2021 = cb_NFL_data2021[cb_NFL_data2021['player'].isin(cbs)]
cb_NFL_data2021 = cb_NFL_data2021.sort_values(by='player')
cb_NFL_data2021.reset_index(drop=True, inplace=True)
cb_NFL_data2021['Target Rate'] = cb_NFL_data2021['targets'] / cb_NFL_data2021['snap_counts_pass_play']
cb_NFL_data2021['YAC%'] = cb_NFL_data2021['yards_after_catch'] / cb_NFL_data2021['receptions']
cb_NFL_data2021 = cb_NFL_data2021.rename(columns = cols)
cb_NFL_data2021 = cb_NFL_data2021.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
cb_NFL_data2021

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,A.J. Terrell,CB,ATL,22.7,10.7,6.9,0.114783,3.206897
1,Adoree' Jackson,CB,NYG,16.4,6.3,8.9,0.122490,2.823529
2,Brandon Stephens,S,BLT,9.5,6.0,15.3,0.086066,7.090909
3,Byron Murphy Jr.,CB,ARZ,7.1,8.3,11.2,0.135048,4.754386
4,Cameron Sutton,CB,PIT,6.8,17.3,11.7,0.107402,4.215686
5,Carlton Davis III,CB,TB,20.3,20.0,10.7,0.128015,4.289474
6,Charvarius Ward,CB,KC,18.4,5.3,11.3,0.165821,4.740000
7,Chidobe Awuzie,CB,CIN,12.5,9.5,11.9,0.170697,4.029412
8,D.J. Reed,CB,SEA,14.7,13.8,10.9,0.112211,3.257143
9,Darious Williams,CB,LA,10.4,8.4,12.0,0.141522,4.014925


In [44]:
NFL_coverage2019 = pd.read_csv("defense_coverage_summary_2019.csv")
NFL_coverage2019 = NFL_coverage2019.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
cb_NFL_data2019 = NFL_coverage2019.copy()
cb_NFL_data2019 = cb_NFL_data2019[cb_NFL_data2019['snap_counts_pass_play'] >= 400]
cb_NFL_data2019 = cb_NFL_data2019[cb_NFL_data2019['player'].isin(cbs)]
cb_NFL_data2019 = cb_NFL_data2019.sort_values(by='player')
cb_NFL_data2019.reset_index(drop=True, inplace=True)
cb_NFL_data2019['Target Rate'] = cb_NFL_data2019['targets'] / cb_NFL_data2019['snap_counts_pass_play']
cb_NFL_data2019['YAC%'] = cb_NFL_data2019['yards_after_catch'] / cb_NFL_data2019['receptions']
cb_NFL_data2019 = cb_NFL_data2019.rename(columns = cols)
cb_NFL_data2019 = cb_NFL_data2019.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
cb_NFL_data2019

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,Adoree' Jackson,CB,TEN,20.6,6.8,12.3,0.113924,4.435897
1,Byron Murphy Jr.,CB,ARZ,9.6,11.3,9.4,0.147727,2.042254
2,Carlton Davis III,CB,TB,21.0,12.1,12.0,0.159817,2.418182
3,Charvarius Ward,CB,KC,14.0,8.8,16.6,0.122850,5.187500
4,Chidobe Awuzie,CB,DAL,18.9,8.3,13.1,0.137195,4.214286
5,Darius Slay,CB,DET,13.1,7.5,14.1,0.160612,2.653061
6,Denzel Ward,CB,CLV,18.8,17.6,13.7,0.154018,5.096774
7,Donte Jackson,CB,CAR,10.0,19.4,16.9,0.140187,5.026316
8,Fabian Moreau,CB,WAS,4.0,10.8,12.6,0.121655,6.325000
9,Jalen Ramsey,CB,LA,9.4,8.9,12.3,0.132231,3.162791


In [45]:
college_coverage2022 = pd.read_csv("college_coverage_summary2022.csv")
college_coverage2022 = college_coverage2022.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
cb_college_data2022 = college_coverage2022.copy()
cb_college_data2022 = cb_college_data2022[cb_college_data2022['snap_counts_pass_play'] >= 250]
cb_college_data2022 = cb_college_data2022[cb_college_data2022['player'].isin(cbs)]
cb_college_data2022 = cb_college_data2022[cb_college_data2022['position'] == 'CB']
cb_college_data2022 = cb_college_data2022.sort_values(by='player')
cb_college_data2022.reset_index(drop=True, inplace=True)
cb_college_data2022['Target Rate'] = cb_college_data2022['targets'] / cb_college_data2022['snap_counts_pass_play']
cb_college_data2022['YAC%'] = cb_college_data2022['yards_after_catch'] / cb_college_data2022['receptions']
cb_college_data2022 = cb_college_data2022.rename(columns = cols)
cb_college_data2022 = cb_college_data2022.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
i = cb_college_data2022[cb_college_data2022['Player'] == 'Jaylon Jones'].index[0]
cb_college_data2022 = cb_college_data2022.drop(i)
cb_college_data2022

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,Carrington Valentine,CB,KENTUCKY,13.8,12.8,13.9,0.170157,5.750000
1,DJ Turner II,CB,MICHIGAN,20.0,10.7,13.0,0.151134,3.814815
2,Deonte Banks,CB,MARYLAND,21.7,3.7,9.9,0.156658,2.384615
3,Devon Witherspoon,CB,ILLINOIS,29.0,10.0,9.4,0.143852,3.227273
5,Jaylon Jones,CB,TEXAS A&M,21.1,14.3,9.4,0.067857,4.000000
6,Joey Porter Jr.,CB,PENN STATE,40.0,17.4,9.5,0.105634,3.400000
7,Tyrique Stevenson,CB,MIAMI FL,20.0,13.6,20.8,0.135135,9.647059


In [46]:
college_coverage2021 = pd.read_csv("college_coverage_summary2021.csv")
college_coverage2021 = college_coverage2021.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
cb_college_data2021 = college_coverage2021.copy()
cb_college_data2021 = cb_college_data2021[cb_college_data2021['snap_counts_pass_play'] >= 250]
cb_college_data2021 = cb_college_data2021[cb_college_data2021['player'].isin(cbs)]
cb_college_data2021 = cb_college_data2021[cb_college_data2021['position'] == 'CB']
cb_college_data2021 = cb_college_data2021.sort_values(by='player')
cb_college_data2021.reset_index(drop=True, inplace=True)
cb_college_data2021['Target Rate'] = cb_college_data2021['targets'] / cb_college_data2021['snap_counts_pass_play']
cb_college_data2021['YAC%'] = cb_college_data2021['yards_after_catch'] / cb_college_data2021['receptions']
cb_college_data2021 = cb_college_data2021.rename(columns = cols)
cb_college_data2021 = cb_college_data2021.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
i = cb_college_data2021[cb_college_data2021['Player'] == 'Jaylon Jones'].index[0]
cb_college_data2021 = cb_college_data2021.drop(i)
cb_college_data2021

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,Akayleb Evans,CB,MISSOURI,19.1,0.0,10.8,0.167260,3.360000
1,Alontae Taylor,CB,TENNESSEE,11.5,19.6,13.7,0.110403,6.000000
2,Cam Taylor-Britt,CB,NEBRASKA,18.2,9.1,10.0,0.156398,2.857143
3,Carrington Valentine,CB,KENTUCKY,4.3,7.3,11.2,0.159722,3.553191
4,DJ Turner II,CB,MICHIGAN,15.2,7.7,11.1,0.151815,2.500000
5,DaRon Bland,CB,FRESNO ST,21.7,11.1,12.5,0.139394,4.473684
6,Derion Kendrick,CB,GEORGIA,18.4,4.0,15.3,0.105263,6.157895
7,Devon Witherspoon,CB,ILLINOIS,12.1,12.1,12.4,0.159341,4.470588
8,Ja'Quan McMillian,CB,E CAROLINA,15.3,9.8,12.2,0.183206,2.750000
10,Jaylon Jones,CB,GA STATE,13.6,12.1,14.4,0.156028,3.483871


In [47]:
college_coverage2019 = pd.read_csv("college_coverage_summary2019.csv")
college_coverage2019 = college_coverage2019.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
cb_college_data2019 = college_coverage2019.copy()
cb_college_data2019 = cb_college_data2019[cb_college_data2019['snap_counts_pass_play'] >= 250]
cb_college_data2019 = cb_college_data2019[cb_college_data2019['player'].isin(cbs)]
cb_college_data2019 = cb_college_data2019[cb_college_data2019['position'] == 'CB']
cb_college_data2019 = cb_college_data2019.sort_values(by='player')
cb_college_data2019.reset_index(drop=True, inplace=True)
cb_college_data2019['Target Rate'] = cb_college_data2019['targets'] / cb_college_data2019['snap_counts_pass_play']
cb_college_data2019['YAC%'] = cb_college_data2019['yards_after_catch'] / cb_college_data2019['receptions']
cb_college_data2019 = cb_college_data2019.rename(columns = cols)
cb_college_data2019 = cb_college_data2019.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
i = cb_college_data2019[cb_college_data2019['Player'] == 'Jaylon Jones'].index[0]
cb_college_data2019 = cb_college_data2019.drop(i)
cb_college_data2019

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,A.J. Terrell,CB,CLEMSON,7.7,7.1,12.4,0.093863,1.333333
1,Alontae Taylor,CB,TENNESSEE,2.8,17.4,12.9,0.112150,4.136364
2,Ambry Thomas,CB,MICHIGAN,12.8,7.1,13.3,0.150160,5.240000
3,Amik Robertson,CB,LA TECH,24.7,15.2,12.1,0.169978,6.658537
4,Asante Samuel Jr.,CB,FLORIDA ST,18.8,8.9,12.7,0.145455,3.088235
5,Brandon Stephens,CB,SMU,16.9,5.9,16.6,0.133745,4.645161
6,Deommodore Lenoir,CB,OREGON,21.9,3.2,14.3,0.148148,5.000000
7,Derek Stingley Jr.,CB,LSU,23.9,14.3,19.5,0.145969,6.555556
8,Ja'Quan McMillian,CB,E CAROLINA,18.5,13.3,12.5,0.133005,4.200000
9,Jaylon Johnson,CB,UTAH,18.6,3.7,9.9,0.143552,3.740741


In [48]:
college_coverage2018 = pd.read_csv("college_coverage_summary2018.csv")
college_coverage2018 = college_coverage2018.drop(columns = ['player_id', 'player_game_count', 'assists', 'avg_depth_of_target', 'catch_rate', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_percent', 'coverage_snaps_per_reception', 'declined_penalties', 'dropped_ints', 'forced_incompletes', 'franchise_id', 'grades_coverage_defense','grades_defense', 'grades_defense_penalty', 'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle', 'interceptions', 'longest', 'coverage_snaps_per_target', 'missed_tackles', 'pass_break_ups', 'penalties',
       'qb_rating_against', 'snap_counts_coverage', 'stops', 'tackles', 'touchdowns',
       'yards', 'yards_per_coverage_snap'])
cb_college_data2018 = college_coverage2018.copy()
cb_college_data2018 = cb_college_data2018[cb_college_data2018['snap_counts_pass_play'] >= 250]
cb_college_data2018 = cb_college_data2018[cb_college_data2018['player'].isin(cbs)]
cb_college_data2018 = cb_college_data2018[cb_college_data2018['position'] == 'CB']
cb_college_data2018 = cb_college_data2018.sort_values(by='player')
cb_college_data2018.reset_index(drop=True, inplace=True)
cb_college_data2018['Target Rate'] = cb_college_data2018['targets'] / cb_college_data2018['snap_counts_pass_play']
cb_college_data2018['YAC%'] = cb_college_data2018['yards_after_catch'] / cb_college_data2018['receptions']
cb_college_data2018 = cb_college_data2018.rename(columns = cols)
cb_college_data2018 = cb_college_data2018.drop(columns = ['snap_counts_pass_play', 'targets', 'yards_after_catch', 'receptions'])
cb_college_data2018

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,A.J. Terrell,CB,CLEMSON,14.6,0.0,9.5,0.123494,1.956522
1,Alontae Taylor,CB,TENNESSEE,5.9,13.0,8.8,0.116838,4.227273
2,Amik Robertson,CB,LA TECH,16.4,17.1,12.0,0.163415,5.820513
3,Byron Murphy Jr.,CB,WASHINGTON,26.9,8.3,9.9,0.115813,2.517241
4,Deommodore Lenoir,CB,OREGON,18.5,9.4,13.3,0.140389,3.485714
5,Jamel Dean,CB,AUBURN,19.6,20.0,16.1,0.156997,9.052632
6,Jaylon Johnson,CB,UTAH,12.3,12.5,11.0,0.144766,4.270270
7,Jerry Jacobs,CB,ARK STATE,15.6,20.0,10.5,0.130814,3.863636
8,Keisean Nixon,CB,S CAROLINA,18.2,9.3,13.1,0.171429,2.128205
9,L'Jarius Sneed,CB,LA TECH,15.3,6.1,15.6,0.149367,2.727273


In [49]:
NFLpassRush2023 = pd.read_csv("pass_rush_summary_2023.csv")
NFLpassRush2023 = NFLpassRush2023[NFLpassRush2023['snap_counts_pass_rush'] >= 329]
NFLpassRush2023 = NFLpassRush2023.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
NFLpassRush2023['Pressure%'] = NFLpassRush2023['total_pressures'] / NFLpassRush2023['snap_counts_pass_rush']
NFLpassRush2023 = NFLpassRush2023.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
NFLpassRush2023 = NFLpassRush2023.sort_values(by='player')
NFLpassRush2023.reset_index(drop=True, inplace=True)
NFL_ED_data_2023 = NFLpassRush2023.copy()
NFL_ED_data_2023 = NFL_ED_data_2023[NFL_ED_data_2023['position'] == 'ED']
edges = NFL_ED_data_2023['player'].tolist()
NFL_ED_data_2023

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
2,Aidan Hutchinson,ED,DET,21.0,10.0,0.168524
3,Alex Highsmith,ED,PIT,18.0,8.0,0.135593
5,Arden Key,ED,TEN,12.6,5.3,0.084034
9,Boye Mafe,ED,SEA,13.1,8.0,0.130926
10,Bradley Chubb,ED,MIA,16.6,9.3,0.147368
11,Brian Burns,ED,CAR,13.1,6.8,0.105541
13,Bryce Huff,ED,NYJ,22.9,12.2,0.200599
14,Bud Dupree,ED,ATL,6.6,5.6,0.087760
15,Byron Young,ED,LA,8.7,6.1,0.098456
16,Calais Campbell,ED,ATL,11.9,6.3,0.101695


In [50]:
NFLpassRush2022 = pd.read_csv("pass_rush_summary_2022.csv")
NFLpassRush2022 = NFLpassRush2022[NFLpassRush2022['snap_counts_pass_rush'] >= 325]
NFLpassRush2022 = NFLpassRush2022.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
NFLpassRush2022['Pressure%'] = NFLpassRush2022['total_pressures'] / NFLpassRush2022['snap_counts_pass_rush']
NFLpassRush2022.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
NFLpassRush2022 = NFLpassRush2022.sort_values(by='player')
NFLpassRush2022.reset_index(drop=True, inplace=True)
NFL_ED_data_2022 = NFLpassRush2022.copy()
NFL_ED_data_2022 = NFL_ED_data_2022[NFL_ED_data_2022['player'].isin(edges)]
NFL_ED_data_2022

,player,position,team_name,pass_rush_win_rate,prp,snap_counts_pass_rush,total_pressures,Pressure%
1,Aidan Hutchinson,ED,DET,12.4,6.0,566,53,0.093640
2,Alex Highsmith,ED,PIT,9.8,7.0,526,55,0.104563
4,Arden Key,ED,JAX,13.8,8.0,368,51,0.138587
7,Bradley Chubb,ED,MIA,9.8,7.3,471,54,0.114650
8,Brian Burns,ED,CAR,10.5,8.2,525,68,0.129524
9,Calais Campbell,DI,BLT,10.3,5.4,403,35,0.086849
11,Cameron Jordan,ED,NO,6.8,5.3,468,37,0.079060
19,D.J. Wonnum,ED,MIN,8.2,5.5,368,34,0.092391
23,Danielle Hunter,ED,MIN,15.9,7.8,605,76,0.125620
28,DeMarcus Lawrence,ED,DAL,10.3,7.2,478,59,0.123431


In [51]:
NFLpassRush2021 = pd.read_csv("pass_rush_summary_2021.csv")
NFLpassRush2021 = NFLpassRush2021[NFLpassRush2021['snap_counts_pass_rush'] >= 325]
NFLpassRush2021 = NFLpassRush2021.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
NFLpassRush2021['Pressure%'] = NFLpassRush2021['total_pressures'] / NFLpassRush2021['snap_counts_pass_rush']
NFLpassRush2021.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
NFLpassRush2021 = NFLpassRush2021.sort_values(by='player')
NFLpassRush2021.reset_index(drop=True, inplace=True)
NFL_ED_data_2021 = NFLpassRush2021.copy()
NFL_ED_data_2021 = NFL_ED_data_2021[NFL_ED_data_2021['player'].isin(edges)]
NFL_ED_data_2021

,player,position,team_name,pass_rush_win_rate,prp,snap_counts_pass_rush,total_pressures,Pressure%
3,Alex Highsmith,ED,PIT,11.9,5.0,497,38,0.076459
10,Brian Burns,ED,CAR,9.6,6.8,471,51,0.108280
11,Calais Campbell,DI,BLT,10.9,4.1,423,31,0.073286
13,Cameron Jordan,ED,NO,9.4,6.2,545,51,0.093578
21,D.J. Wonnum,ED,MIN,7.1,4.6,586,42,0.071672
31,Denico Autry,ED,TEN,12.9,7.7,535,64,0.119626
44,Haason Reddick,ED,CAR,9.8,7.2,438,44,0.100457
45,Harold Landry III,ED,TEN,7.7,8.0,569,70,0.123023
47,Jadeveon Clowney,ED,CLV,10.8,8.2,422,53,0.125592
57,John Franklin-Myers,ED,NYJ,15.3,7.2,431,53,0.122970


In [52]:
NFLpassRush2019 = pd.read_csv("pass_rush_summary_2019.csv")
NFLpassRush2019 = NFLpassRush2019[NFLpassRush2019['snap_counts_pass_rush'] >= 325]
NFLpassRush2019 = NFLpassRush2019.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
NFLpassRush2019['Pressure%'] = NFLpassRush2019['total_pressures'] / NFLpassRush2019['snap_counts_pass_rush']
NFLpassRush2019.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
NFLpassRush2019 = NFLpassRush2019.sort_values(by='player')
NFLpassRush2019.reset_index(drop=True, inplace=True)
NFL_ED_data_2019 = NFLpassRush2019.copy()
NFL_ED_data_2019 = NFL_ED_data_2019[NFL_ED_data_2019['player'].isin(edges)]
NFL_ED_data_2019

,player,position,team_name,pass_rush_win_rate,prp,snap_counts_pass_rush,total_pressures,Pressure%
6,Bud Dupree,ED,PIT,9.5,6.3,542,51,0.094096
7,Calais Campbell,DI,JAX,14.4,8.2,515,72,0.139806
9,Cameron Jordan,ED,NO,14.4,8.8,626,84,0.134185
20,Danielle Hunter,ED,MIN,14.2,9.9,616,97,0.157468
26,DeMarcus Lawrence,ED,DAL,12.4,7.5,443,56,0.126411
28,Denico Autry,DI,IND,8.6,4.3,424,30,0.070755
41,Harold Landry III,ED,TEN,6.1,5.9,678,66,0.097345
44,Jadeveon Clowney,ED,SEA,14.7,6.9,482,58,0.120332
54,Josh Allen,ED,JAX,11.2,8.5,388,49,0.126289
58,Khalil Mack,ED,CHI,12.4,7.9,531,70,0.131827


In [53]:
college_pass_rush2023 = pd.read_csv("college_pass_rush_summary2023.csv")
college_pass_rush2023 = college_pass_rush2023[college_pass_rush2023['player'].isin(prospects)]
college_pass_rush2023 = college_pass_rush2023.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
college_pass_rush2023['Pressure%'] = college_pass_rush2023['total_pressures'] / college_pass_rush2023['snap_counts_pass_rush']
college_pass_rush2023 = college_pass_rush2023.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
college_pass_rush2023 = college_pass_rush2023.sort_values(by='player')
college_pass_rush2023.reset_index(drop=True, inplace=True)
edge_predict_data = college_pass_rush2023.copy()
edge_predict_data = edge_predict_data[edge_predict_data['position'] == 'ED']
edge_predict_data.reset_index(drop=True, inplace=True)
edge_predict_data

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
0,Adisa Isaac,ED,PENN STATE,13.2,10.6,0.157360
1,Austin Booker,ED,KANSAS,14.8,10.5,0.160338
2,Bralen Trice,ED,WASHINGTON,19.0,9.0,0.153132
3,Chop Robinson,ED,PENN STATE,20.9,9.8,0.175676
4,Chris Braswell,ED,ALABAMA,17.8,12.3,0.182156
5,Dallas Turner,ED,ALABAMA,20.2,11.9,0.191571
6,Darius Robinson,ED,MISSOURI,17.2,9.0,0.140684
7,Gabriel Murphy,ED,UCLA,18.0,9.8,0.165165
8,Jared Verse,ED,FLORIDA ST,21.2,10.9,0.178694
9,Jonah Elliss,ED,UTAH,17.7,8.4,0.116766


In [54]:
college_pass_rush2022 = pd.read_csv("college_pass_rush_summary2022.csv")
college_pass_rush2022 = college_pass_rush2022.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
college_pass_rush2022['Pressure%'] = college_pass_rush2022['total_pressures'] / college_pass_rush2022['snap_counts_pass_rush']
college_pass_rush2022 = college_pass_rush2022.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
college_pass_rush2022.sort_values(by='player')
college_pass_rush2022.reset_index(drop=True, inplace=True)
edge_college_data2022 = college_pass_rush2022.copy()
edge_college_data2022 = edge_college_data2022[edge_college_data2022['player'].isin(edges)]
edge_college_data2022 = edge_college_data2022[edge_college_data2022['position'] == 'ED']
edge_college_data2022

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
1952,Yaya Diaby,ED,LOUISVILLE,9.8,8.2,0.117871
2132,Will Anderson Jr.,ED,ALABAMA,15.8,10.9,0.162534
2556,Tuli Tuipulotu,ED,USC,14.4,9.4,0.138418
3192,Byron Young,ED,TENNESSEE,8.3,6.6,0.103746


In [55]:
college_pass_rush2021 = pd.read_csv("college_pass_rush_summary2021.csv")
college_pass_rush2021 = college_pass_rush2021.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
college_pass_rush2021['Pressure%'] = college_pass_rush2021['total_pressures'] / college_pass_rush2021['snap_counts_pass_rush']
college_pass_rush2021 = college_pass_rush2021.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
college_pass_rush2021.sort_values(by='player')
college_pass_rush2021.reset_index(drop=True, inplace=True)
edge_college_data2021 = college_pass_rush2021.copy()
edge_college_data2021 = edge_college_data2021[edge_college_data2021['player'].isin(edges)]
edge_college_data2021 = edge_college_data2021[edge_college_data2021['position'] == 'ED']
edge_college_data2021

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
342,Boye Mafe,ED,MINNESOTA,15.8,10.4,0.168067
696,Josh Paschal,ED,KENTUCKY,15.4,6.0,0.097686
1402,Aidan Hutchinson,ED,MICHIGAN,23.3,11.8,0.188889
1945,George Karlaftis,ED,PURDUE,21.0,9.4,0.161194
1972,Kayvon Thibodeaux,ED,OREGON,20.7,10.7,0.169231
2088,Travon Walker,ED,GEORGIA,7.0,5.1,0.082353
2115,Jermaine Johnson,ED,FLORIDA ST,11.4,7.6,0.110843
2762,Yaya Diaby,ED,LOUISVILLE,15.4,5.1,0.088000
2941,Will Anderson Jr.,ED,ALABAMA,14.5,11.2,0.178947
3335,Tuli Tuipulotu,ED,USC,14.2,6.5,0.100346


In [56]:
college_pass_rush2019 = pd.read_csv("college_pass_rush_summary2019.csv")
college_pass_rush2019 = college_pass_rush2019.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
college_pass_rush2019['Pressure%'] = college_pass_rush2019['total_pressures'] / college_pass_rush2019['snap_counts_pass_rush']
college_pass_rush2019 = college_pass_rush2019.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
college_pass_rush2019.sort_values(by='player')
college_pass_rush2019.reset_index(drop=True, inplace=True)
edge_college_data2019 = college_pass_rush2019.copy()
edge_college_data2019 = edge_college_data2019[edge_college_data2019['player'].isin(edges)]
edge_college_data2019 = edge_college_data2019[edge_college_data2019['position'] == 'ED']
edge_college_data2019

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
77,Bryce Huff,ED,MEMPHIS,16.8,10.2,0.168750
574,Jonathon Cooper,ED,OHIO STATE,21.2,6.8,0.128571
696,Alex Highsmith,ED,CHARLOTTE,19.1,10.8,0.153094
783,Pat Jones II,ED,PITTSBURGH,14.2,9.7,0.156010
927,D.J. Wonnum,ED,S CAROLINA,7.8,5.5,0.086957
975,Jonathan Greenard,ED,FLORIDA,14.1,9.5,0.153846
976,Mike Danna,ED,MICHIGAN,10.0,7.0,0.115207
1265,Malcolm Koonce,ED,BUFFALO,20.3,10.1,0.163763
1315,Boye Mafe,ED,MINNESOTA,15.8,7.9,0.134454
1559,Chase Young,ED,OHIO STATE,29.2,13.9,0.194215


In [57]:
college_pass_rush2018 = pd.read_csv("college_pass_rush_summary2018.csv")
college_pass_rush2018 = college_pass_rush2018.drop(columns = ['player_id', 'player_game_count', 'batted_passes', 'declined_penalties', 'franchise_id',
       'grades_pass_rush_defense', 'hits', 'hurries', 'pass_rush_opp', 'pass_rush_percent','pass_rush_wins', 'penalties', 'sacks', 'snap_counts_pass_play', 'true_pass_set_batted_passes', 'true_pass_set_grades_pass_rush_defense',
       'true_pass_set_hits', 'true_pass_set_hurries',
       'true_pass_set_pass_rush_opp', 'true_pass_set_pass_rush_percent',
       'true_pass_set_pass_rush_win_rate', 'true_pass_set_pass_rush_wins',
       'true_pass_set_prp', 'true_pass_set_sacks',
       'true_pass_set_snap_counts_pass_play',
       'true_pass_set_snap_counts_pass_rush', 'true_pass_set_total_pressures'])
college_pass_rush2018['Pressure%'] = college_pass_rush2018['total_pressures'] / college_pass_rush2018['snap_counts_pass_rush']
college_pass_rush2018 = college_pass_rush2018.drop(columns = ['snap_counts_pass_rush', 'total_pressures'])
college_pass_rush2018.sort_values(by='player')
college_pass_rush2018.reset_index(drop=True, inplace=True)
edge_college_data2018 = college_pass_rush2018.copy()
edge_college_data2018 = edge_college_data2018[edge_college_data2018['player'].isin(edges)]
edge_college_data2018 = edge_college_data2018[edge_college_data2018['position'] == 'ED']
edge_college_data2018

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
92,Maxx Crosby,ED,E MICHIGAN,14.8,10.1,0.165079
138,Bryce Huff,ED,MEMPHIS,14.8,8.9,0.133562
143,Montez Sweat,ED,MISS STATE,16.6,11.1,0.173745
838,Nick Bosa,ED,OHIO STATE,29.9,13.4,0.200000
883,Rashan Gary,ED,MICHIGAN,14.5,8.7,0.137566
918,Jonathon Cooper,ED,OHIO STATE,8.1,5.6,0.089928
1057,Alex Highsmith,ED,CHARLOTTE,14.2,7.8,0.136508
1172,Pat Jones II,ED,PITTSBURGH,8.7,4.8,0.074890
1329,Brian Burns,ED,FLORIDA ST,17.5,9.2,0.155660
1336,D.J. Wonnum,ED,S CAROLINA,6.7,4.3,0.064220


In [58]:
NFL_blocking2023 = pd.read_csv("NFLoffense_blocking2023.csv")
NFL_blocking2023 = NFL_blocking2023.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
validOL = ['G', 'T', 'C']
NFL_blocking2023['Pressure%'] = NFL_blocking2023['pressures_allowed'] / NFL_blocking2023['snap_counts_block']
NFL_blocking2023 = NFL_blocking2023[NFL_blocking2023['position'].isin(validOL)]
NFL_blocking2023 = NFL_blocking2023[NFL_blocking2023['snap_counts_block'] > 590]
NFL_blocking2023 = NFL_blocking2023.sort_values(by='player')
NFL_blocking2023.reset_index(drop=True, inplace=True)
NFL_blocking2023 = NFL_blocking2023.drop(columns=['pressures_allowed'])
ol = NFL_blocking2023['player'].tolist()
NFL_blocking2023

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Aaron Banks,G,SF,54.9,55.2,837,0.027479
1,Aaron Brewer,C,TEN,52.5,78.7,1050,0.032381
2,Aaron Stinnie,G,TB,51.4,60.7,717,0.040446
3,Alaric Jackson,T,LA,67.2,60.1,968,0.040289
4,Alex Cappa,G,CIN,58.6,67.3,1066,0.032833
...,...,...,...,...,...,...,...
143,Will Hernandez,G,ARZ,70.2,60.6,1109,0.026150
144,Wyatt Teller,G,CLV,68.6,73.6,1187,0.022746
145,Zach Tom,T,GB,73.9,75.8,1068,0.030899
146,Zack Martin,G,DAL,77.2,64.3,908,0.024229


In [59]:
NFL_blocking2022 = pd.read_csv("NFLoffense_blocking2022.csv")
NFL_blocking2022 = NFL_blocking2022.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
NFL_blocking2022['Pressure%'] = NFL_blocking2022['pressures_allowed'] / NFL_blocking2022['snap_counts_block']
NFL_blocking2022 = NFL_blocking2022[NFL_blocking2022['player'].isin(ol)]
NFL_blocking2022 = NFL_blocking2022[NFL_blocking2022['snap_counts_block'] > 590]
NFL_blocking2022 = NFL_blocking2022.sort_values(by='player')
NFL_blocking2022.reset_index(drop=True, inplace=True)
NFL_blocking2022 = NFL_blocking2022.drop(columns=['pressures_allowed'])
NFL_blocking2022

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Aaron Banks,G,SF,66.5,59.1,969,0.025800
1,Aaron Brewer,G,TEN,45.4,63.3,1031,0.034918
2,Alex Cappa,G,CIN,71.4,62.3,1086,0.018416
3,Andre James,C,LV,64.5,59.3,964,0.024896
4,Andrew Wylie,T,KC,60.8,64.3,1093,0.044831
...,...,...,...,...,...,...,...
95,Will Fries,G,IND,44.4,66.2,642,0.031153
96,Will Hernandez,G,ARZ,79.1,61.1,843,0.018980
97,Wyatt Teller,G,CLV,67.9,70.3,927,0.028047
98,Zack Martin,G,DAL,81.2,64.8,1143,0.013998


In [60]:
NFL_blocking2021 = pd.read_csv("NFLoffense_blocking2021.csv")
NFL_blocking2021 = NFL_blocking2021.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
NFL_blocking2021['Pressure%'] = NFL_blocking2021['pressures_allowed'] / NFL_blocking2021['snap_counts_block']
NFL_blocking2021 = NFL_blocking2021[NFL_blocking2021['player'].isin(ol)]
NFL_blocking2021 = NFL_blocking2021[NFL_blocking2021['snap_counts_block'] > 590]
NFL_blocking2021 = NFL_blocking2021.sort_values(by='player')
NFL_blocking2021.reset_index(drop=True, inplace=True)
NFL_blocking2021 = NFL_blocking2021.drop(columns=['pressures_allowed'])
NFL_blocking2021

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Alex Cappa,G,TB,69.6,71.2,1182,0.027073
1,Andre James,C,LV,67.2,61.7,1139,0.014925
2,Austin Jackson,G,MIA,49.1,54.8,1096,0.044708
3,Ben Powers,G,BLT,66.1,63.6,844,0.024882
4,Bradley Bozeman,C,BLT,73.8,71.1,1125,0.018667
...,...,...,...,...,...,...,...
72,Tyler Biadasz,C,DAL,67.7,69.4,1202,0.017471
73,Tyron Smith,T,DAL,88.0,90.5,738,0.014905
74,Will Hernandez,G,NYG,56.0,58.4,1049,0.034318
75,Wyatt Teller,G,CLV,70.4,88.4,1107,0.021680


In [61]:
NFL_blocking2019 = pd.read_csv("NFLoffense_blocking2019.csv")
NFL_blocking2019 = NFL_blocking2019.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
NFL_blocking2019['Pressure%'] = NFL_blocking2019['pressures_allowed'] / NFL_blocking2019['snap_counts_block']
NFL_blocking2019 = NFL_blocking2019[NFL_blocking2019['player'].isin(ol)]
NFL_blocking2019 = NFL_blocking2019[NFL_blocking2019['snap_counts_block'] > 590]
NFL_blocking2019 = NFL_blocking2019.sort_values(by='player')
NFL_blocking2019.reset_index(drop=True, inplace=True)
NFL_blocking2019 = NFL_blocking2019.drop(columns=['pressures_allowed'])
NFL_blocking2019

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Alex Cappa,G,TB,59.4,61.5,869,0.035673
1,Andrew Wylie,G,KC,78.4,61.4,716,0.019553
2,Bradley Bozeman,G,BLT,55.8,65.8,1105,0.020814
3,Brandon Scherff,G,WAS,72.7,76.6,643,0.013997
4,Brian O'Neill,T,MIN,72.4,67.7,967,0.019648
5,Charles Leno Jr.,T,CHI,71.6,47.5,1066,0.037523
6,Cody Whitehair,G,CHI,64.1,63.3,1069,0.017774
7,Connor McGovern,C,DEN,82.5,64.1,1013,0.014808
8,D.J. Humphries,T,ARZ,76.3,52.3,1046,0.028681
9,Dalton Risner,G,DEN,69.8,60.3,975,0.022564


In [62]:
ol_predict_data = pd.read_csv('college_offense_blocking2023.csv')
ol_predict_data = ol_predict_data.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
ol_predict_data['Pressure%'] = ol_predict_data['pressures_allowed'] / ol_predict_data['snap_counts_block']
ol_predict_data = ol_predict_data[ol_predict_data['position'].isin(validOL)]
ol_predict_data = ol_predict_data[ol_predict_data['player'].isin(prospects)]
ol_predict_data = ol_predict_data.sort_values(by='player')
ol_predict_data.reset_index(drop=True, inplace=True)
ol_predict_data = ol_predict_data.drop(columns=['pressures_allowed'])
ol_predict_data

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Amarius Mims,T,GEORGIA,79.5,72.2,231,0.008658
1,Andrew Raym,C,OKLAHOMA,63.6,71.0,700,0.020000
2,Beaux Limmer,G,ARKANSAS,79.5,81.0,898,0.013363
3,Blake Fisher,T,NOTRE DAME,72.7,70.7,794,0.018892
4,Brandon Coleman,T,TCU,70.6,81.0,806,0.026055
5,Caedan Wallace,T,PENN STATE,60.8,56.2,320,0.034375
6,Christian Haynes,G,UCONN,90.1,87.2,736,0.004076
7,Cooper Beebe,G,KANSAS ST,89.5,77.7,785,0.007643
8,Delmar Glaze,T,MARYLAND,85.7,59.6,860,0.019767
9,Dominick Puni,G,KANSAS,75.7,66.9,750,0.010667


In [63]:
college_blocking2022 = pd.read_csv('college_offense_blocking_2022.csv')
college_blocking2022 = college_blocking2022.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
college_blocking2022['Pressure%'] = college_blocking2022['pressures_allowed'] / college_blocking2022['snap_counts_block']
college_blocking2022 = college_blocking2022[college_blocking2022['player'].isin(ol)]
college_blocking2022 = college_blocking2022[college_blocking2022['snap_counts_block'] > 300]
college_blocking2022 = college_blocking2022.sort_values(by='player')
college_blocking2022.reset_index(drop=True, inplace=True)
college_blocking2022 = college_blocking2022.drop(columns=['pressures_allowed'])
college_blocking2022

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Anthony Bradford,G,LSU,49.3,73.3,703,0.017070
1,Anton Harrison,T,OKLAHOMA,83.3,67.7,862,0.010441
2,Blake Freeland,T,BYU,90.1,85.8,805,0.008696
3,Broderick Jones,T,GEORGIA,85.4,73.8,730,0.006849
4,Darnell Wright,T,TENNESSEE,81.2,62.9,829,0.008444
5,Dawand Jones,T,OHIO STATE,81.9,83.8,693,0.005772
6,Joe Tippmann,C,WISCONSIN,79.2,78.2,758,0.006596
7,John Michael Schmitz Jr.,C,MINNESOTA,81.0,92.4,767,0.010430
8,Matthew Bergeron,T,SYRACUSE,80.8,69.0,686,0.017493
9,O'Cyrus Torrence,G,FLORIDA,76.0,89.9,698,0.011461


In [64]:
college_blocking2021 = pd.read_csv('college_offense_blocking2021.csv')
college_blocking2021 = college_blocking2021.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
college_blocking2021['Pressure%'] = college_blocking2021['pressures_allowed'] / college_blocking2021['snap_counts_block']
college_blocking2021 = college_blocking2021[college_blocking2021['player'].isin(ol)]
college_blocking2021 = college_blocking2021[college_blocking2021['snap_counts_block'] > 500]
college_blocking2021 = college_blocking2021.sort_values(by='player')
college_blocking2021.reset_index(drop=True, inplace=True)
college_blocking2021 = college_blocking2021.drop(columns=['pressures_allowed'])
college_blocking2021

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Anton Harrison,T,OKLAHOMA,85.2,70.7,650,0.026154
1,Bernhard Raimann,T,C MICHIGAN,88.7,94.6,895,0.011173
2,Blake Freeland,T,BYU,91.9,72.2,817,0.006120
3,Cam Jurgens,C,NEBRASKA,73.5,71.5,792,0.016414
4,Charles Cross,T,MISS STATE,84.7,87.2,919,0.017410
5,Darnell Wright,T,TENNESSEE,57.0,62.8,811,0.019729
6,Dawand Jones,T,OHIO STATE,77.8,90.3,721,0.015257
7,Dylan Parham,G,MEMPHIS,84.5,79.2,881,0.011351
8,Ed Ingram,G,LSU,81.8,67.4,756,0.014550
9,Ikem Ekwonu,T,NC STATE,78.3,93.8,819,0.015873


In [65]:
college_blocking2019 = pd.read_csv('college_offense_blocking2019.csv')
college_blocking2019 = college_blocking2019.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
college_blocking2019['Pressure%'] = college_blocking2019['pressures_allowed'] / college_blocking2019['snap_counts_block']
college_blocking2019 = college_blocking2019[college_blocking2019['player'].isin(ol)]
college_blocking2019 = college_blocking2019[college_blocking2019['snap_counts_block'] > 500]
college_blocking2019 = college_blocking2019.sort_values(by='player')
college_blocking2019.reset_index(drop=True, inplace=True)
college_blocking2019 = college_blocking2019.drop(columns=['pressures_allowed'])
college_blocking2019

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Aaron Banks,G,NOTRE DAME,80.7,66.7,778,0.011568
1,Aaron Brewer,T,TEXAS ST,79.6,83.5,781,0.020487
2,Alaric Jackson,T,IOWA,65.1,69.8,535,0.031776
3,Austin Jackson,T,USC,80.4,68.2,859,0.012806
4,Ben Bredeson,G,MICHIGAN,90.0,69.3,844,0.008294
5,Cam Jurgens,C,NEBRASKA,46.1,43.9,834,0.016787
6,Cesar Ruiz,C,MICHIGAN,89.7,66.7,838,0.008353
7,Christian Darrisaw,T,VA TECH,73.2,76.6,806,0.021092
8,Colton McKivitz,T,W VIRGINIA,82.5,68.5,808,0.018564
9,Creed Humphrey,C,OKLAHOMA,68.2,71.5,716,0.009777


In [66]:
college_blocking2018 = pd.read_csv('college_offense_blocking2018.csv')
college_blocking2018 = college_blocking2018.drop(columns=['player_id', 'player_game_count',
       'block_percent', 'declined_penalties', 'franchise_id', 'grades_offense', 'hits_allowed',
       'hurries_allowed', 'non_spike_pass_block','non_spike_pass_block_percentage', 'pass_block_percent', 'pbe',
       'penalties', 'sacks_allowed', 'snap_counts_ce', 'snap_counts_lg', 'snap_counts_lt',
       'snap_counts_offense', 'snap_counts_pass_block',
       'snap_counts_pass_play', 'snap_counts_rg', 'snap_counts_rt',
       'snap_counts_run_block', 'snap_counts_te'])
college_blocking2018['Pressure%'] = college_blocking2018['pressures_allowed'] / college_blocking2018['snap_counts_block']
college_blocking2018 = college_blocking2018[college_blocking2018['player'].isin(ol)]
college_blocking2018 = college_blocking2018[college_blocking2018['snap_counts_block'] > 500]
college_blocking2018 = college_blocking2018.sort_values(by='player')
college_blocking2018.reset_index(drop=True, inplace=True)
college_blocking2018 = college_blocking2018.drop(columns=['pressures_allowed'])
college_blocking2018

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Aaron Brewer,T,TEXAS ST,86.1,78.8,799,0.020025
1,Alaric Jackson,T,IOWA,83.1,73.6,766,0.011749
2,Andre James,T,UCLA,62.4,62.1,873,0.037801
3,Austin Jackson,T,USC,78.7,68.9,822,0.020681
4,Ben Bredeson,G,MICHIGAN,88.3,64.5,806,0.004963
5,Ben Powers,G,OKLAHOMA,87.9,56.0,679,0.008837
6,Cesar Ruiz,C,MICHIGAN,76.2,59.7,804,0.011194
7,Chris Lindstrom,G,BOSTON COL,91.6,70.7,864,0.004630
8,Christian Darrisaw,T,VA TECH,74.8,65.6,703,0.017070
9,Colton McKivitz,T,W VIRGINIA,84.9,73.1,807,0.007435


In [67]:
NFL_DL_data2023 = NFLpassRush2023.copy()
NFL_DL_data2023 = NFL_DL_data2023[NFL_DL_data2023['position'] == 'DI']
dl = NFL_DL_data2023['player'].tolist()
NFL_DL_data2023

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
0,Aaron Donald,DI,LA,22.3,8.7,0.145973
1,Adam Butler,DI,LV,9.2,4.9,0.077348
4,Alim McNeill,DI,DET,12.5,6.1,0.100233
6,Arik Armstead,DI,SF,15.2,7.5,0.125813
7,Austin Johnson,DI,LAC,2.5,2.5,0.047904
8,B.J. Hill,DI,CIN,8.2,5.0,0.084168
12,Bryan Bresee,DI,NO,7.3,4.9,0.080311
17,Calijah Kancey,DI,TB,9.5,6.1,0.101580
21,Chris Jones,DI,KC,17.8,8.8,0.143760
22,Christian Barmore,DI,NE,13.3,6.2,0.102510


In [68]:
NFL_DL_data2022 = NFLpassRush2022.copy()
NFL_DL_data2022 = NFL_DL_data2022[NFL_DL_data2022['position'] == 'DI']
NFL_DL_data2022

,player,position,team_name,pass_rush_win_rate,prp,snap_counts_pass_rush,total_pressures,Pressure%
0,Aaron Donald,DI,LA,17.8,5.9,396,40,0.101010
3,Alim McNeill,DI,DET,7.9,3.8,420,29,0.069048
5,B.J. Hill,DI,CIN,7.1,4.3,615,45,0.073171
6,Bilal Nichols,DI,LV,9.8,3.6,465,30,0.064516
9,Calais Campbell,DI,BLT,10.3,5.4,403,35,0.086849
10,Cameron Heyward,DI,PIT,15.5,7.4,498,58,0.116466
16,Chris Jones,DI,KC,18.2,8.2,741,97,0.130904
17,Christian Wilkins,DI,MIA,7.4,3.4,633,33,0.052133
18,D.J. Jones,DI,DEN,8.5,4.8,349,30,0.085960
20,DaQuan Jones,DI,BUF,11.0,4.5,457,33,0.072210


In [69]:
NFL_DL_data2021 = NFLpassRush2021.copy()
NFL_DL_data2021 = NFL_DL_data2021[NFL_DL_data2021['position'] == 'DI']
NFL_DL_data2021

,player,position,team_name,pass_rush_win_rate,prp,snap_counts_pass_rush,total_pressures,Pressure%
0,Aaron Donald,DI,LA,20.7,8.1,816,109,0.133578
1,Adam Butler,DI,MIA,6.9,3.2,419,24,0.057279
4,Arik Armstead,DI,SF,13.0,5.8,633,59,0.093207
5,Armon Watts,DI,MIN,10.1,4.9,414,33,0.079710
6,Austin Johnson,DI,NYG,6.5,3.7,332,21,0.063253
8,B.J. Hill,DI,CIN,10.4,5.1,481,38,0.079002
9,Bilal Nichols,DI,CHI,5.4,5.0,374,31,0.082888
11,Calais Campbell,DI,BLT,10.9,4.1,423,31,0.073286
12,Cameron Heyward,DI,PIT,12.2,6.3,605,61,0.100826
16,Chris Jones,DI,KC,16.8,8.7,527,75,0.142315


In [70]:
NFL_DL_data2019 = NFLpassRush2019.copy()
NFL_DL_data2019 = NFL_DL_data2019[NFL_DL_data2019['position'] == 'DI']
NFL_DL_data2019

,player,position,team_name,pass_rush_win_rate,prp,snap_counts_pass_rush,total_pressures,Pressure%
0,Aaron Donald,DI,LA,19.5,8.7,576,80,0.138889
1,Adam Butler,DI,NE,4.9,4.9,378,26,0.068783
4,Brandon Williams,DI,BLT,4.7,2.7,330,15,0.045455
7,Calais Campbell,DI,JAX,14.4,8.2,515,72,0.139806
8,Cameron Heyward,DI,PIT,13.0,7.4,505,60,0.118812
13,Charles Omenihu,DI,HST,9.6,5.2,356,32,0.089888
14,Chris Jones,DI,KC,15.0,8.2,480,64,0.133333
15,Christian Wilkins,DI,MIA,7.7,5.0,353,30,0.084986
17,Corey Peters,DI,ARZ,6.4,3.1,493,26,0.052738
18,DJ Reader,DI,HST,10.6,5.1,409,36,0.088020


In [71]:
dl_predict_data = college_pass_rush2023.copy()
dl_predict_data = dl_predict_data[dl_predict_data['position'] == 'DI']
dl_predict_data.reset_index(drop=True, inplace=True)
dl_predict_data

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
0,Braden Fiske,DI,FLORIDA ST,9.1,5.9,0.097458
1,Brandon Dorlus,DI,OREGON,10.8,7.3,0.125749
2,Byron Murphy II,DI,TEXAS,19.6,9.3,0.155172
3,DeWayne Carter,DI,DUKE,5.7,4.3,0.074576
4,Jer'Zhan Newton,DI,ILLINOIS,15.4,6.8,0.106965
5,Kris Jenkins,DI,MICHIGAN,10.9,5.1,0.084337
6,Leonard Taylor III,DI,MIAMI FL,14.7,5.8,0.104478
7,Mekhi Wingo,DI,LSU,7.2,4.4,0.068293
8,Ruke Orhorhoro,DI,CLEMSON,8.7,7.1,0.110638
9,T'Vondre Sweat,DI,TEXAS,15.5,6.2,0.109434


In [72]:
dl_college_data2022 = college_pass_rush2022.copy()
dl_college_data2022 = dl_college_data2022[dl_college_data2022['player'].isin(dl)]
dl_college_data2022 = dl_college_data2022[dl_college_data2022['position'] == 'DI']
dl_college_data2022

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
298,Kobie Turner,DI,WAKE,11.6,5.8,0.100000
1413,Calijah Kancey,DI,PITTSBURGH,19.6,10.6,0.170909
1854,Bryan Bresee,DI,CLEMSON,8.8,5.6,0.095238
2161,Jalen Carter,DI,GEORGIA,14.6,6.1,0.104396


In [73]:
dl_college_data2021 = college_pass_rush2021.copy()
dl_college_data2021 = dl_college_data2021[dl_college_data2021['player'].isin(dl)]
dl_college_data2021 = dl_college_data2021[dl_college_data2021['position'] == 'DI']
dl_college_data2021

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
1063,Logan Hall,DI,HOUSTON,13.6,6.8,0.103571
1382,Devonte Wyatt,DI,GEORGIA,12.0,6.6,0.103261
2226,Calijah Kancey,DI,PITTSBURGH,10.9,6.7,0.098684
2671,Bryan Bresee,DI,CLEMSON,5.2,5.7,0.100000
2968,Jalen Carter,DI,GEORGIA,17.3,8.1,0.134409


In [74]:
dl_college_data2019 = college_pass_rush2019.copy()
dl_college_data2019 = dl_college_data2019[dl_college_data2019['player'].isin(dl)]
dl_college_data2019 = dl_college_data2019[dl_college_data2019['position'] == 'DI']
dl_college_data2019

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
503,Osa Odighizuwa,DI,UCLA,11.0,4.9,0.081006
767,Derrick Brown,DI,AUBURN,13.4,5.8,0.094972
860,Justin Madubuike,DI,TEXAS A&M,11.8,7.3,0.119534
1696,Javon Kinlaw,DI,S CAROLINA,15.7,7.1,0.113636
2331,Logan Hall,DI,HOUSTON,7.6,2.4,0.045685
2370,Alim McNeill,DI,NC STATE,9.5,5.2,0.071429
2688,Christian Barmore,DI,ALABAMA,16.1,10.5,0.171642
2720,Devonte Wyatt,DI,GEORGIA,12.2,6.3,0.120172


In [75]:
dl_college_data2018 = college_pass_rush2018.copy()
dl_college_data2018 = dl_college_data2018[dl_college_data2018['player'].isin(dl)]
dl_college_data2018 = dl_college_data2018[dl_college_data2018['position'] == 'DI']
dl_college_data2018

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
76,Dexter Lawrence,DI,CLEMSON,12.8,7.0,0.128631
191,Dre'Mont Jones,DI,OHIO STATE,13.4,7.1,0.109181
743,Ed Oliver,DI,HOUSTON,14.2,5.7,0.097015
839,Osa Odighizuwa,DI,UCLA,9.2,6.4,0.104265
1114,Jeffery Simmons,DI,MISS STATE,15.4,5.3,0.094156
1147,Derrick Brown,DI,AUBURN,11.4,5.4,0.085809
1247,Quinnen Williams,DI,ALABAMA,18.0,10.4,0.164706
1266,Justin Madubuike,DI,TEXAS A&M,11.1,6.4,0.106267
1811,Christian Wilkins,DI,CLEMSON,16.9,8.5,0.134328
2583,Javon Kinlaw,DI,S CAROLINA,11.1,4.8,0.074324


In [76]:
cb_valid_data = cb_NFL_data2023.copy()
cbs = cb_valid_data['Player'].tolist()
cbs22 = cb_NFL_data2022['Player'].tolist()
cbs21 = cb_NFL_data2021['Player'].tolist()
cbs19 = cb_NFL_data2019['Player'].tolist()
for c in cbs:
    if c in cbs22:
        if c in cbs21:
            if c in cbs19:
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'FI%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'FI%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'FI%'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'FI%'].iloc[0] + cb_NFL_data2019.loc[cb_NFL_data2019['Player'] == c, 'FI%'].iloc[0])/4
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'MIS%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'MIS%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'MIS%'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'MIS%'].iloc[0]  + cb_NFL_data2019.loc[cb_NFL_data2019['Player'] == c, 'MIS%'].iloc[0])/4 
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'YAC%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'YAC%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'YAC%'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'YAC%'].iloc[0] + cb_NFL_data2019.loc[cb_NFL_data2019['Player'] == c, 'YAC%'].iloc[0])/4
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'Yards/Rec'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'Yards/Rec'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'Yards/Rec'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'Yards/Rec'].iloc[0] + cb_NFL_data2019.loc[cb_NFL_data2019['Player'] == c, 'Yards/Rec'].iloc[0])/4 
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'Target Rate'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'Target Rate'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'Target Rate'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'Target Rate'].iloc[0] + cb_NFL_data2019.loc[cb_NFL_data2019['Player'] == c, 'Target Rate'].iloc[0])/2 
            else:
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'FI%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'FI%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'FI%'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'FI%'].iloc[0])/3
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'MIS%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'MIS%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'MIS%'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'MIS%'].iloc[0])/3 
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'YAC%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'YAC%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'YAC%'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'YAC%'].iloc[0])/3
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'Yards/Rec'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'Yards/Rec'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'Yards/Rec'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'Yards/Rec'].iloc[0])/3 
                cb_valid_data.loc[cb_valid_data['Player'] == c, 'Target Rate'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'Target Rate'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'Target Rate'].iloc[0] + cb_NFL_data2021.loc[cb_NFL_data2021['Player'] == c, 'Target Rate'].iloc[0])/3 
        
        else:
            cb_valid_data.loc[cb_valid_data['Player'] == c, 'FI%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'FI%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'FI%'].iloc[0])/2
            cb_valid_data.loc[cb_valid_data['Player'] == c, 'MIS%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'MIS%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'MIS%'].iloc[0])/2 
            cb_valid_data.loc[cb_valid_data['Player'] == c, 'YAC%'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'YAC%'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'YAC%'].iloc[0])/2
            cb_valid_data.loc[cb_valid_data['Player'] == c, 'Yards/Rec'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'Yards/Rec'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'Yards/Rec'].iloc[0])/2 
            cb_valid_data.loc[cb_valid_data['Player'] == c, 'Target Rate'] = (cb_valid_data.loc[cb_valid_data['Player'] == c, 'Target Rate'].iloc[0] + cb_NFL_data2022.loc[cb_NFL_data2022['Player'] == c, 'Target Rate'].iloc[0])/2 
cb_valid_data        

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,A.J. Terrell,CB,ATL,20.566667,16.033333,9.800000,0.130659,3.532641
1,Adoree' Jackson,CB,NYG,15.225000,9.500000,11.400000,0.285228,4.414456
2,Ahkello Witherspoon,CB,LA,12.500000,19.100000,15.100000,0.150072,3.865385
3,Akayleb Evans,CB,MIN,8.700000,14.300000,12.400000,0.130435,5.020833
4,Alontae Taylor,CB,NO,11.500000,13.600000,11.100000,0.188648,5.493333
...,...,...,...,...,...,...,...,...
70,Taron Johnson,CB,BUF,9.500000,9.033333,9.766667,0.129597,5.586445
71,Trent McDuffie,CB,KC,11.250000,10.600000,10.650000,0.114003,4.464056
72,Tyrique Stevenson,CB,CHI,12.700000,9.400000,11.600000,0.198915,4.763889
73,Xavien Howard,CB,MIA,14.866667,13.366667,14.000000,0.133902,5.375740


In [77]:
cb_train_data = cb_college_data2018.copy()
cbList = cb_train_data['Player'].tolist()
temp1 = cb_college_data2019.copy()
temp2 = cb_college_data2021.copy()
temp3 = cb_college_data2022.copy()
temp1 = temp1[~temp1['Player'].isin(cbList)]
cb_train_data = pd.concat([cb_train_data, temp1]).drop_duplicates(keep=False)
tempCB = temp1['Player'].tolist()
collegeCB2019 = cb_college_data2019.copy()
collegeCB2019 = collegeCB2019[~collegeCB2019['Player'].isin(tempCB)]
cbList = cb_train_data['Player'].tolist()
temp2 = temp2[temp2['Player'].isin(cbList)]
tempCB = temp2['Player'].tolist()
collegeCB2021 = cb_college_data2021.copy()
collegeCB2021 = collegeCB2021[~collegeCB2021['Player'].isin(tempCB)]
cb_train_data = pd.concat([cb_train_data, temp2]).drop_duplicates(keep=False)
cbList = cb_train_data['Player'].tolist()
temp3 = temp3[~temp3['Player'].isin(cbList)]
cb_train_data = pd.concat([cb_train_data, temp3]).drop_duplicates(keep=False)
tempCB = temp3['Player'].tolist()
collegeCB2022 = cb_college_data2022.copy()
collegeCB2022 = collegeCB2022[~collegeCB2022['Player'].isin(tempCB)]
cb_train_data.reset_index(drop=True, inplace=True)
cbs = cb_train_data['Player'].tolist()
cb22 = collegeCB2022['Player'].tolist()
cb21 = collegeCB2021['Player'].tolist()
cb19 = collegeCB2019['Player'].tolist()
for c in cbs:
    if c in cb22:
        if c in cb21:
            if c in cb19:
                cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'FI%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'FI%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'FI%'].iloc[0])/4
                cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'MIS%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'MIS%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'MIS%'].iloc[0])/4 
                cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'YAC%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'YAC%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'YAC%'].iloc[0])/4
                cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'Yards/Rec'].iloc[0])/4 
                cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'Target Rate'].iloc[0])/4
            else:
                cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'FI%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'FI%'].iloc[0])/3
                cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'MIS%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'MIS%'].iloc[0])/3 
                cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'YAC%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'YAC%'].iloc[0])/3
                cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'Yards/Rec'].iloc[0])/3 
                cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'Target Rate'].iloc[0])/3 
        else:
            cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'FI%'].iloc[0])/2
            cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'MIS%'].iloc[0])/2
            cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'YAC%'].iloc[0])/2
            cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'Yards/Rec'].iloc[0])/2 
            cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2022.loc[collegeCB2022['Player'] == c, 'Target Rate'].iloc[0])/2
    elif c in cb21:
        if c in cb19:
            cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'FI%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'FI%'].iloc[0])/3
            cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'MIS%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'MIS%'].iloc[0])/3
            cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'YAC%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'YAC%'].iloc[0])/3
            cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'Yards/Rec'].iloc[0])/3 
            cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'Target Rate'].iloc[0])/3
        else:
            cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'FI%'].iloc[0])/2
            cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'MIS%'].iloc[0])/2
            cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'YAC%'].iloc[0])/2
            cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'Yards/Rec'].iloc[0])/2 
            cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2021.loc[collegeCB2021['Player'] == c, 'Target Rate'].iloc[0])/2
    elif c in cb19:
        cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'FI%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'FI%'].iloc[0])/2
        cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'MIS%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'MIS%'].iloc[0])/2 
        cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'YAC%'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'YAC%'].iloc[0])/2
        cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Yards/Rec'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'Yards/Rec'].iloc[0])/2 
        cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == c, 'Target Rate'].iloc[0] + collegeCB2019.loc[collegeCB2019['Player'] == c, 'Target Rate'].iloc[0])/2
    else:
        pass
                
cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'FI%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'FI%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'MIS%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'MIS%'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'YAC%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'YAC%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'Yards/Rec'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'Yards/Rec'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'Target Rate'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Ja\'Quan McMillian', 'Target Rate'].iloc[1])/2    
cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'FI%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'FI%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'MIS%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'MIS%'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'YAC%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'YAC%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'Yards/Rec'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'Yards/Rec'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'Target Rate'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Kyler Gordon', 'Target Rate'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'FI%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'FI%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'MIS%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'MIS%'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'YAC%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'YAC%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'Yards/Rec'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'Yards/Rec'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'Target Rate'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Roger McCreary', 'Target Rate'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'FI%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'FI%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'MIS%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'MIS%'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'YAC%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'YAC%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'Yards/Rec'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'Yards/Rec'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'Target Rate'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Sauce Gardner', 'Target Rate'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'FI%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'FI%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'FI%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'MIS%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'MIS%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'MIS%'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'YAC%'] = (cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'YAC%'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'YAC%'].iloc[1])/2
cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'Yards/Rec'] = (cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'Yards/Rec'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'Yards/Rec'].iloc[1])/2 
cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'Target Rate'] = (cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'Target Rate'].iloc[0] + cb_train_data.loc[cb_train_data['Player'] == 'Trent McDuffie', 'Target Rate'].iloc[1])/2 
cb_train_data = cb_train_data.drop_duplicates(subset='Player')
cb_train_data = cb_train_data.sort_values(by='Player')
cb_train_data.reset_index(drop=True, inplace=True)
cb_train_data  

,Player,Position,Team,FI%,MIS%,Yards/Rec,Target Rate,YAC%
0,A.J. Terrell,CB,CLEMSON,11.150,3.55,10.950,0.108678,1.644928
1,Alontae Taylor,CB,TENNESSEE,3.575,16.30,11.875,0.113322,4.159091
2,Ambry Thomas,CB,MICHIGAN,12.800,7.10,13.300,0.150160,5.240000
3,Amik Robertson,CB,LA TECH,20.550,16.15,12.050,0.166696,6.239525
4,Asante Samuel Jr.,CB,FLORIDA ST,18.800,8.90,12.700,0.145455,3.088235
5,Brandon Stephens,CB,SMU,16.900,5.90,16.600,0.133745,4.645161
6,Byron Murphy Jr.,CB,WASHINGTON,26.900,8.30,9.900,0.115813,2.517241
7,Carrington Valentine,CB,KENTUCKY,9.050,10.05,12.550,0.164940,4.651596
8,DJ Turner II,CB,MICHIGAN,17.600,9.20,12.050,0.151474,3.157407
9,Deommodore Lenoir,CB,OREGON,20.200,6.30,13.800,0.144268,4.242857


In [78]:
ol_valid_data = NFL_blocking2023.copy()
ols = ol_valid_data['player'].tolist()
ols22 = NFL_blocking2022['player'].tolist()
ols21 = NFL_blocking2021['player'].tolist()
ols19 = NFL_blocking2019['player'].tolist()
for o in ols:
    if o in ols22:
        if o in ols21:
            if o in ols19:
                ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_pass_block'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_pass_block'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'grades_pass_block'].iloc[0] + NFL_blocking2021.loc[NFL_blocking2021['player'] == o, 'grades_pass_block'].iloc[0] + NFL_blocking2019.loc[NFL_blocking2019['player'] == o, 'grades_pass_block'].iloc[0])/4
                ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_run_block'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_run_block'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'grades_run_block'].iloc[0] + NFL_blocking2021.loc[NFL_blocking2021['player'] == o, 'grades_run_block'].iloc[0] + NFL_blocking2019.loc[NFL_blocking2019['player'] == o, 'grades_run_block'].iloc[0])/4
                ol_valid_data.loc[ol_valid_data['player'] == o, 'Pressure%'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'Pressure%'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'Pressure%'].iloc[0] + NFL_blocking2021.loc[NFL_blocking2021['player'] == o, 'Pressure%'].iloc[0] + NFL_blocking2019.loc[NFL_blocking2019['player'] == o, 'Pressure%'].iloc[0])/4
            else:
                ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_pass_block'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_pass_block'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'grades_pass_block'].iloc[0] + NFL_blocking2021.loc[NFL_blocking2021['player'] == o, 'grades_pass_block'].iloc[0])/3
                ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_run_block'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_run_block'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'grades_run_block'].iloc[0] + NFL_blocking2021.loc[NFL_blocking2021['player'] == o, 'grades_run_block'].iloc[0])/3
                ol_valid_data.loc[ol_valid_data['player'] == o, 'Pressure%'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'Pressure%'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'Pressure%'].iloc[0] + NFL_blocking2021.loc[NFL_blocking2021['player'] == o, 'Pressure%'].iloc[0])/3
      
        else:
            ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_pass_block'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_pass_block'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'grades_pass_block'].iloc[0])/2
            ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_run_block'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'grades_run_block'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'grades_run_block'].iloc[0])/2
            ol_valid_data.loc[ol_valid_data['player'] == o, 'Pressure%'] = (ol_valid_data.loc[ol_valid_data['player'] == o, 'Pressure%'].iloc[0] + NFL_blocking2022.loc[NFL_blocking2022['player'] == o, 'Pressure%'].iloc[0])/2
ol_valid_data        

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Aaron Banks,G,SF,60.700000,57.150000,837,0.026639
1,Aaron Brewer,C,TEN,48.950000,71.000000,1050,0.033649
2,Aaron Stinnie,G,TB,51.400000,60.700000,717,0.040446
3,Alaric Jackson,T,LA,67.200000,60.100000,968,0.040289
4,Alex Cappa,G,CIN,64.750000,65.575000,1066,0.028499
...,...,...,...,...,...,...,...
143,Will Hernandez,G,ARZ,69.450000,56.450000,1109,0.027119
144,Wyatt Teller,G,CLV,68.966667,77.433333,1187,0.024158
145,Zach Tom,T,GB,73.900000,75.800000,1068,0.030899
146,Zack Martin,G,DAL,84.475000,75.500000,908,0.018135


In [79]:
ol_train_data = college_blocking2018.copy()
olList = ol_train_data['player'].tolist()
temp1 = college_blocking2019.copy()
temp2 = college_blocking2021.copy()
temp3 = college_blocking2022.copy()
temp1 = temp1[~temp1['player'].isin(olList)]
ol_train_data = pd.concat([ol_train_data, temp1]).drop_duplicates(keep=False)
tempOL = temp1['player'].tolist()
collegeOL2019 = college_blocking2019.copy()
collegeOL2019 = collegeOL2019[~collegeOL2019['player'].isin(tempOL)]
olList = ol_train_data['player'].tolist()
temp2 = temp2[temp2['player'].isin(olList)]
tempOL = temp2['player'].tolist()
collegeOL2021 = college_blocking2021.copy()
collegeOL2021 = collegeOL2021[~collegeOL2021['player'].isin(tempOL)]
ol_train_data = pd.concat([ol_train_data, temp2]).drop_duplicates(keep=False)
olList = ol_train_data['player'].tolist()
temp3 = temp3[~temp3['player'].isin(olList)]
ol_train_data = pd.concat([ol_train_data, temp3]).drop_duplicates(keep=False)
tempOL = temp3['player'].tolist()
collegeOL2022 = college_blocking2022.copy()
collegeOL2022 = collegeOL2022[~collegeOL2022['player'].isin(tempOL)]
ol_train_data.reset_index(drop=True, inplace=True)
ol22 = collegeOL2022['player'].tolist()
ol21 = collegeOL2021['player'].tolist()
ol19 = collegeOL2019['player'].tolist()
ols = ol_train_data['player'].tolist()
for o in ols:
    if o in ol22:
        if o in ol21:
            if o in ol19:
                ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'grades_pass_block'].iloc[0])/4
                ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'grades_run_block'].iloc[0])/4
                ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'Pressure%'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'Pressure%'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'Pressure%'].iloc[0])/4
            else:
                ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'grades_pass_block'].iloc[0])/3
                ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'grades_run_block'].iloc[0])/3
                ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'Pressure%'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'Pressure%'].iloc[0])/3
        else:
            ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'grades_pass_block'].iloc[0])/2
            ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'grades_run_block'].iloc[0])/2
            ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'].iloc[0] + collegeOL2022.loc[collegeOL2022['player'] == o, 'Pressure%'].iloc[0])/2
    elif o in ol21:
        if o in ol19:
            ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'grades_pass_block'].iloc[0])/3
            ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'grades_run_block'].iloc[0])/3
            ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'Pressure%'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'Pressure%'].iloc[0])/3
        else:
            ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'grades_pass_block'].iloc[0])/2
            ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'grades_run_block'].iloc[0])/2
            ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'].iloc[0] + collegeOL2021.loc[collegeOL2021['player'] == o, 'Pressure%'].iloc[0])/2
    elif o in ol19:
        ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_pass_block'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'grades_pass_block'].iloc[0])/2
        ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == o, 'grades_run_block'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'grades_run_block'].iloc[0])/2
        ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == o, 'Pressure%'].iloc[0] + collegeOL2019.loc[collegeOL2019['player'] == o, 'Pressure%'].iloc[0])/2
    else:
        pass

    
ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'grades_pass_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'grades_pass_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'grades_run_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'grades_run_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'Pressure%'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Cam Jurgens', 'Pressure%'].iloc[1])/2    
ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'grades_pass_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'grades_pass_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'grades_run_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'grades_run_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'Pressure%'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Ikem Ekwonu', 'Pressure%'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'grades_pass_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'grades_pass_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'grades_run_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'grades_run_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'Pressure%'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Luke Fortner', 'Pressure%'].iloc[1])/2  
ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'grades_pass_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'grades_pass_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'grades_run_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'grades_run_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'Pressure%'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Luke Goedeke', 'Pressure%'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'grades_pass_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'grades_pass_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'grades_run_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'grades_run_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'Pressure%'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Rasheed Walker', 'Pressure%'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'grades_pass_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'grades_pass_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'grades_run_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'grades_run_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'Pressure%'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Tyler Linderbaum', 'Pressure%'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'grades_pass_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'grades_pass_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'grades_run_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'grades_run_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'Pressure%'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Zach Tom', 'Pressure%'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'grades_pass_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'grades_pass_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'grades_pass_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'grades_run_block'] = (ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'grades_run_block'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'grades_run_block'].iloc[1])/2
ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'Pressure%'] = (ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'Pressure%'].iloc[0] + ol_train_data.loc[ol_train_data['player'] == 'Zion Johnson', 'Pressure%'].iloc[1])/2
ol_train_data = ol_train_data.drop_duplicates(subset='player')
ol_train_data = ol_train_data.sort_values(by='player')
ol_train_data.reset_index(drop=True, inplace=True)
ol_train_data        

,player,position,team_name,grades_pass_block,grades_run_block,snap_counts_block,Pressure%
0,Aaron Banks,G,NOTRE DAME,80.70,66.70,778,0.011568
1,Aaron Brewer,T,TEXAS ST,82.85,81.15,799,0.020256
2,Alaric Jackson,T,IOWA,74.10,71.70,766,0.021763
3,Andre James,T,UCLA,62.40,62.10,873,0.037801
4,Anthony Bradford,G,LSU,49.30,73.30,703,0.017070
...,...,...,...,...,...,...,...
68,Tyler Linderbaum,C,IOWA,78.95,88.55,800,0.009491
69,Walker Little,T,STANFORD,81.20,60.00,790,0.015190
70,Will Fries,T,PENN STATE,53.00,66.05,676,0.040627
71,Zach Tom,C,WAKE,85.05,69.05,957,0.012414


In [80]:
edge_valid_data = NFL_ED_data_2023.copy()
edges = edge_valid_data['player'].tolist()
edge22 = NFL_ED_data_2022['player'].tolist()
edge21 = NFL_ED_data_2021['player'].tolist()
edge19 = NFL_ED_data_2019['player'].tolist()
for e in edges:
    if e in edge22:
        if e in edge21:
            if e in edge19:
                edge_valid_data.loc[edge_valid_data['player'] == e, 'pass_rush_win_rate'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'pass_rush_win_rate'].iloc[0] + NFL_ED_data_2021.loc[NFL_ED_data_2021['player'] == e, 'pass_rush_win_rate'].iloc[0] + NFL_ED_data_2019.loc[NFL_ED_data_2019['player'] == e, 'pass_rush_win_rate'].iloc[0])/4
                edge_valid_data.loc[edge_valid_data['player'] == e, 'prp'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'prp'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'prp'].iloc[0] + NFL_ED_data_2021.loc[NFL_ED_data_2021['player'] == e, 'prp'].iloc[0] + NFL_ED_data_2019.loc[NFL_ED_data_2019['player'] == e, 'prp'].iloc[0])/4
                edge_valid_data.loc[edge_valid_data['player'] == e, 'Pressure%'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'Pressure%'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'Pressure%'].iloc[0] + NFL_ED_data_2021.loc[NFL_ED_data_2021['player'] == e, 'Pressure%'].iloc[0] + NFL_ED_data_2019.loc[NFL_ED_data_2019['player'] == e, 'Pressure%'].iloc[0])/4  
            else:
                edge_valid_data.loc[edge_valid_data['player'] == e, 'pass_rush_win_rate'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'pass_rush_win_rate'].iloc[0] + NFL_ED_data_2021.loc[NFL_ED_data_2021['player'] == e, 'pass_rush_win_rate'].iloc[0])/3
                edge_valid_data.loc[edge_valid_data['player'] == e, 'prp'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'prp'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'prp'].iloc[0] + NFL_ED_data_2021.loc[NFL_ED_data_2021['player'] == e, 'prp'].iloc[0])/3
                edge_valid_data.loc[edge_valid_data['player'] == e, 'Pressure%'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'Pressure%'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'Pressure%'].iloc[0] + NFL_ED_data_2021.loc[NFL_ED_data_2021['player'] == e, 'Pressure%'].iloc[0])/3        

        else:
            edge_valid_data.loc[edge_valid_data['player'] == e, 'pass_rush_win_rate'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'pass_rush_win_rate'].iloc[0])/2
            edge_valid_data.loc[edge_valid_data['player'] == e, 'prp'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'prp'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'prp'].iloc[0])/2
            edge_valid_data.loc[edge_valid_data['player'] == e, 'Pressure%'] = (edge_valid_data.loc[edge_valid_data['player'] == e, 'Pressure%'].iloc[0] + NFL_ED_data_2022.loc[NFL_ED_data_2022['player'] == e, 'Pressure%'].iloc[0])/2        
edge_valid_data

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
2,Aidan Hutchinson,ED,DET,16.700000,8.000000,0.131082
3,Alex Highsmith,ED,PIT,13.233333,6.666667,0.105538
5,Arden Key,ED,TEN,13.200000,6.650000,0.111310
9,Boye Mafe,ED,SEA,13.100000,8.000000,0.130926
10,Bradley Chubb,ED,MIA,13.200000,8.300000,0.131009
11,Brian Burns,ED,CAR,11.066667,7.266667,0.114448
13,Bryce Huff,ED,NYJ,22.900000,12.200000,0.200599
14,Bud Dupree,ED,ATL,6.600000,5.600000,0.087760
15,Byron Young,ED,LA,8.700000,6.100000,0.098456
16,Calais Campbell,ED,ATL,11.875000,6.000000,0.100409


In [81]:
edge_train_data = edge_college_data2018.copy()
edgeList = edge_train_data['player'].tolist()
temp1 = edge_college_data2019.copy()
temp2 = edge_college_data2021.copy()
temp3 = edge_college_data2022.copy()
temp1 = temp1[~temp1['player'].isin(edgeList)]
edge_train_data = pd.concat([edge_train_data, temp1]).drop_duplicates(keep=False)
tempED = temp1['player'].tolist()
collegeED2019 = edge_college_data2019.copy()
collegeED2019 = collegeED2019[~collegeED2019['player'].isin(tempED)]
edgeList = edge_train_data['player'].tolist()
temp2 = temp2[temp2['player'].isin(edgeList)]
tempED = temp2['player'].tolist()
collegeED2021 = edge_college_data2021.copy()
collegeED2021 = collegeED2021[~collegeED2021['player'].isin(tempED)]
edge_train_data = pd.concat([edge_train_data, temp2]).drop_duplicates(keep=False)
edgeList = edge_train_data['player'].tolist()
temp3 = temp3[~temp3['player'].isin(edgeList)]
edge_train_data = pd.concat([edge_train_data, temp3]).drop_duplicates(keep=False)
tempED = temp3['player'].tolist()
collegeED2022 = edge_college_data2022.copy()
collegeED2022 = collegeED2022[~collegeED2022['player'].isin(tempED)]
edge_train_data.reset_index(drop=True, inplace=True)
ed22 = collegeED2022['player'].tolist()
ed21 = collegeED2021['player'].tolist()
ed19 = collegeED2019['player'].tolist()
edges = edge_train_data['player'].tolist()
for e in edges:
    if e in ed22:
        if e in ed21:
            if e in ed19:
                edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'pass_rush_win_rate'].iloc[0])/4
                edge_train_data.loc[edge_train_data['player'] == e, 'prp'] = (edge_train_data.loc[edge_train_data['player'] == e, 'prp'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'prp'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'prp'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'prp'].iloc[0])/4
                edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'Pressure%'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'Pressure%'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'Pressure%'].iloc[0])/4
            else:
                edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'pass_rush_win_rate'].iloc[0])/3
                edge_train_data.loc[edge_train_data['player'] == e, 'prp'] = (edge_train_data.loc[edge_train_data['player'] == e, 'prp'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'prp'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'prp'].iloc[0])/3
                edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'Pressure%'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'Pressure%'].iloc[0])/3
        else:
            edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'pass_rush_win_rate'].iloc[0])/2
            edge_train_data.loc[edge_train_data['player'] == e, 'prp'] = (edge_train_data.loc[edge_train_data['player'] == e, 'prp'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'prp'].iloc[0])/2
            edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'].iloc[0] + collegeED2022.loc[collegeED2022['player'] == e, 'Pressure%'].iloc[0])/2    
    elif e in ed21:
        if e in ed19:
            edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'pass_rush_win_rate'].iloc[0])/3
            edge_train_data.loc[edge_train_data['player'] == e, 'prp'] = (edge_train_data.loc[edge_train_data['player'] == e, 'prp'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'prp'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'prp'].iloc[0])/3
            edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'Pressure%'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'Pressure%'].iloc[0])/3
        else:
            edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'pass_rush_win_rate'].iloc[0])/2
            edge_train_data.loc[edge_train_data['player'] == e, 'prp'] = (edge_train_data.loc[edge_train_data['player'] == e, 'prp'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'prp'].iloc[0])/2
            edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'].iloc[0] + collegeED2021.loc[collegeED2021['player'] == e, 'Pressure%'].iloc[0])/2
    elif e in ed19:
        edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == e, 'pass_rush_win_rate'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'pass_rush_win_rate'].iloc[0])/2
        edge_train_data.loc[edge_train_data['player'] == e, 'prp'] = (edge_train_data.loc[edge_train_data['player'] == e, 'prp'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'prp'].iloc[0])/2
        edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == e, 'Pressure%'].iloc[0] + collegeED2019.loc[collegeED2019['player'] == e, 'Pressure%'].iloc[0])/2
    else:
        pass

edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'pass_rush_win_rate'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'pass_rush_win_rate'].iloc[1])/2
edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'prp'] = (edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'prp'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'prp'].iloc[1])/2
edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'Pressure%'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'George Karlaftis', 'Pressure%'].iloc[1])/2    
edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'pass_rush_win_rate'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'pass_rush_win_rate'].iloc[1])/2
edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'prp'] = (edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'prp'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'prp'].iloc[1])/2
edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'Pressure%'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'Jermaine Johnson', 'Pressure%'].iloc[1])/2 
edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'pass_rush_win_rate'] = (edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'pass_rush_win_rate'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'pass_rush_win_rate'].iloc[1])/2
edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'prp'] = (edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'prp'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'prp'].iloc[1])/2
edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'Pressure%'] = (edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'Pressure%'].iloc[0] + edge_train_data.loc[edge_train_data['player'] == 'Kayvon Thibodeaux', 'Pressure%'].iloc[1])/2 
edge_train_data = edge_train_data.drop_duplicates(subset='player')    
edge_train_data = edge_train_data.sort_values(by='player')  
edge_train_data.reset_index(drop=True, inplace=True)
edge_train_data

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
0,Aidan Hutchinson,ED,MICHIGAN,10.25,6.200,0.102195
1,Alex Highsmith,ED,CHARLOTTE,16.65,9.300,0.144801
2,Boye Mafe,ED,MINNESOTA,16.60,7.900,0.134422
3,Brian Burns,ED,FLORIDA ST,17.50,9.200,0.155660
4,Bryce Huff,ED,MEMPHIS,15.80,9.550,0.151156
5,Byron Young,ED,TENNESSEE,9.30,7.400,0.115222
6,Carl Granderson,ED,WYOMING,12.10,6.200,0.112540
7,Chase Young,ED,OHIO STATE,23.80,11.850,0.178224
8,D.J. Wonnum,ED,S CAROLINA,7.25,4.900,0.075588
9,Dayo Odeyingbo,ED,VANDERBILT,10.60,5.250,0.087673


In [82]:
dl_valid_data = NFL_DL_data2023.copy()
dls = dl_valid_data['player'].tolist()
dl22 = NFL_DL_data2022['player'].tolist()
dl21 = NFL_DL_data2021['player'].tolist()
dl19 = NFL_DL_data2019['player'].tolist()
for d in dls:
    if d in dl22:
        if d in dl21:
            if d in dl19:
                dl_valid_data.loc[dl_valid_data['player'] == d, 'pass_rush_win_rate'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'pass_rush_win_rate'].iloc[0] + NFL_DL_data2021.loc[NFL_DL_data2021['player'] == d, 'pass_rush_win_rate'].iloc[0] + NFL_DL_data2019.loc[NFL_DL_data2019['player'] == d, 'pass_rush_win_rate'].iloc[0])/4
                dl_valid_data.loc[dl_valid_data['player'] == d, 'prp'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'prp'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'prp'].iloc[0] + NFL_DL_data2021.loc[NFL_DL_data2021['player'] == d, 'prp'].iloc[0] + NFL_DL_data2019.loc[NFL_DL_data2019['player'] == d, 'prp'].iloc[0])/4
                dl_valid_data.loc[dl_valid_data['player'] == d, 'Pressure%'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'Pressure%'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'Pressure%'].iloc[0] + NFL_DL_data2021.loc[NFL_DL_data2021['player'] == d, 'Pressure%'].iloc[0] + NFL_DL_data2019.loc[NFL_DL_data2019['player'] == d, 'Pressure%'].iloc[0])/4  
            else:
                dl_valid_data.loc[dl_valid_data['player'] == d, 'pass_rush_win_rate'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'pass_rush_win_rate'].iloc[0] + NFL_DL_data2021.loc[NFL_DL_data2021['player'] == d, 'pass_rush_win_rate'].iloc[0])/3
                dl_valid_data.loc[dl_valid_data['player'] == d, 'prp'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'prp'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'prp'].iloc[0] + NFL_DL_data2021.loc[NFL_DL_data2021['player'] == d, 'prp'].iloc[0])/3
                dl_valid_data.loc[dl_valid_data['player'] == d, 'Pressure%'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'Pressure%'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'Pressure%'].iloc[0] + NFL_DL_data2021.loc[NFL_DL_data2021['player'] == d, 'Pressure%'].iloc[0])/3
        else:     
            dl_valid_data.loc[dl_valid_data['player'] == d, 'pass_rush_win_rate'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'pass_rush_win_rate'].iloc[0])/2
            dl_valid_data.loc[dl_valid_data['player'] == d, 'prp'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'prp'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'prp'].iloc[0])/2
            dl_valid_data.loc[dl_valid_data['player'] == d, 'Pressure%'] = (dl_valid_data.loc[dl_valid_data['player'] == d, 'Pressure%'].iloc[0] + NFL_DL_data2022.loc[NFL_DL_data2022['player'] == d, 'Pressure%'].iloc[0])/2        
dl_valid_data

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
0,Aaron Donald,DI,LA,20.075000,7.850000,0.129863
1,Adam Butler,DI,LV,9.200000,4.900000,0.077348
4,Alim McNeill,DI,DET,10.200000,4.950000,0.084640
6,Arik Armstead,DI,SF,15.200000,7.500000,0.125813
7,Austin Johnson,DI,LAC,2.500000,2.500000,0.047904
8,B.J. Hill,DI,CIN,8.566667,4.800000,0.078780
12,Bryan Bresee,DI,NO,7.300000,4.900000,0.080311
17,Calijah Kancey,DI,TB,9.500000,6.100000,0.101580
21,Chris Jones,DI,KC,16.950000,8.475000,0.137578
22,Christian Barmore,DI,NE,13.300000,6.200000,0.102510


In [83]:
dl_train_data = dl_college_data2018.copy()
dlList = dl_train_data['player'].tolist()
temp1 = dl_college_data2019.copy()
temp2 = dl_college_data2021.copy()
temp3 = dl_college_data2021.copy()
temp1 = temp1[~temp1['player'].isin(dlList)]
dl_train_data = pd.concat([dl_train_data, temp1]).drop_duplicates(keep=False)
tempDL = temp1['player'].tolist()
collegeDL2019 = dl_college_data2019.copy()
collegeDL2019 = collegeDL2019[~collegeDL2019['player'].isin(tempDL)]
dlList = dl_train_data['player'].tolist()
temp2 = temp2[temp2['player'].isin(dlList)]
tempDL = temp2['player'].tolist()
collegeDL2021 = dl_college_data2021.copy()
collegeDL2021 = collegeDL2021[~collegeDL2021['player'].isin(tempDL)]
dl_train_data = pd.concat([dl_train_data, temp2]).drop_duplicates(keep=False)
dlList = dl_train_data['player'].tolist()
temp3 = temp3[~temp3['player'].isin(dlList)]
dl_train_data = pd.concat([dl_train_data, temp3]).drop_duplicates(keep=False)
tempDL = temp3['player'].tolist()
collegeDL2022 = dl_college_data2022.copy()
collegeDL2022 = collegeDL2022[~collegeDL2022['player'].isin(tempDL)]
dl_train_data.reset_index(drop=True, inplace=True)
dl22 = collegeDL2022['player'].tolist()
dl21 = collegeDL2021['player'].tolist()
dl19 = collegeDL2019['player'].tolist()
dls = dl_train_data['player'].tolist()
for d in dls:
    if d in dl22:
        if d in dl21:
            if d in dl19:
                dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'] = (dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'pass_rush_win_rate'].iloc[0])/4
                dl_train_data.loc[dl_train_data['player'] == d, 'prp'] = (dl_train_data.loc[dl_train_data['player'] == d, 'prp'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'prp'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'prp'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'prp'].iloc[0])/4
                dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'] = (dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'Pressure%'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'Pressure%'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'Pressure%'].iloc[0])/4
            else:
                dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'] = (dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'pass_rush_win_rate'].iloc[0])/3
                dl_train_data.loc[dl_train_data['player'] == d, 'prp'] = (dl_train_data.loc[dl_train_data['player'] == d, 'prp'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'prp'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'prp'].iloc[0])/3
                dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'] = (dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'Pressure%'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'Pressure%'].iloc[0])/3
        else:
            dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'] = (dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'pass_rush_win_rate'].iloc[0])/2
            dl_train_data.loc[dl_train_data['player'] == d, 'prp'] = (dl_train_data.loc[dl_train_data['player'] == d, 'prp'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'prp'].iloc[0])/2
            dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'] = (dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'].iloc[0] + collegeDL2022.loc[collegeDL2022['player'] == d, 'Pressure%'].iloc[0])/2    
    elif d in dl21:
        if d in dl19:
            dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'] = (dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'pass_rush_win_rate'].iloc[0])/3
            dl_train_data.loc[dl_train_data['player'] == d, 'prp'] = (dl_train_data.loc[dl_train_data['player'] == d, 'prp'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'prp'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'prp'].iloc[0])/3
            dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'] = (dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'Pressure%'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'Pressure%'].iloc[0])/3
        else:
            dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'] = (dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'pass_rush_win_rate'].iloc[0])/2
            dl_train_data.loc[dl_train_data['player'] == d, 'prp'] = (dl_train_data.loc[dl_train_data['player'] == d, 'prp'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'prp'].iloc[0])/2
            dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'] = (dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'].iloc[0] + collegeDL2021.loc[collegeDL2021['player'] == d, 'Pressure%'].iloc[0])/2
    elif d in dl19:
        dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'] = (dl_train_data.loc[dl_train_data['player'] == d, 'pass_rush_win_rate'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'pass_rush_win_rate'].iloc[0])/2
        dl_train_data.loc[dl_train_data['player'] == d, 'prp'] = (dl_train_data.loc[dl_train_data['player'] == d, 'prp'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'prp'].iloc[0])/2
        dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'] = (dl_train_data.loc[dl_train_data['player'] == d, 'Pressure%'].iloc[0] + collegeDL2019.loc[collegeDL2019['player'] == d, 'Pressure%'].iloc[0])/2
    else:
        pass

dl_train_data = dl_train_data.drop_duplicates(subset=['player'])
dl_train_data = dl_train_data.sort_values(by='player')
dl_train_data.reset_index(drop=True, inplace=True)
dl_train_data

,player,position,team_name,pass_rush_win_rate,prp,Pressure%
0,Alim McNeill,DI,NC STATE,8.500,5.000,0.075799
1,Bryan Bresee,DI,CLEMSON,5.200,5.700,0.100000
2,Calijah Kancey,DI,PITTSBURGH,10.900,6.700,0.098684
3,Christian Barmore,DI,ALABAMA,16.100,10.500,0.171642
4,Christian Wilkins,DI,CLEMSON,16.900,8.500,0.134328
5,Derrick Brown,DI,AUBURN,12.400,5.600,0.090390
6,Devonte Wyatt,DI,GEORGIA,10.675,5.625,0.106638
7,Dexter Lawrence,DI,CLEMSON,12.800,7.000,0.128631
8,Dre'Mont Jones,DI,OHIO STATE,13.400,7.100,0.109181
9,Ed Oliver,DI,HOUSTON,14.200,5.700,0.097015


In [84]:
wrs22 = wr_train_data['Player'].tolist()
rbs22 = rb_train_data['player'].tolist()
cbs22 = cb_train_data['Player'].tolist()
ols22 = ol_train_data['player'].tolist()
edges22 = edge_train_data['player'].tolist()
dl22 = dl_train_data['player'].tolist()
wr_valid_data = wr_valid_data[wr_valid_data['Player'].isin(wrs22)]
rb_valid_data = rb_valid_data[rb_valid_data['player'].isin(rbs22)]
cb_valid_data = cb_valid_data[cb_valid_data['Player'].isin(cbs22)]
ol_valid_data = ol_valid_data[ol_valid_data['player'].isin(ols22)]
edge_valid_data = edge_valid_data[edge_valid_data['player'].isin(edges22)]
dl_valid_data = dl_valid_data[dl_valid_data['player'].isin(dl22)]

In [85]:
X = wr_train_data[['Yards/Rec']]
y = wr_valid_data[['Yards/Rec']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
wrYards_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
wrYards_model.fit(X_train, y_train)
y_test_pred = wrYards_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.65999e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.42903e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.30376e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.87328e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Il

Validation RMSE: 1.4923050314810167
Validation R2 Score -0.13731616492175114
    Yards/Rec
18      19.10
12      16.35
22      20.80
15      15.60
7       15.40
20      13.75
    Yards/Rec
44  14.425000
30  14.250000
25  14.066667
37  10.733333
20  11.700000
21  12.550000
[[13.68649597]
 [13.73580022]
 [13.81727012]
 [13.65887625]
 [13.62428611]
 [13.02768106]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [86]:
X = wr_train_data[['YAC/Rec']]
y = wr_valid_data[['YAC/Rec']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
wrYAC_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
wrYAC_model.fit(X_train, y_train)
y_test_pred = wrYAC_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 1.0630912442441462
Validation R2 Score 0.05245815077774085
    YAC/Rec
18     6.30
12     7.35
22     8.00
15     5.35
7      9.00
20     5.90
     YAC/Rec
44  4.375000
30  6.000000
25  6.133333
37  4.066667
20  4.575000
21  3.000000
[[4.90706304]
 [5.01164135]
 [5.0763803 ]
 [4.81244457]
 [5.17597869]
 [4.86722368]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [87]:
X = wr_train_data[['CTC%']]
y = wr_valid_data[['CTC%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
ctc_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
ctc_model.fit(X_train, y_train)
y_test_pred = ctc_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.31981e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.27449e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.15945e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.41108e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Il

Validation RMSE: 16.183496167558005
Validation R2 Score -0.9112690215419084
     CTC%
18  53.30
12  58.35
22  54.20
15  46.90
7    0.00
20  52.70
         CTC%
44  57.150000
30  25.200000
25  39.266667
37  55.966667
20  36.475000
21  52.650000
[[36.36848742]
 [40.79660554]
 [36.78463381]
 [37.76451321]
 [19.51580884]
 [36.18139722]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [88]:
X = wr_train_data[['Drop%']]
y = wr_valid_data[['Drop%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
drop_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
drop_model.fit(X_train, y_train)
y_test_pred = drop_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 1.4915975882970178
Validation R2 Score -0.31565074052144704
    Drop%
18   6.10
12   7.05
22   5.40
15   2.75
7   14.00
20   6.85
       Drop%
44  5.475000
30  7.500000
25  6.766667
37  4.866667
20  5.600000
21  3.450000
[[6.01582715]
 [6.17252116]
 [5.90036841]
 [5.4632746 ]
 [7.31886153]
 [6.13953295]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [89]:
X = wr_train_data[['ATR']]
y = wr_valid_data[['ATR']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
atr_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
atr_model.fit(X_train, y_train)
y_test_pred = atr_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 0.06435640223068662
Validation R2 Score -0.6182174469451829
         ATR
18  0.161290
12  0.232520
22  0.300000
15  0.122465
7   0.232558
20  0.296429
         ATR
44  0.110766
30  0.166394
25  0.180833
37  0.066216
20  0.201377
21  0.084541
[[0.13134503]
 [0.17644018]
 [0.22745363]
 [0.11018901]
 [0.17646656]
 [0.22453561]]


In [90]:
X = wr_train_data[['Target%']]
y = wr_valid_data[['Target%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
wrTargets_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
wrTargets_model.fit(X_train, y_train)
y_test_pred = wrTargets_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 0.016139803727613575
Validation R2 Score -0.01939636197497463
     Target%
18  0.151408
12  0.274132
22  0.257895
15  0.225309
7   0.250784
20  0.261470
     Target%
44  0.191410
30  0.190626
25  0.218644
37  0.221795
20  0.219009
21  0.233559
[[0.21020472]
 [0.21025033]
 [0.21024429]
 [0.21023218]
 [0.21024165]
 [0.21024562]]


In [91]:
prospectWRs = wrs_predict_data['Player'].tolist()
teams = wrs_predict_data['Team'].tolist()
X_yards = wrs_predict_data[['Yards/Rec']]
wrYards_predict = wrYards_model.predict(X_yards)
X_yac = wrs_predict_data[['YAC/Rec']]
wrYAC_predict = wrYAC_model.predict(X_yac)
X_ctc = wrs_predict_data[['CTC%']]
ctc_predict = ctc_model.predict(X_ctc)
X_drop = wrs_predict_data[['Drop%']]
drop_predict = drop_model.predict(X_drop)
X_atr = wrs_predict_data[['ATR']]
atr_predict = atr_model.predict(X_atr)
X_targets = wrs_predict_data[['Target%']]
wrTargets_predict = wrTargets_model.predict(X_targets)
pos = ['WR'] * 14
wrYards_predict = wrYards_predict.tolist()
wrYAC_predict = wrYAC_predict.tolist()
ctc_predict = ctc_predict.tolist()
atr_predict = atr_predict.tolist()
drop_predict = drop_predict.tolist()
wrTargets_predict = wrTargets_predict.tolist()
wrYards = []
wrYAC = []
ctc = []
atr = []
drop = []
wrTargets = []
for l in wrYards_predict:
    l[0] = round(l[0], 2)
    wrYards.extend(l)
for l in wrYAC_predict:
    l[0] = round(l[0], 2)
    wrYAC.extend(l)
for l in ctc_predict:
    l[0] = round(l[0], 2)
    ctc.extend(l)
for l in atr_predict:
    l[0] = round(l[0], 2)
    atr.extend(l)
for l in drop_predict:
    l[0] = round(l[0], 2)
    drop.extend(l)
for l in wrTargets_predict:
    l[0] = round(l[0], 2)
    wrTargets.extend(l)
wr_data = pd.DataFrame({'Player' : prospectWRs, 'Position' : pos, 'Team' : teams, 'Yards/Rec' : wrYards, 'YAC/Rec' : wrYAC, 'Target%' : wrTargets, 'CTC%' : ctc, 'Drop%' : drop, 'ATR' : atr})
wr_data

,Player,Position,Team,Yards/Rec,YAC/Rec,Target%,CTC%,Drop%,ATR
0,Adonai Mitchell,WR,TEXAS,13.66,4.58,0.21,52.27,5.37,0.10
1,Brenden Rice,WR,USC,13.74,4.80,0.21,54.39,5.72,0.15
2,Brian Thomas Jr.,WR,LSU,13.72,4.89,0.21,40.73,6.28,0.14
3,Ja'Lynn Polk,WR,WASHINGTON,13.75,4.72,0.21,37.29,6.13,0.14
4,Javon Baker,WR,UCF,14.45,5.02,0.21,38.31,7.02,0.12
5,Keon Coleman,WR,FLORIDA ST,13.11,4.90,0.21,52.27,5.01,0.17
6,Malachi Corley,WR,W KENTUCKY,12.36,5.14,0.21,53.57,6.23,0.14
7,Malik Nabers,WR,LSU,13.72,4.95,0.21,37.28,5.92,0.27
8,Marvin Harrison Jr.,WR,OHIO STATE,13.72,4.92,0.21,41.12,6.36,0.09
9,Ricky Pearsall,WR,FLORIDA,13.48,4.86,0.21,36.20,5.50,0.12


In [92]:
X = rb_train_data[['elusive_rating']]
y = rb_valid_data[['elusive_rating']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
elu_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
elu_model.fit(X_train, y_train)
y_test_pred = elu_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.8969e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.57949e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.37404e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.21966e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill

Validation RMSE: 11.64717145346416
Validation R2 Score -0.05403465864140644
    elusive_rating
9            89.75
11           96.00
0            80.25
    elusive_rating
18            70.7
25            59.7
0             43.1
[[60.39250288]
 [60.30905239]
 [60.43073475]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [93]:
X = rb_train_data[['breakaway_percent']]
y = rb_valid_data[['breakaway_percent']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
brk_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
brk_model.fit(X_train, y_train)
y_test_pred = brk_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.16807e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.3795e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.16415e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.38274e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill

Validation RMSE: 10.644625081653222
Validation R2 Score 0.06994109391984404
    breakaway_percent
9               49.45
11              40.75
0               35.90
    breakaway_percent
18               38.6
25               22.8
0                11.7
[[26.29710198]
 [25.56141915]
 [25.15129711]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [94]:
X = rb_train_data[['explosive']]
y = rb_valid_data[['explosive']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
exp_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
exp_model.fit(X_train, y_train)
y_test_pred = exp_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.47483e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.35262e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.89169e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.34217e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Il

Validation RMSE: 8.860446407741094
Validation R2 Score -0.17370032375426647
    explosive
9        21.0
11       53.5
0        32.5
    explosive
18         27
25         23
0           8
[[19.8932337 ]
 [24.49094073]
 [21.52011465]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [95]:
X = rb_train_data[['ypa']]
y = rb_valid_data[['ypa']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
rbYPA_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
rbYPA_model.fit(X_train, y_train)
y_test_pred = rbYPA_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 0.7683771491326502
Validation R2 Score -0.08885880938175594
     ypa
9   5.75
11  6.70
0   5.15
    ypa
18  5.2
25  4.4
0   3.4
[[4.20553181]
 [4.14165715]
 [4.24587371]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [96]:
X = rb_train_data[['ATR']]
y = rb_valid_data[['ATR']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
rbATR_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
rbATR_model.fit(X_train, y_train)
y_test_pred = rbATR_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 0.04904080427896024
Validation R2 Score -0.391025516483511
         ATR
9   0.247558
11  0.241935
0   0.245017
         ATR
18  0.219780
25  0.171598
0   0.117978
[[0.19579136]
 [0.19578469]
 [0.19578834]]


In [97]:
X = rb_train_data[['YCO']]
y = rb_valid_data[['YCO']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
yco_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
yco_model.fit(X_train, y_train)
y_test_pred = yco_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 0.2278150101666593
Validation R2 Score -0.27133969164105665
         YCO
9   3.301445
11  4.035842
0   3.327608
         YCO
18  3.120879
25  3.118343
0   2.691011
[[2.96535317]
 [2.87795822]
 [2.96253615]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [98]:
prospectRBs = rb_predict_data['player'].tolist()
teams = rb_predict_data['team_name'].tolist()
X_brk = rb_predict_data[['breakaway_percent']]
brk_predict = brk_model.predict(X_brk)
X_elu = rb_predict_data[['elusive_rating']]
elu_predict = elu_model.predict(X_elu)
X_exp = rb_predict_data[['explosive']]
exp_predict = exp_model.predict(X_exp)
X_ypa = rb_predict_data[['ypa']]
rbYPA_predict = rbYPA_model.predict(X_ypa)
X_atr = rb_predict_data[['ATR']]
rbATR_predict = rbATR_model.predict(X_atr)
X_yco = rb_predict_data[['YCO']]
yco_predict = yco_model.predict(X_yco)
pos = ['RB'] * 15
brk_predict = brk_predict.tolist()
elu_predict = elu_predict.tolist()
exp_predict = exp_predict.tolist()
rbYPA_predict = rbYPA_predict.tolist()
rbATR_predict = rbATR_predict.tolist()
yco_predict = yco_predict.tolist()
brk = []
elu = []
exp = []
rbYPA = []
rbATR = []
yco = []
for l in brk_predict:
    l[0] = round(l[0], 2)
    brk.extend(l)
for l in elu_predict:
    l[0] = round(l[0], 2)
    elu.extend(l)
for l in exp_predict:
    l[0] = np.ceil(l[0]).astype(int)
    exp.extend(l)
for l in rbYPA_predict:
    l[0] = round(l[0], 2)
    rbYPA.extend(l)
for l in rbATR_predict:
    l[0] = round(l[0], 2)
    rbATR.extend(l)
for l in yco_predict:
    l[0] = np.ceil(l[0]).astype(int)
    yco.extend(l)
rb_data = pd.DataFrame({'Player' : prospectRBs, 'Position' : pos, 'Team' : teams, 'BRK%' : brk, 'Elusive Rating' : elu, 'EXP' : exp, 'YPA' : rbYPA, 'ATR' : rbATR, 'YCO' : yco})
rb_data

,Player,Position,Team,BRK%,Elusive Rating,EXP,YPA,ATR,YCO
0,Audric Estime,RB,NOTRE DAME,25.84,54.96,23,4.16,0.2,3
1,Blake Corum,RB,MICHIGAN,24.70,60.44,20,4.27,0.2,4
2,Blake Watson,RB,MEMPHIS,25.84,60.08,22,4.20,0.2,3
3,Braelon Allen,RB,WISCONSIN,25.53,60.23,21,4.23,0.2,3
4,Bucky Irving,RB,OREGON,25.40,50.97,22,4.16,0.2,3
5,Carson Steele,RB,UCLA,24.01,60.19,21,4.25,0.2,3
6,Cody Schrader,RB,MISSOURI,25.41,60.44,23,4.19,0.2,3
7,Daijun Edwards,RB,GEORGIA,25.41,60.44,21,4.24,0.2,4
8,Dillon Johnson,RB,WASHINGTON,25.37,60.44,21,4.22,0.2,3
9,Emani Bailey,RB,TCU,25.83,58.27,23,4.23,0.2,3


In [99]:
X = cb_train_data[['Yards/Rec']]
y = cb_valid_data[['Yards/Rec']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
cbYards_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
cbYards_model.fit(X_train, y_train)
y_test_pred = cbYards_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.98072e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.92924e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.09557e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.1079e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill

Validation RMSE: 2.7805991482175965
Validation R2 Score 0.02046052667957321
    Yards/Rec
31       8.35
15      10.45
26      11.15
17      10.50
8       12.05
9       13.80
19      13.10
    Yards/Rec
71  10.650000
38   7.800000
60  11.366667
40  12.800000
21  17.500000
26  11.700000
45   9.700000
[[10.93910004]
 [11.27189088]
 [11.38282116]
 [11.27981447]
 [11.52544581]
 [11.80277152]
 [11.69184124]]


In [100]:
X = cb_train_data[['YAC%']]
y = cb_valid_data[['YAC%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
cbYAC_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
cbYAC_model.fit(X_train, y_train)
y_test_pred = cbYAC_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 1.4369053703509465
Validation R2 Score 0.030226825663793644
        YAC%
31  2.859375
15  4.005506
26  4.578005
17  3.863636
8   3.157407
9   4.242857
19  2.128205
        YAC%
71  4.464056
38  4.600000
60  3.401609
40  2.680851
21  7.710526
26  4.883673
45  4.500000
[[4.5409113 ]
 [4.38673424]
 [4.3097218 ]
 [4.40581843]
 [4.50082009]
 [4.35480579]
 [4.639268  ]]


In [101]:
X = cb_train_data[['Target Rate']]
y = cb_valid_data[['Target Rate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
cbTargets_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
cbTargets_model.fit(X_train, y_train)
y_test_pred = cbTargets_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 0.02142413609097618
Validation R2 Score -0.4261692328842692
    Target Rate
31     0.125527
15     0.144159
26     0.094822
17     0.130814
8      0.151474
9      0.144268
19     0.171429
    Target Rate
71     0.114003
38     0.094340
60     0.130734
40     0.146998
21     0.127413
26     0.140333
45     0.148206
[[0.14057186]
 [0.14057302]
 [0.14056995]
 [0.14057219]
 [0.14057348]
 [0.14057303]
 [0.14057472]]


In [102]:
X = cb_train_data[['FI%']]
y = cb_valid_data[['FI%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
fi_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
fi_model.fit(X_train, y_train)
y_test_pred = fi_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.36974e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.56827e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.35582e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=3.23808e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Il

Validation RMSE: 4.071431017785868
Validation R2 Score -0.21105864229918891
      FI%
31  14.50
15  15.45
26  15.55
17  15.60
8   17.60
9   20.20
19  18.20
      FI%
71  11.25
38  18.00
60  12.70
40   8.50
21  10.60
26   6.95
45   6.30
[[11.53295454]
 [11.67262617]
 [11.68732844]
 [11.69467958]
 [11.9887251 ]
 [12.37098429]
 [12.07693876]]


In [103]:
X = cb_train_data[['MIS%']]
y = cb_valid_data[['MIS%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
mis_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
mis_model.fit(X_train, y_train)
y_test_pred = mis_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.02539e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.95926e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.98474e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.22978e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Il

Validation RMSE: 3.6942989598221163
Validation R2 Score -0.2969378668742566
     MIS%
31   4.35
15   8.10
26   5.95
17  20.00
8    9.20
9    6.30
19   9.30
         MIS%
71  10.600000
38  16.700000
60   7.433333
40  14.600000
21  10.000000
26   7.600000
45  13.000000
[[12.22248742]
 [11.63168928]
 [11.97041355]
 [ 9.75688987]
 [11.4583885 ]
 [11.91527239]
 [11.44263388]]


In [104]:
prospectCBs = cb_predict_data['Player'].tolist()
teams = cb_predict_data['Team'].tolist()
X_yards = cb_predict_data[['Yards/Rec']]
cbYards_predict = cbYards_model.predict(X_yards)
X_yac = cb_predict_data[['YAC%']]
cbYAC_predict = cbYAC_model.predict(X_yac)
X_fi = cb_predict_data[['FI%']]
fi_predict = fi_model.predict(X_fi)
X_mis = cb_predict_data[['MIS%']]
mis_predict = mis_model.predict(X_mis)
X_targets = cb_predict_data[['Target Rate']]
cbTargets_predict = cbTargets_model.predict(X_targets)
pos = ['CB'] * 18
cbYards_predict = cbYards_predict.tolist()
cbYAC_predict = cbYAC_predict.tolist()
fi_predict = fi_predict.tolist()
mis_predict = mis_predict.tolist()
cbTargets_predict = cbTargets_predict.tolist()
cbYards = []
cbYAC = []
fi = []
mis = []
cbTargets = []
for l in cbYards_predict:
    l[0] = round(l[0], 2)
    cbYards.extend(l)
for l in cbYAC_predict:
    l[0] = round(l[0], 2)
    cbYAC.extend(l)
for l in fi_predict:
    l[0] = round(l[0], 2)
    fi.extend(l)
for l in mis_predict:
    l[0] = round(l[0], 2)
    mis.extend(l)
for l in cbTargets_predict:
    l[0] = round(l[0], 2)
    cbTargets.extend(l)
cb_data = pd.DataFrame({'Player' : prospectCBs, 'Position' : pos, 'Team' : teams, 'Yards/Rec' : cbYards, 'YAC%' : cbYAC, 'Target%' : cbTargets, 'FI%' : fi, 'MIS%' : mis})
cb_data

,Player,Position,Team,Yards/Rec,YAC%,Target%,FI%,MIS%
0,Andru Phillips,CB,KENTUCKY,11.49,4.36,0.14,10.49,9.91
1,Caelen Carson,CB,WAKE,11.53,4.38,0.14,11.00,10.04
2,Cooper DeJean,CB,IOWA,11.15,4.63,0.14,11.31,10.94
3,D.J. James,CB,AUBURN,11.52,4.47,0.14,13.64,11.66
4,Ennis Rakestraw Jr.,CB,MISSOURI,11.34,4.18,0.14,10.97,11.60
5,Jarvis Brownlee Jr.,CB,LOUISVILLE,11.98,4.66,0.14,11.30,11.85
6,Josh Newton,CB,TCU,11.53,4.28,0.14,12.22,10.94
7,Kalen King,CB,PENN STATE,11.45,4.35,0.14,10.47,10.58
8,Kamari Lassiter,CB,GEORGIA,10.96,4.59,0.14,13.49,12.28
9,Khyree Jackson,CB,OREGON,11.26,4.20,0.14,13.08,12.03


In [105]:
X = ol_train_data[['grades_pass_block']]
y = ol_valid_data[['grades_pass_block']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
pblk_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
pblk_model.fit(X_train, y_train)
y_test_pred = pblk_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.06047e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=8.6193e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.00692e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=8.33556e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill

Validation RMSE: 10.807717966970719
Validation R2 Score -0.15923613621297927
    grades_pass_block
4               49.30
63              70.45
18              90.60
0               80.70
28              85.60
72              77.40
10              85.40
34              78.40
12              82.95
55              62.75
65              67.20
31              78.60
9               91.00
45              75.20
5               84.25
     grades_pass_block
8            38.900000
130          56.833333
34           70.850000
0            60.700000
54           51.825000
147          55.700000
19           52.100000
74           61.800000
21           61.833333
114          74.100000
134          66.100000
68           70.950000
14           42.600000
95           71.200000
9            60.700000
[[53.90194427]
 [60.66402503]
 [67.10638566]
 [63.94115637]
 [65.50778501]
 [62.88607994]
 [65.44384098]
 [63.20580007]
 [64.66052666]
 [58.20218003]
 [59.62493461]
 [63.26974409]
 [67.23427371]
 [62.182

In [106]:
X = ol_train_data[['grades_run_block']]
y = ol_valid_data[['grades_run_block']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
rblk_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
rblk_model.fit(X_train, y_train)
y_test_pred = rblk_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=8.71446e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=8.08135e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.33488e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.63469e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Il

Validation RMSE: 10.534304270421796
Validation R2 Score -0.5397033420402526
    grades_run_block
4              73.30
63             65.30
18             85.40
0              66.70
28             84.00
72             77.05
10             73.80
34             70.80
12             63.20
55             63.05
65             82.70
31             85.70
9              79.00
45             88.40
5              69.20
     grades_run_block
8           58.500000
130         68.866667
34          56.725000
0           57.150000
54          66.700000
147         63.350000
19          62.900000
74          56.900000
21          50.233333
114         57.900000
134         73.200000
68          46.300000
14          47.600000
95          70.500000
9           45.800000
[[67.35536751]
 [63.97939024]
 [62.53013677]
 [65.0291524 ]
 [62.91152831]
 [66.48182648]
 [67.32654251]
 [67.05472474]
 [62.18531078]
 [62.05053848]
 [63.48911895]
 [62.49273894]
 [65.54143588]
 [63.17700374]
 [66.47636655]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.27225e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.27225e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [107]:
X = ol_train_data[['Pressure%']]
y = ol_valid_data[['Pressure%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
olPres_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
olPres_model.fit(X_train, y_train)
y_test_pred = olPres_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 0.01093889600020606
Validation R2 Score -0.2217992612905899
    Pressure%
4    0.017070
63   0.024775
18   0.006135
0    0.011568
28   0.013158
72   0.008478
10   0.006849
34   0.010005
12   0.009774
55   0.030305
65   0.016035
31   0.009421
9    0.007408
45   0.016410
5    0.018297
     Pressure%
8     0.042489
130   0.034810
34    0.029516
0     0.026639
54    0.027643
147   0.033293
19    0.037859
74    0.023256
21    0.023419
114   0.038778
134   0.034382
68    0.031861
14    0.064194
95    0.036298
9     0.024281
[[0.02982146]
 [0.03094119]
 [0.02823355]
 [0.02902239]
 [0.02925326]
 [0.02857376]
 [0.02833725]
 [0.02879541]
 [0.02876181]
 [0.03174501]
 [0.02967115]
 [0.02871065]
 [0.02841833]
 [0.02972557]
 [0.02999981]]


In [108]:
prospectOLs = ol_predict_data['player'].tolist()
pos = ol_predict_data['position']
teams = ol_predict_data['team_name'].tolist()
X_pblk = ol_predict_data[['grades_pass_block']]
pblk_predict = pblk_model.predict(X_pblk)
X_rblk = ol_predict_data[['grades_run_block']]
rblk_predict = rblk_model.predict(X_rblk)
X_pres = ol_predict_data[['Pressure%']]
olPres_predict = olPres_model.predict(X_pres)
pblk_predict = pblk_predict.tolist()
rblk_predict = rblk_predict.tolist()
olPres_predict = olPres_predict.tolist()
pblk = []
rblk = []
olPres = []
for l in pblk_predict:
    l[0] = round(l[0], 1)
    pblk.extend(l)
for l in rblk_predict:
    l[0] = round(l[0], 1)
    rblk.extend(l)
for l in olPres_predict:
    l[0] = round(l[0], 2)
    olPres.extend(l)
ol_data = pd.DataFrame({'Player' : prospectOLs, 'Position' : pos, 'Team' : teams, 'PBLK Grade' : pblk, 'RBLK Grade' : rblk, 'Pressure%' : olPres})
ol_data

,Player,Position,Team,PBLK Grade,RBLK Grade,Pressure%
0,Amarius Mims,T,GEORGIA,63.6,67.3,0.03
1,Andrew Raym,C,OKLAHOMA,58.5,67.1,0.03
2,Beaux Limmer,G,ARKANSAS,63.6,64.4,0.03
3,Blake Fisher,T,NOTRE DAME,61.4,67.0,0.03
4,Brandon Coleman,T,TCU,60.7,64.4,0.03
5,Caedan Wallace,T,PENN STATE,57.6,56.5,0.03
6,Christian Haynes,G,UCONN,66.9,62.6,0.03
7,Cooper Beebe,G,KANSAS ST,66.8,66.2,0.03
8,Delmar Glaze,T,MARYLAND,65.5,58.9,0.03
9,Dominick Puni,G,KANSAS,62.3,65.2,0.03


In [109]:
X = edge_train_data[['Pressure%']]
y = edge_valid_data[['Pressure%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
edgePres_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
edgePres_model.fit(X_train, y_train)
y_test_pred = edgePres_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 0.022594499419611124
Validation R2 Score 0.4793551273736437
    Pressure%
27   0.119382
15   0.230435
23   0.200000
17   0.138782
8    0.075588
9    0.087673
     Pressure%
102   0.114865
61    0.139620
85    0.163871
66    0.090591
25    0.081928
31    0.079452
[[0.11706082]
 [0.14843488]
 [0.13964939]
 [0.12240762]
 [0.10519302]
 [0.10844007]]


In [110]:
X = edge_train_data[['pass_rush_win_rate']]
y = edge_valid_data[['pass_rush_win_rate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
edgeWin_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
edgeWin_model.fit(X_train, y_train)
y_test_pred = edgeWin_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.73682e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.75615e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.82231e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.69771e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Il

Validation RMSE: 2.891291330371129
Validation R2 Score 0.39397660108125454
    pass_rush_win_rate
27               14.30
15               27.80
23               29.90
17               16.35
8                 7.25
9                10.60
     pass_rush_win_rate
102           11.900000
61            14.675000
85            18.150000
66             8.050000
25             7.833333
31             9.700000
[[12.81654217]
 [15.75719905]
 [16.21463456]
 [13.26308636]
 [11.2808658 ]
 [12.01058436]]


In [111]:
X = edge_train_data[['prp']]
y = edge_valid_data[['prp']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 8
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
edgePRP_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
edgePRP_model.fit(X_train, y_train)
y_test_pred = edgePRP_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 8 folds for each of 28 candidates, totalling 224 fits
Validation RMSE: 1.1815485026266381
Validation R2 Score 0.5722446359883988
      prp
27   7.95
15  15.50
23  13.40
17   8.90
8    4.90
9    5.25
       prp
102   6.90
61    8.70
85   10.15
66    5.60
25    5.10
31    5.90
[[ 7.30949277]
 [10.0775335 ]
 [ 9.30761489]
 [ 7.65778929]
 [ 6.19127764]
 [ 6.31959741]]


In [112]:
prospectEDGEs = edge_predict_data['player'].tolist()
pos = edge_predict_data['position']
teams = edge_predict_data['team_name'].tolist()
X_edgeWin = edge_predict_data[['pass_rush_win_rate']]
edgeWin_predict = edgeWin_model.predict(X_edgeWin)
X_prp = edge_predict_data[['prp']]
edgePRP_predict = edgePRP_model.predict(X_prp)
X_pres = edge_predict_data[['Pressure%']]
edgePres_predict = edgePres_model.predict(X_pres)
edgeWin_predict = edgeWin_predict.tolist()
edgePRP_predict = edgePRP_predict.tolist()
edgePres_predict = edgePres_predict.tolist()
edgeWin = []
edgePRP = []
edgePres = []
for l in edgeWin_predict:
    l[0] = round(l[0], 2)
    edgeWin.extend(l)
for l in edgePRP_predict:
    l[0] = round(l[0], 2)
    edgePRP.extend(l)
for l in edgePres_predict:
    l[0] = round(l[0], 2)
    edgePres.extend(l)
edge_data = pd.DataFrame({'Player' : prospectEDGEs, 'Position' : pos, 'Team' : teams, 'Win%' : edgeWin, 'PRP' : edgePRP, 'Pressure%' : edgePres})
edge_data

,Player,Position,Team,Win%,PRP,Pressure%
0,Adisa Isaac,ED,PENN STATE,12.58,8.28,0.13
1,Austin Booker,ED,KANSAS,12.93,8.24,0.13
2,Bralen Trice,ED,WASHINGTON,13.84,7.69,0.13
3,Chop Robinson,ED,PENN STATE,14.25,7.99,0.13
4,Chris Braswell,ED,ALABAMA,13.58,8.90,0.13
5,Dallas Turner,ED,ALABAMA,14.10,8.76,0.14
6,Darius Robinson,ED,MISSOURI,13.45,7.69,0.12
7,Gabriel Murphy,ED,UCLA,13.62,7.99,0.13
8,Jared Verse,ED,FLORIDA ST,14.32,8.39,0.13
9,Jonah Elliss,ED,UTAH,13.56,7.47,0.12


In [113]:
X = dl_train_data[['pass_rush_win_rate']]
y = dl_valid_data[['pass_rush_win_rate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 4
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
dlWin_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
dlWin_model.fit(X_train, y_train)
y_test_pred = dlWin_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 4 folds for each of 28 candidates, totalling 112 fits
Validation RMSE: 1.053866111830708
Validation R2 Score 0.5509945125921976
    pass_rush_win_rate
0                  8.5
1                  5.2
5                 12.4
15                10.1
    pass_rush_win_rate
4            10.200000
12            7.300000
37           11.066667
87           11.200000
[[10.33920988]
 [ 9.31408111]
 [11.55072569]
 [10.83624201]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.47901e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=1.14534e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=9.72211e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=7.16901e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Il

In [114]:
X = dl_train_data[['prp']]
y = dl_valid_data[['prp']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 4
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
dlPRP_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
dlPRP_model.fit(X_train, y_train)
y_test_pred = dlPRP_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 4 folds for each of 28 candidates, totalling 112 fits
Validation RMSE: 1.5649626086316104
Validation R2 Score -5.762961736684745
     prp
0   5.00
1   5.70
5   5.60
15  5.65
         prp
4   4.950000
12  4.900000
37  4.333333
87  6.000000
[[6.29141847]
 [6.56813332]
 [6.54746488]
 [6.55855266]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:239: UserW

In [115]:
X = dl_train_data[['Pressure%']]
y = dl_valid_data[['Pressure%']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

degree_range = [1, 2, 3, 4, 5, 6, 20]  
alpha_range = [0.01, 0.1, 1, 10]  
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('ridge', Ridge())
])

param_grid = {
    'poly__degree': degree_range,
    'ridge__alpha': alpha_range
}
k_folds = 4
grid_search = GridSearchCV(pipeline, param_grid, cv=k_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_alpha = grid_search.best_params_['ridge__alpha']
dlPres_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_degree)),
    ('ridge', Ridge(alpha=best_alpha))
])
dlPres_model.fit(X_train, y_train)
y_test_pred = dlPres_model.predict(X_test)
val_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)
print("Validation RMSE:", val_rmse)
print("Validation R2 Score", r2_test)
print(X_test)
print(y_test)
print(y_test_pred)

Fitting 4 folds for each of 28 candidates, totalling 112 fits
Validation RMSE: 0.017470003833551066
Validation R2 Score -2.0561638545255208
    Pressure%
0    0.075799
1    0.100000
5    0.090390
15   0.092635
    Pressure%
4    0.084640
12   0.080311
37   0.075368
87   0.101905
[[0.09983472]
 [0.09992333]
 [0.09988814]
 [0.09989636]]


In [116]:
prospectDLs = dl_predict_data['player'].tolist()
pos = ['DL'] * 10
teams = dl_predict_data['team_name'].tolist()
X_dlWin = dl_predict_data[['pass_rush_win_rate']]
dlWin_predict = dlWin_model.predict(X_dlWin)
X_prp = dl_predict_data[['prp']]
dlPRP_predict = dlPRP_model.predict(X_prp)
X_pres = dl_predict_data[['Pressure%']]
dlPres_predict = dlPres_model.predict(X_pres)
dlWin_predict = dlWin_predict.tolist()
dlPRP_predict = dlPRP_predict.tolist()
dlPres_predict = dlPres_predict.tolist()
dlWin = []
dlPRP = []
dlPres = []
for l in dlWin_predict:
    l[0] = round(l[0], 2)
    dlWin.extend(l)
for l in dlPRP_predict:
    l[0] = round(l[0], 2)
    dlPRP.extend(l)
for l in dlPres_predict:
    l[0] = round(l[0], 2)
    dlPres.extend(l)
dl_data = pd.DataFrame({'Player' : prospectDLs, 'Position' : pos, 'Team' : teams, 'Win%' : dlWin, 'PRP' : dlPRP, 'Pressure%' : dlPres})
dl_data

,Player,Position,Team,Win%,PRP,Pressure%
0,Braden Fiske,DL,FLORIDA ST,10.53,6.59,0.1
1,Brandon Dorlus,DL,OREGON,11.05,6.23,0.1
2,Byron Murphy II,DL,TEXAS,13.79,5.49,0.1
3,DeWayne Carter,DL,DUKE,9.47,5.69,0.1
4,Jer'Zhan Newton,DL,ILLINOIS,12.48,6.44,0.1
5,Kris Jenkins,DL,MICHIGAN,11.08,6.35,0.1
6,Leonard Taylor III,DL,MIAMI FL,12.27,6.58,0.1
7,Mekhi Wingo,DL,LSU,9.94,5.80,0.1
8,Ruke Orhorhoro,DL,CLEMSON,10.40,6.32,0.1
9,T'Vondre Sweat,DL,TEXAS,12.51,6.59,0.1


In [117]:
qb_draft = qb_data.copy()
qb_draft['Yards'] = qb_draft['Yards'] / 300
qb_draft['YPA'] = qb_draft['YPA'] * 2
qb_draft['Accuracy (%)'] = qb_draft['Accuracy (%)'] / 7
qb_draft['ADOT'] = qb_draft['ADOT'] * 2
qb_draft['TWP%'] = qb_draft['TWP%'] * 4
qb_draft['DV'] = np.ceil(((5 * qb_draft['Yards']) + (2 * qb_draft['YPA'] * qb_draft['YPA'] * qb_draft['YPA'] / 100) + (qb_draft['Accuracy (%)'] * qb_draft['Accuracy (%)'] *  qb_draft['Accuracy (%)'] / 100) + ( qb_draft['ADOT'] * qb_draft['ADOT'] * qb_draft['ADOT'] / 100) - (4.99 * qb_draft['TWP%'] * qb_draft['TWP%'] / 10) - (0.5 * qb_draft['P2S%'])) / 1.8).astype(int) * 5  
qbs = qb_draft['Player'].tolist()
qbDV = qb_draft['DV'].tolist()
positions = qb_draft['Position'].tolist()
qb_draft

,Player,Position,Team,Yards,YPA,Accuracy (%),ADOT,P2S%,TWP%,DV
0,Bo Nix,QB,OREGON,12.760000,14.34,10.940000,16.18,13.94,13.68,220
1,Caleb Williams,QB,USC,12.816667,14.36,10.751429,16.42,18.54,13.40,230
2,Drake Maye,QB,N CAROLINA,12.846667,14.38,10.728571,16.56,18.64,13.56,225
3,J.J. McCarthy,QB,MICHIGAN,11.586667,14.36,10.765714,16.50,16.38,13.36,215
4,Jayden Daniels,QB,LSU,12.360000,14.34,10.800000,16.40,21.15,13.72,205
5,Joe Milton III,QB,TENNESSEE,11.043333,14.28,10.594286,17.04,14.82,13.40,220
6,Michael Penix Jr.,QB,WASHINGTON,13.230000,14.36,10.695714,16.56,13.28,13.64,235
7,Spencer Rattler,QB,S CAROLINA,12.173333,14.42,10.805714,16.20,18.86,13.24,220


In [118]:
wr_draft = wr_data.copy()
wr_draft['YAC/Rec'] = wr_draft['YAC/Rec'] * 10
wr_draft['Target%'] = wr_draft['Target%'] * 50
wr_draft['CTC%'] = wr_draft['CTC%'] / 3
wr_draft['Drop%'] = wr_draft['Drop%'] * 2
wr_draft['ATR'] = wr_draft['ATR'] * 100
wr_draft['DV'] = np.ceil((wr_draft['YAC/Rec'] + wr_draft['Yards/Rec'] + wr_draft['Target%'] + wr_draft['CTC%'] - wr_draft['Drop%'] + wr_draft['ATR']) / 1.75).astype(int) * 3
wrDV = wr_draft['DV'].tolist()
wrs = wr_draft['Player'].tolist()
positions = positions + wr_draft['Position'].tolist()
wr_draft

,Player,Position,Team,Yards/Rec,YAC/Rec,Target%,CTC%,Drop%,ATR,DV
0,Adonai Mitchell,WR,TEXAS,13.66,45.8,10.5,17.423333,10.74,10.0,150
1,Brenden Rice,WR,USC,13.74,48.0,10.5,18.130000,11.44,15.0,162
2,Brian Thomas Jr.,WR,LSU,13.72,48.9,10.5,13.576667,12.56,14.0,153
3,Ja'Lynn Polk,WR,WASHINGTON,13.75,47.2,10.5,12.430000,12.26,14.0,147
4,Javon Baker,WR,UCF,14.45,50.2,10.5,12.770000,14.04,12.0,150
5,Keon Coleman,WR,FLORIDA ST,13.11,49.0,10.5,17.423333,10.02,17.0,168
6,Malachi Corley,WR,W KENTUCKY,12.36,51.4,10.5,17.856667,12.46,14.0,162
7,Malik Nabers,WR,LSU,13.72,49.5,10.5,12.426667,11.84,27.0,174
8,Marvin Harrison Jr.,WR,OHIO STATE,13.72,49.2,10.5,13.706667,12.72,9.0,144
9,Ricky Pearsall,WR,FLORIDA,13.48,48.6,10.5,12.066667,11.00,12.0,147


In [119]:
rb_draft = rb_data.copy()
rb_draft['BRK%'] = rb_draft['BRK%'] / 2
rb_draft['Elusive Rating'] = rb_draft['Elusive Rating'] / 4
rb_draft['YPA'] = rb_draft['YPA'] * 4
rb_draft['ATR'] = rb_draft['ATR'] * 5
rb_draft['YCO'] = rb_draft['YCO'] * 4
rb_draft['DV'] = np.ceil((rb_draft['BRK%'] + rb_draft['Elusive Rating'] + rb_draft['YPA'] + rb_draft['ATR'] + rb_draft['YCO'])).astype(int)
rbDv = rb_draft['DV'].tolist()
rbs = rb_draft['Player'].tolist()
positions = positions + rb_draft['Position'].tolist()
rb_draft

,Player,Position,Team,BRK%,Elusive Rating,EXP,YPA,ATR,YCO,DV
0,Audric Estime,RB,NOTRE DAME,12.920,13.7400,23,16.64,1.0,12,57
1,Blake Corum,RB,MICHIGAN,12.350,15.1100,20,17.08,1.0,16,62
2,Blake Watson,RB,MEMPHIS,12.920,15.0200,22,16.80,1.0,12,58
3,Braelon Allen,RB,WISCONSIN,12.765,15.0575,21,16.92,1.0,12,58
4,Bucky Irving,RB,OREGON,12.700,12.7425,22,16.64,1.0,12,56
5,Carson Steele,RB,UCLA,12.005,15.0475,21,17.00,1.0,12,58
6,Cody Schrader,RB,MISSOURI,12.705,15.1100,23,16.76,1.0,12,58
7,Daijun Edwards,RB,GEORGIA,12.705,15.1100,21,16.96,1.0,16,62
8,Dillon Johnson,RB,WASHINGTON,12.685,15.1100,21,16.88,1.0,12,58
9,Emani Bailey,RB,TCU,12.915,14.5675,23,16.92,1.0,12,58


In [120]:
cb_draft = cb_data.copy()
cb_draft['YAC%'] = cb_draft['YAC%'] * 3
cb_draft['Target%'] = cb_draft['Target%'] * 10
cb_draft['DV'] = np.ceil(( 12 / ((1 / cb_draft['Yards/Rec']) + (1 / cb_draft['Target%']) + (1 / cb_draft['YAC%']) + (1 / cb_draft['MIS%'])) + (2 * cb_draft['FI%']))).astype(int) * 3
cb_DV = cb_draft['DV'].tolist()
cbs = cb_draft['Player'].tolist()
positions = positions + cb_draft['Position'].tolist()
cb_draft

,Player,Position,Team,Yards/Rec,YAC%,Target%,FI%,MIS%,DV
0,Andru Phillips,CB,KENTUCKY,11.49,13.08,1.4,10.49,9.91,102
1,Caelen Carson,CB,WAKE,11.53,13.14,1.4,11.00,10.04,105
2,Cooper DeJean,CB,IOWA,11.15,13.89,1.4,11.31,10.94,108
3,D.J. James,CB,AUBURN,11.52,13.41,1.4,13.64,11.66,120
4,Ennis Rakestraw Jr.,CB,MISSOURI,11.34,12.54,1.4,10.97,11.60,105
5,Jarvis Brownlee Jr.,CB,LOUISVILLE,11.98,13.98,1.4,11.30,11.85,108
6,Josh Newton,CB,TCU,11.53,12.84,1.4,12.22,10.94,111
7,Kalen King,CB,PENN STATE,11.45,13.05,1.4,10.47,10.58,102
8,Kamari Lassiter,CB,GEORGIA,10.96,13.77,1.4,13.49,12.28,120
9,Khyree Jackson,CB,OREGON,11.26,12.60,1.4,13.08,12.03,117


In [121]:
ol_draft = ol_data.copy()
ol_draft['Pressure%'] = ol_draft['Pressure%'] * 40 
ol_draft['DV'] = np.ceil((ol_draft['PBLK Grade'] + ol_draft['RBLK Grade'] - ol_draft['Pressure%']) / 3).astype(int)
ol_draft.loc[ol_draft['Position'] == 'T', 'DV'] = ol_draft.loc[ol_draft['Position'] == 'T', 'DV'] * 3
ol_draft.loc[ol_draft['Position'] == 'G', 'DV'] = ol_draft.loc[ol_draft['Position'] == 'G', 'DV'] * 2
ol_draft.loc[ol_draft['Position'] == 'C', 'DV'] = ol_draft.loc[ol_draft['Position'] == 'C', 'DV'] * 2
ol_DV = ol_draft['DV'].tolist()
ols = ol_draft['Player'].tolist()
positions = positions + ol_draft['Position'].tolist()
ol_draft

,Player,Position,Team,PBLK Grade,RBLK Grade,Pressure%,DV
0,Amarius Mims,T,GEORGIA,63.6,67.3,1.2,132
1,Andrew Raym,C,OKLAHOMA,58.5,67.1,1.2,84
2,Beaux Limmer,G,ARKANSAS,63.6,64.4,1.2,86
3,Blake Fisher,T,NOTRE DAME,61.4,67.0,1.2,129
4,Brandon Coleman,T,TCU,60.7,64.4,1.2,126
5,Caedan Wallace,T,PENN STATE,57.6,56.5,1.2,114
6,Christian Haynes,G,UCONN,66.9,62.6,1.2,86
7,Cooper Beebe,G,KANSAS ST,66.8,66.2,1.2,88
8,Delmar Glaze,T,MARYLAND,65.5,58.9,1.2,126
9,Dominick Puni,G,KANSAS,62.3,65.2,1.2,86


In [122]:
edge_draft = edge_data.copy()
edge_draft['Win%'] = edge_draft['Win%'] * 1.5
edge_draft['PRP'] = edge_draft['PRP'] * 1.5
edge_draft['Pressure%'] = edge_draft['Pressure%'] * 150
edge_draft['DV'] = np.ceil(edge_draft['Win%'] + edge_draft['PRP'] + edge_draft['Pressure%']).astype(int) * 3
edge_DV = edge_draft['DV'].tolist()
edges = edge_draft['Player'].tolist()
positions = positions + edge_draft['Position'].tolist()
edge_draft

,Player,Position,Team,Win%,PRP,Pressure%,DV
0,Adisa Isaac,ED,PENN STATE,18.870,12.420,19.5,153
1,Austin Booker,ED,KANSAS,19.395,12.360,19.5,156
2,Bralen Trice,ED,WASHINGTON,20.760,11.535,19.5,156
3,Chop Robinson,ED,PENN STATE,21.375,11.985,19.5,159
4,Chris Braswell,ED,ALABAMA,20.370,13.350,19.5,162
5,Dallas Turner,ED,ALABAMA,21.150,13.140,21.0,168
6,Darius Robinson,ED,MISSOURI,20.175,11.535,18.0,150
7,Gabriel Murphy,ED,UCLA,20.430,11.985,19.5,156
8,Jared Verse,ED,FLORIDA ST,21.480,12.585,19.5,162
9,Jonah Elliss,ED,UTAH,20.340,11.205,18.0,150


In [123]:
dl_draft = dl_data.copy()
dl_draft['Win%'] = dl_draft['Win%'] 
dl_draft['PRP'] = dl_draft['PRP'] * 2 
dl_draft['Pressure%'] = dl_draft['Pressure%'] * 200
dl_draft['DV'] = np.ceil(dl_draft['Win%'] + dl_draft['PRP'] + dl_draft['Pressure%']).astype(int) * 2
dl_DV = dl_draft['DV'].tolist()
dls = dl_draft['Player'].tolist()
positions = positions + dl_draft['Position'].tolist()
dl_draft

,Player,Position,Team,Win%,PRP,Pressure%,DV
0,Braden Fiske,DL,FLORIDA ST,10.53,13.18,20.0,88
1,Brandon Dorlus,DL,OREGON,11.05,12.46,20.0,88
2,Byron Murphy II,DL,TEXAS,13.79,10.98,20.0,90
3,DeWayne Carter,DL,DUKE,9.47,11.38,20.0,82
4,Jer'Zhan Newton,DL,ILLINOIS,12.48,12.88,20.0,92
5,Kris Jenkins,DL,MICHIGAN,11.08,12.70,20.0,88
6,Leonard Taylor III,DL,MIAMI FL,12.27,13.16,20.0,92
7,Mekhi Wingo,DL,LSU,9.94,11.60,20.0,84
8,Ruke Orhorhoro,DL,CLEMSON,10.40,12.64,20.0,88
9,T'Vondre Sweat,DL,TEXAS,12.51,13.18,20.0,92


In [124]:
#Since Brock Bowers is the only tight end that was highly ranked, tight ends were omitted however Bowers was too good not to include so I gave him a DV of a high end wide receiever as his high value comes from being a high end wide reciever as a tight end.
draftees = qbs + wrs + rbs + cbs + ols + edges + dls
draftees_DV = qbDV + wrDV + rbDv + cb_DV + ol_DV + edge_DV + dl_DV
draftees.append('Brock Bowers')
draftees_DV.append(150)
positions.append('TE')
draftee_data = pd.DataFrame({'Player' : draftees, 'Position' : positions, 'DV' : draftees_DV})
draftee_data

,Player,Position,DV
0,Bo Nix,QB,220
1,Caleb Williams,QB,230
2,Drake Maye,QB,225
3,J.J. McCarthy,QB,215
4,Jayden Daniels,QB,205
...,...,...,...
108,Leonard Taylor III,DL,92
109,Mekhi Wingo,DL,84
110,Ruke Orhorhoro,DL,88
111,T'Vondre Sweat,DL,92


In [125]:
 draft_needs = {
    'CHI': ['QB', 'WR', 'DL', 'G', 'C'],
    'WAS': ['QB', 'G', 'T', 'ED', 'CB'],
    'NE': ['QB', 'WR', 'T', 'ED', 'RB'],
    'AZ': ['WR', 'C', 'G', 'DL', 'CB'],
    'LAC': ['WR', 'DL', 'CB', 'T', 'G', 'C', 'TE', 'RB'],
    'NYG': ['QB', 'WR', 'G', 'T', 'DL', 'CB', 'RB'],
    'TEN': ['WR', 'G', 'T', 'ED', 'CB', 'TE'],
    'ATL': ['DL','CB', 'ED', 'WR', 'TE'],
    'NYJ': ['WR', 'T', 'DL', 'TE', 'G', 'C'],
    'MIN': ['QB', 'G', 'DL', 'CB', 'C'],
    'DEN': ['QB', 'WR', 'C', 'ED', 'CB'],
    'LV': ['QB', 'G', 'T', 'DL', 'CB', 'RB', 'TE'],
    'NO': ['QB', 'WR', 'T', 'DL', 'G', 'TE', 'ED'],
    'IND': ['WR', 'CB', 'TE', 'ED', 'DL'],
    'SEA': ['G', 'ED', 'TE', 'QB', 'C'],
    'JAX': ['G', 'ED', 'CB', 'T', 'WR', 'TE'],
    'CIN': ['T', 'DL', 'WR', 'G', 'TE'],
    'LAR': ['QB', 'T', 'DL', 'CB', 'ED'],
    'PIT': ['WR', 'C', 'T', 'CB', 'G'],
    'MIA': ['G', 'T', 'DL', 'CB', 'ED'],
    'PHI': ['WR', 'CB', 'G', 'T', 'ED'],
    'DAL': ['WR', 'C', 'T', 'DL', 'CB', 'RB'],
    'GB': ['G', 'T', 'DL', 'CB', 'ED'],
    'TB': ['C', 'G', 'ED', 'CB', 'WR'],
    'BUF': ['WR', 'T', 'DL', 'CB', 'ED'],
    'DET': ['G', 'ED', 'CB', 'WR', 'T'],
    'BAL': ['WR', 'G', 'T', 'ED', 'CB'],
    'SF': ['G', 'T', 'CB', 'WR', 'TE'],
    'KC': ['WR', 'T', 'DL', 'ED', 'G'],
     'CAR' : ['WR', 'T', 'G', 'ED', 'CB']
}

In [126]:
#Mock Draft #1: The Day Before The Draft
draft_order = ['CHI', 'WAS', 'NE', 'AZ', 'LAC', 'NYG', 'TEN', 'ATL', 'CHI', 'NYJ', 'MIN', 'DEN', 'LV', 'NO', 'IND', 'SEA', 'JAX', 'CIN', 'LAR', 'PIT', 'MIA', 'PHI', 'MIN', 'DAL', 'GB', 'TB', 'AZ', 'BUF', 'DET', 'BAL', 'SF', 'KC']
counter = 1
team = []
draftee = []
pos = []
pick = []
predraft = draftee_data.copy()
draft_needs1 = copy.deepcopy(draft_needs)
taken = False
#Iterate through the draft
for t in draft_order:
    potential = []
    largest = 0
    #Iterate through each position
    for p in draft_needs1[t]:
        temp = predraft.copy()
        tempPos = temp['Position'].tolist()
        if p in tempPos:
            temp = temp[temp['Position'] == p]
            temp = temp.sort_values(by='DV', ascending=False)
            temp.reset_index(drop=True, inplace=True)
            name = temp['Player'].tolist()
            tempDV = temp['DV'].tolist()
            #Only add prospect if DV is larger than prospects already being considered.
            if tempDV[0] > largest and len(tempDV) > 0:
                l = [name[0], p, tempDV[0]]
                largest = tempDV[0]
                potential.append(l)
    #Select player with largest DV, remove them from the draftees dataframe, remove position from team needs
    for item in potential:
        if item[2] == largest:
            team.append(t)
            draftee.append(item[0])
            pos.append(item[1])
            pick.append(counter)
            counter = counter + 1
            predraft = predraft[predraft['Player'] != item[0]]
            draft_needs1[t].remove(item[1]) 
            break
#Create dataframe with draft data
mockDraft1 = pd.DataFrame({'Pick' : pick, 'Player' : draftee, 'Position' : pos, 'Team' : team})
mockDraft1

,Pick,Player,Position,Team
0,1,Michael Penix Jr.,QB,CHI
1,2,Caleb Williams,QB,WAS
2,3,Drake Maye,QB,NE
3,4,Malik Nabers,WR,AZ
4,5,Keon Coleman,WR,LAC
5,6,Bo Nix,QB,NYG
6,7,Laiatu Latu,ED,TEN
7,8,Dallas Turner,ED,ATL
8,9,Rome Odunze,WR,CHI
9,10,Brenden Rice,WR,NYJ


In [127]:
draft_order = ['CHI', 'WAS', 'NE', 'AZ', 'LAC', 'NYG', 'TEN', 'ATL', 'CHI', 'MIN', 'NYJ', 'DEN', 'LV', 'NO', 'IND', 'SEA', 'MIN', 'CIN', 'LAR', 'PIT', 'MIA', 'PHI', 'JAX', 'DET', 'GB', 'TB', 'AZ', 'KC', 'DAL', 'BAL', 'SF', 'CAR']
counter = 1
team = []
draftee = []
pos = []
pick = []
postdraft = draftee_data.copy()
draft_needs2 = copy.deepcopy(draft_needs)
taken = False
for t in draft_order:
    potential = []
    largest = 0
    for p in draft_needs2[t]:
        temp = postdraft.copy()
        tempPos = temp['Position'].tolist()
        if p in tempPos:
            temp = temp[temp['Position'] == p]
            temp = temp.sort_values(by='DV', ascending=False)
            temp.reset_index(drop=True, inplace=True)
            name = temp['Player'].tolist()
            tempDV = temp['DV'].tolist()
            if tempDV[0] > largest and len(tempDV) > 0:
                l = [name[0], p, tempDV[0]]
                largest = tempDV[0]
                potential.append(l)
    for item in potential:
        if item[2] == largest:
            team.append(t)
            draftee.append(item[0])
            pos.append(item[1])
            pick.append(counter)
            counter = counter + 1
            postdraft = postdraft[postdraft['Player'] != item[0]]
            draft_needs2[t].remove(item[1]) 
            break
mockDraft2 = pd.DataFrame({'Pick' : pick, 'Player' : draftee, 'Position' : pos, 'Team' : team})
mockDraft2

,Pick,Player,Position,Team
0,1,Michael Penix Jr.,QB,CHI
1,2,Caleb Williams,QB,WAS
2,3,Drake Maye,QB,NE
3,4,Malik Nabers,WR,AZ
4,5,Keon Coleman,WR,LAC
5,6,Bo Nix,QB,NYG
6,7,Laiatu Latu,ED,TEN
7,8,Dallas Turner,ED,ATL
8,9,Rome Odunze,WR,CHI
9,10,Joe Milton III,QB,MIN
